## Imports and constants

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as st
import statsmodels.api as sm

import os
import time
import random
import itertools

from copy import copy
from cycler import cycler
from datetime import date
from scipy.optimize import curve_fit
from statannot import add_stat_annotation

pd.set_option('mode.chained_assignment',None)

In [6]:
import re
dateFormatExcel = re.compile('\d{2}/\d{2}/\d{4}')
dateFormatOk = re.compile('\d{2}-\d{2}-\d{2}')

In [7]:
import matplotlib
import matplotlib.pyplot as plt

%matplotlib widget 
# %matplotlib inline
matplotlib.rcParams.update({'figure.autolayout': True})


SMALLER_SIZE = 10
SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALLER_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

# prop_cycle = plt.rcParams['axes.prop_cycle']
# colors = prop_cycle.by_key()['color']
my_default_color_list = ['#ff7f0e', '#1f77b4', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
my_default_color_cycle = cycler(color=my_default_color_list)
plt.rcParams['axes.prop_cycle'] = my_default_color_cycle

In [8]:
pairedPalette = sns.color_palette("tab20")
pairedPalette = pairedPalette.as_hex()
print(pairedPalette)
pairedPalette

['#1f77b4', '#aec7e8', '#ff7f0e', '#ffbb78', '#2ca02c', '#98df8a', '#d62728', '#ff9896', '#9467bd', '#c5b0d5', '#8c564b', '#c49c94', '#e377c2', '#f7b6d2', '#7f7f7f', '#c7c7c7', '#bcbd22', '#dbdb8d', '#17becf', '#9edae5']


['#1f77b4',
 '#aec7e8',
 '#ff7f0e',
 '#ffbb78',
 '#2ca02c',
 '#98df8a',
 '#d62728',
 '#ff9896',
 '#9467bd',
 '#c5b0d5',
 '#8c564b',
 '#c49c94',
 '#e377c2',
 '#f7b6d2',
 '#7f7f7f',
 '#c7c7c7',
 '#bcbd22',
 '#dbdb8d',
 '#17becf',
 '#9edae5']

In [9]:
clist = ['#1f77b4', '#aec7e8', '#ff7f0e', '#ffbb78', '#2ca02c', '#98df8a']
sns.color_palette(clist)

[(0.12156862745098039, 0.4666666666666667, 0.7058823529411765),
 (0.6823529411764706, 0.7803921568627451, 0.9098039215686274),
 (1.0, 0.4980392156862745, 0.054901960784313725),
 (1.0, 0.7333333333333333, 0.47058823529411764),
 (0.17254901960784313, 0.6274509803921569, 0.17254901960784313),
 (0.596078431372549, 0.8745098039215686, 0.5411764705882353)]

In [10]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, Range1d
from bokeh.transform import factor_cmap
from bokeh.palettes import Category10
from bokeh.layouts import gridplot
output_notebook()

Loading BokehJS ...

In [11]:
mainDir = "C://Users//JosephVermeil//Desktop//ActinCortexAnalysis"
# mainDir = "C://Users//josep//Desktop//ActinCortexAnalysis"
experimentalDataDir = os.path.join(mainDir, "Data_Experimental")
dataDir = os.path.join(mainDir, "Data_Analysis")
figDir = os.path.join(dataDir, "Figures")
todayFigDir = os.path.join(figDir, "Historique//" + str(date.today()))
timeSeriesDataDir = os.path.join(dataDir, "TimeSeriesData")

In [12]:
# Display all detected time Series Data Files.

# allTimeSeriesDataFiles = [f for f in os.listdir(timeSeriesDataDir) if (os.path.isfile(os.path.join(timeSeriesDataDir, f)) and f.endswith(".csv"))]
# allTimeSeriesDataFiles

Some generic useful subfunctions:

In [13]:
def get_R2(Y1, Y2):
    meanY = np.mean(Y1)
    meanYarray = meanY*np.ones(len(Y1))
    SST = np.sum((Y1-meanYarray)**2)
    SSE = np.sum((Y2-meanYarray)**2)
    R2 = SSE/SST
    return(R2)

def getDictAggMean(df):
    dictAggMean = {}
    for c in df.columns:
    #         t = df[c].dtype
    #         print(c, t)
            try :
                if np.array_equal(df[c], df[c].astype(bool)):
                    dictAggMean[c] = 'min'
                else:
                    try:
                        if not c.isnull().all():
                            np.mean(df[c])
                            dictAggMean[c] = 'mean'
                    except:
                        dictAggMean[c] = 'first'
            except:
                    dictAggMean[c] = 'first'
    return(dictAggMean)

def findFirst(x, A):
    idx = (A==x).view(bool).argmax()
    return(idx)

def fitLine(X, Y):
    X = sm.add_constant(X)
    model = sm.OLS(Y, X)
    results = model.fit()
    params = results.params # Y=a*X+b ; params[0] = b,  params[1] = a
#     print(dir(results))
#     R2 = results.rsquared
#     ci = results.conf_int(alpha=0.05)
#     CovM = results.cov_params()
#     p = results.pvalues

# This is how are computed conf_int:
#
#     bse = results.bse
#     dist = stats.t
#     alpha = 0.05
#     q = dist.ppf(1 - alpha / 2, results.df_resid)
#     params = results.params
#     lower = params - q * bse
#     upper = params + q * bse
#     print(lower, upper)
    
    return(results.params, results)

def archiveFig(fig, ax, name='auto', figDir = todayFigDir, figSubDir=''):
    if not os.path.exists(figDir):
        os.makedirs(figDir)
    
    saveDir = os.path.join(todayFigDir, figSubDir)
    if not os.path.exists(saveDir):
        os.makedirs(saveDir)
    
    if name != 'auto':
        fig.savefig(os.path.join(saveDir, name + '.png'))
    
    else:
        suptitle = fig._suptitle.get_text()
        if len(suptitle) > 0:
            name = suptitle
            fig.savefig(os.path.join(saveDir, name + '.png'))
        
        else:
            try:
                N = len(ax)
                ax = ax[0]
            except:
                N = 1
                ax = ax
                
            xlabel = ax.get_xlabel()
            ylabel = ax.get_ylabel()
            if len(xlabel) > 0 and len(ylabel) > 0:
                name = ylabel + ' Vs ' + xlabel
                if N > 1:
                    name = name + '___etc'
                fig.savefig(os.path.join(saveDir, name + '.png'))
            
            else:
                title = ax.get_title()
                if len(title) > 0:
                    if N > 1:
                        name = name + '___etc'
                    fig.savefig(os.path.join(saveDir, name + '.png'))
                
                else:
                    figNum = gcf().number
                    name = 'figure ' + str(figNum) 
                    fig.savefig(os.path.join(saveDir, name + '.png'))

Note: How to compute confidence intervals of fitted parameters with (1-alpha) confidence:

    0) from scipy import stats
    1) df = nb_pts - nb_parms ; se = diag(cov)**0.5
    2) Student t coefficient : q = stat.t.ppf(1 - alpha / 2, df)
    3) ConfInt = [params - q*se, params + q*se]

In [14]:
# Test of the fitLine function

# Npts = 10
# seed = 10
# std = 2

# np.random.seed(seed)
# X = np.array([i for i in range(Npts)])
# Y = np.array([i for i in range(Npts)])
# Y = Y + np.random.normal(0, std, Npts)
# p, R = fitLine(X, Y)

# Ypred = p[1]*X+p[0]
# fig, ax = plt.subplots(1,1)
# ax.plot(X, Y, 'bo')
# ax.plot(X, Ypred, 'r-', label='best fit')
# ax.legend(loc='best')
# fig.show()

In [15]:
# Test of the lmfit library // <examples/doc_model_gaussian.py>

# import matplotlib.pyplot as plt
# from lmfit import Model

# np.random.seed(seed)

# x = np.array([i for i in range(Npts)])
# y = np.array([i for i in range(Npts)])
# y = y + np.random.normal(0, std, Npts)


# # def gaussian(x, amp, cen, wid):
# #     """1-d gaussian: gaussian(x, amp, cen, wid)"""
# #     return (amp / (sqrt(2*pi) * wid)) * exp(-(x-cen)**2 / (2*wid**2))

# def linear(x, a, b):
#     """linear function"""
#     return (a*x + b)

# # gmodel = Model(gaussian)
# # result = gmodel.fit(y, x=x, amp=5, cen=5, wid=1)

# # print(result.fit_report())

# # plt.plot(x, y, 'bo')
# # plt.plot(x, result.init_fit, 'k--', label='initial fit')
# # plt.plot(x, result.best_fit, 'r-', label='best fit')
# # plt.legend(loc='best')
# # plt.show()

# lmodel = Model(linear)
# result = lmodel.fit(y, x=x, a=1.5, b=-1)

# print(result.fit_report())

# fig, ax = plt.subplots(1,1)
# ax.plot(x, y, 'bo')
# # ax.plot(x, result.init_fit, 'k--', label='initial fit')
# ax.plot(x, result.best_fit, 'r-', label='best fit')
# ax.legend(loc='best')
# fig.show()

# # print(dir(result))

In [16]:
# Test of the curve_fit function and subsequent computation of confidence intervals

# np.random.seed(seed)

# x = np.array([i for i in range(Npts)])
# y = np.array([i for i in range(Npts)])
# y = y + np.random.normal(0, std, Npts)

# def linear(x, a, b):
#     """linear function"""
#     return (a*x + b)


# initialParameters = [1.5, -1]
# #     print(initialParameters)

# # bounds on parameters - initial parameters must be within these
# lowerBounds = (-np.Inf, -np.Inf)
# upperBounds = (np.Inf, np.Inf)
# parameterBounds = [lowerBounds, upperBounds]

# # fittedParameters, pcov = curve_fit(chadwickModel, hCompr, fCompr, initialParameters, bounds = parameterBounds)
# fittedParameters, pcov = curve_fit(linear, x, y, initialParameters, bounds = parameterBounds)

# A, B = fittedParameters
# print(A, B)
# ypredict = linear(x, A, B)

# varA = pcov[0,0]
# print(pcov)
# print(pcov[0,0]**0.5)
# SSR = np.sum((np.array(y)-np.array(ypredict))**2)
# seA = ((SSR/(len(x)-2))*varA)**0.5
# seA = (varA)**0.5
# confIntA = st.t.interval(alpha=0.95, df=len(x)-2, loc=A, scale=seA)
# confIntAHalfWidthBis = st.t.ppf(0.975, len(x)-2) * seA
# print(confIntAHalfWidthBis)
# # confIntA = (A + seA*st.t.interval(alpha=0.95, df=len(x), loc=0, scale=1)[0], A + seA*st.t.interval(alpha=0.95, df=len(x), loc=0, scale=1)[1])
# confIntAWidth = confIntA[1] - confIntA[0]
# confIntAHalfWidth = confIntAWidth/2
# R2 = get_R2(y,ypredict)

# print(str(A) + ' +/- ' + str(confIntAHalfWidth))

# fig, ax = plt.subplots(1,1)
# ax.plot(x, y, 'bo')
# ax.plot(x, ypredict, 'r-', label='best fit')
# ax.legend(loc='best')
# fig.show()

In [17]:
# Test of the get_R2 function.
# T = df['T']
# Y = df['D3']
# plt.plot(T,Y)
# p, residuals, rank, singular_values, rcond = np.polyfit(T, Y, deg=5, full=True)
# plt.plot(T, Y)
# Y2 = np.zeros(len(T))
# for i in range(len(T)):
#     deg = len(p)-1
#     for k in range(deg+1):
#         Y2[i] += p[k]*(T[i]**(deg-k))
# plt.plot(T,Y2)
# get_R2(Y, Y2)

## TimeSeries functions

In [18]:
def getCellTimeSeriesData(cellID):
    allTimeSeriesDataFiles = [f for f in os.listdir(timeSeriesDataDir) if (os.path.isfile(os.path.join(timeSeriesDataDir, f)) and f.endswith(".csv"))]
    fileFound = False
    nFile = len(allTimeSeriesDataFiles)
    iFile = 0
    while (not fileFound) and (iFile < nFile):
        f = allTimeSeriesDataFiles[iFile]
        if f.startswith(cellID):
            timeSeriesDataFilePath = os.path.join(timeSeriesDataDir, f)
            timeSeriesDataFrame = pd.read_csv(timeSeriesDataFilePath, sep=';')
            fileFound = True
        iFile += 1
    if not fileFound:
        timeSeriesDataFrame = pd.DataFrame([])
    else:
        for c in timeSeriesDataFrame.columns:
                if 'Unnamed' in c:
                    timeSeriesDataFrame = timeSeriesDataFrame.drop([c], axis=1)
    return(timeSeriesDataFrame)

def plotCellTimeSeriesData(cellID):
    X = 'T'
    Y = np.array(['B', 'F', 'dx', 'dy', 'dz', 'D2', 'D3'])
    units = np.array([' (mT)', ' (pN)', ' (µm)', ' (µm)', ' (µm)', ' (µm)', ' (µm)'])
    timeSeriesDataFrame = getCellTimeSeriesData(cellID)
    if not timeSeriesDataFrame.size == 0:
#         plt.tight_layout()
#         fig.show() # figsize=(20,20)
        axes = timeSeriesDataFrame.plot(x=X, y=Y, kind='line', ax=None, subplots=True, sharex=True, sharey=False, layout=None, \
                       figsize=(8,10), use_index=True, title = cellID + '- Time dependant data', grid=None, legend=False, style=None, logx=False, logy=False, \
                       loglog=False, xticks=None, yticks=None, xlim=None, ylim=None, rot=None, fontsize=None, colormap=None, \
                       table=False, yerr=None, xerr=None, secondary_y=False, sort_columns=False)
        plt.gcf().tight_layout()
        for i in range(len(Y)):
            axes[i].set_ylabel(Y[i] + units[i])
        
    else:
        print('cell not found')
        
def addExcludedCell(cellID, motive):
    f = open(os.path.join(experimentalDataDir, 'ExcludedCells.txt'), 'r')
    lines = f.readlines()
    nLines = len(lines)
    excludedCellsList = []
    for iLine in range(nLines):
        line = lines[iLine]
        splitLine = line[:-1].split(',')
        excludedCellsList.append(splitLine[0])
    if cellID in excludedCellsList:
        newlines = copy(lines)
        iLineOfInterest = excludedCellsList.index(cellID)
        if motive not in newlines[iLineOfInterest][:-1].split(','):
            newlines[iLineOfInterest] = newlines[iLineOfInterest][:-1] + ',' + motive + '\n'            
    else:
        newlines = copy(lines)
        newlines.append('' + cellID + ',' + motive + '\n')
    f.close()
    f = open(os.path.join(experimentalDataDir, 'ExcludedCells.txt'), 'w')
    f.writelines(newlines)
    
def getExcludedCells():
    f = open(os.path.join(experimentalDataDir, 'ExcludedCells.txt'), 'r')
    lines = f.readlines()
    nLines = len(lines)
    excludedCellsDict = {}
    for iLine in range(nLines):
        line = lines[iLine]
        splitLine = line[:-1].split(',')
        excludedCellsDict[splitLine[0]] = splitLine[1:]
    return(excludedCellsDict)

In [19]:
df = getCellTimeSeriesData('21-09-09_M1_P1_C2')
# plotCellTimeSeriesData('21-01-21_M1_P1_C11')
# plotCellTimeSeriesData('21-04-21_M1_P1_C5')
df

,idxCompression,T,Tabs,B,F,dx,dy,dz,D2,D3
0,0,0.000,70100.869,5.104,0.0,4.273468,-2.201802,0.234,4.807334,4.813025
1,0,0.790,70101.659,5.112,0.0,4.271619,-2.204493,0.190,4.806923,4.810677
2,0,1.390,70102.259,5.108,0.0,4.263869,-2.217922,0.204,4.806221,4.810548
3,0,1.991,70102.860,5.108,0.0,4.268293,-2.214102,0.190,4.808385,4.812138
4,0,2.591,70103.460,5.128,0.0,4.260061,-2.230178,0.214,4.808515,4.813274
...,...,...,...,...,...,...,...,...,...,...
1445,0,119.710,70220.579,5.145,0.0,4.904196,-1.080292,0.612,5.021770,5.058924
1446,0,120.310,70221.179,5.145,0.0,4.904649,-1.084319,0.616,5.023079,5.060710
1447,0,120.911,70221.780,5.153,0.0,4.910147,-1.083860,0.614,5.028349,5.065697
1448,0,121.510,70222.379,5.153,0.0,4.907331,-1.079375,0.592,5.024634,5.059388


In [20]:
allTimeSeriesDataFiles = [f for f in os.listdir(timeSeriesDataDir) if (os.path.isfile(os.path.join(timeSeriesDataDir, f)) and f.endswith(".csv"))]
for f in allTimeSeriesDataFiles:
    if '21-09-09_M1_P1_C2' in f:
        plotCellTimeSeriesData(f[:-4])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
allTimeSeriesDataFiles = [f for f in os.listdir(timeSeriesDataDir) if (os.path.isfile(os.path.join(timeSeriesDataDir, f)) and f.endswith(".csv"))]
allTimeSeriesDataFiles

['20-08-04_M1_P1_C5_R40.csv',
 '20-08-04_M1_P1_C6_R40.csv',
 '20-08-04_M1_P1_C7_R40.csv',
 '20-08-04_M2_P1_C3_R40.csv',
 '20-08-04_M2_P1_C4_R40.csv',
 '20-08-04_M2_P1_C5_R40.csv',
 '20-08-05_M1_P1_C32_R40.csv',
 '20-08-05_M1_P1_C3_R40.csv',
 '20-08-05_M1_P1_C4_R40.csv',
 '20-08-05_M1_P1_C5_R40.csv',
 '20-08-05_M1_P1_C6_R40.csv',
 '20-08-05_M1_P1_C7_R40.csv',
 '20-08-05_M1_P1_C8_R40.csv',
 '20-08-05_M2_P1_C2_R40.csv',
 '20-08-05_M2_P1_C3_R40.csv',
 '20-08-05_M2_P1_C4_R40.csv',
 '20-08-05_M2_P1_C5_R40.csv',
 '20-08-05_M2_P1_C6_R40.csv',
 '20-08-05_M2_P1_C7_R40.csv',
 '20-08-07_M1_P1_C3_R40.csv',
 '20-08-07_M1_P1_C4_R40.csv',
 '20-08-07_M1_P1_C62_R40.csv',
 '20-08-07_M1_P1_C6_R40.csv',
 '20-08-07_M1_P1_C7_R40.csv',
 '20-08-07_M1_P1_C8_R40.csv',
 '20-08-07_M2_P1_C4_R40.csv',
 '20-08-07_M2_P1_C5_R40.csv',
 '20-08-07_M2_P1_C6_R40.csv',
 '20-08-07_M2_P1_C7_R40.csv',
 '21-01-18_M1-1_P1_C1_R40_disc20um.csv',
 '21-01-18_M1-1_P1_C2_R40_disc20um.csv',
 '21-01-18_M1-1_P1_C3_R40_disc20um.csv',
 '21-

In [29]:
L450 = []
L270 = []
allTimeSeriesDataFiles = [f for f in os.listdir(timeSeriesDataDir) if (os.path.isfile(os.path.join(timeSeriesDataDir, f)) and f.endswith(".csv"))]
for f in allTimeSeriesDataFiles:
    if '21-07-08_M2' in f:
        df = getCellTimeSeriesData(f[:-4])
        L270.append(df)
    elif '21-07-08_M4' in f:
        df = getCellTimeSeriesData(f[:-4])
        L450.append(df)
        
fig, ax = plt.subplots(1,2)
ax[0].plot(L270[3].B,L270[3].F)
ax[1].plot(L450[2].B,L450[2].F)
# ax[1,0].hist(L270[0].B, bins=20, range=None, weights=L270[0].F)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
plt.close('all')

## GlobalTables functions

### Experimental conditions

In [23]:
def getExperimentalConditions(save = False):
    # Getting the table
    experimentalDataFile = 'ExperimentalConditions.csv'
    experimentalDataFilePath = os.path.join(experimentalDataDir, experimentalDataFile)
    expConditionsDF = pd.read_csv(experimentalDataFilePath, sep=';',header=0)
    print('Extracted a table with ' + str(expConditionsDF.shape[0]) + ' lines and ' + str(expConditionsDF.shape[1]) + ' columns.')
    
    # Cleaning the table
    try:
        for c in expConditionsDF.columns:
            if 'Unnamed' in c:
                expConditionsDF = expConditionsDF.drop([c], axis=1)
        expConditionsDF = expConditionsDF.convert_dtypes()

        listTextColumns = []
        for col in expConditionsDF.columns:
            try:
                if expConditionsDF[col].dtype == 'string' and not col == 'drug':
                    listTextColumns.append(col)
            except:
                aaaa=0
                #Ok
        
        expConditionsDF[listTextColumns] = expConditionsDF[listTextColumns].apply(lambda x: x.str.replace(',','.'))

        expConditionsDF['scale pixel per um'] = expConditionsDF['scale pixel per um'].astype(float)
        try:
            expConditionsDF['optical index correction'] = \
                      expConditionsDF['optical index correction'].apply(lambda x: x.split('/')[0]).astype(float) \
                    / expConditionsDF['optical index correction'].apply(lambda x: x.split('/')[1]).astype(float)
        except:
            print('optical index correction already in ' + str(expConditionsDF['optical index correction'].dtype) + ' type.')

        expConditionsDF['magnetic field correction'] = expConditionsDF['magnetic field correction'].astype(float)
        expConditionsDF['with fluo images'] = expConditionsDF['with fluo images'].astype(bool)

        try:
            expConditionsDF['ramp field'] = \
            expConditionsDF['ramp field'].apply(lambda x: [x.split(';')[0], x.split(';')[1]] if not pd.isnull(x) else [])
        except:
            pass
            #Ok

        dateExemple = expConditionsDF.loc[expConditionsDF.index[1],'date']

        if re.match(dateFormatExcel, dateExemple):
            print('dates corrected')
            expConditionsDF.loc[1:,'date'] = expConditionsDF.loc[1:,'date'].apply(lambda x: x.split('/')[0] + '-' + x.split('/')[1] + '-' + x.split('/')[2][2:])        
        
    except:
        print('Unexpected bug with the cleaning step')

    if save:
        saveName = 'ExperimentalConditions.csv'
        savePath = os.path.join(experimentalDataDir, saveName)
        expConditionsDF.to_csv(savePath, sep=';')

    expConditionsDF['manipID'] = expConditionsDF['date'] + '_' + expConditionsDF['manip']
    
    return(expConditionsDF)

In [24]:
getExperimentalConditions(save=True)

Extracted a table with 57 lines and 24 columns.
optical index correction already in Float64 type.


,date,manip,experimentType,drug,substrate,objective magnification,scale pixel per um,objective immersion,optical index correction,magnetic field correction,...,beads bright spot delta,normal field,ramp field,compression duration,with fluo images,normal field multi images,multi image Z step,loop structure,comments,manipID
0,DEFAULT,DEFAULT,DEFAULT,DEFAULT,DEFAULT,100X,15.8,oil,0.875,1.150,...,0,5,3_40,1s,False,3,500,143_95,<NA>,DEFAULT_DEFAULT
1,20-08-04,M1,compressions and constant field,none,BSA coated glass,100X,15.8,oil,0.875,1.150,...,0,10,3_40,1s,False,3,500,143_95,<NA>,20-08-04_M1
2,20-08-04,M2,compressions and constant field,doxycyclin,BSA coated glass,100X,15.8,oil,0.875,1.150,...,0,10,3_40,1s,False,3,500,143_95,<NA>,20-08-04_M2
3,20-08-05,M1,compressions and constant field,doxycyclin,BSA coated glass,100X,15.8,oil,0.875,1.150,...,0,10,3_40,1s,False,3,500,143_95,<NA>,20-08-05_M1
4,20-08-05,M2,compressions and constant field,none,BSA coated glass,100X,15.8,oil,0.875,1.150,...,0,10,3_40,1s,False,3,500,143_95,<NA>,20-08-05_M2
5,20-08-07,M1,compressions and constant field,doxycyclin,BSA coated glass,100X,15.8,oil,0.875,1.150,...,0,10,3_40,1s,False,3,500,143_95,<NA>,20-08-07_M1
6,20-08-07,M2,compressions and constant field,none,BSA coated glass,100X,15.8,oil,0.875,1.150,...,0,10,3_40,1s,False,3,500,143_95,<NA>,20-08-07_M2
7,21-01-18,M1-1,compressions,doxycyclin,20um fibronectin discs,100X,15.8,oil,0.875,1.230,...,0,5,3_40,1s,False,3,500,169_133,<NA>,21-01-18_M1-1
8,21-01-18,M1-2,compressions,doxycyclin,20um fibronectin discs,100X,15.8,oil,0.875,1.230,...,0,5,3_40,1s,True,3,500,170_133_1,<NA>,21-01-18_M1-2
9,21-01-18,M2,compressions,none,20um fibronectin discs,100X,15.8,oil,0.875,1.230,...,0,5,3_40,1s,True,3,500,170_133_1,<NA>,21-01-18_M2


### Constant Field

In [81]:
listColumnsCtField = ['date','cellName','cellID','manipID',\
                      'duration','medianRawB','medianThickness',\
                      '1stDThickness','9thDThickness','fluctuAmpli',\
                      'R2_polyFit','validated']

def analyseTimeSeries_ctField(tsDf):
    results = {}
    results['duration'] = np.max(tsDf['T'])
    results['medianRawB'] = np.median(tsDf.B)
    results['medianThickness'] = np.median(tsDf.D3)
    results['1stDThickness'] = np.percentile(tsDf.D3, 10)
    results['9thDThickness'] = np.percentile(tsDf.D3, 90)
    results['fluctuAmpli'] = results['9thDThickness'] - results['1stDThickness']
    results['validated'] = (results['1stDThickness'] > 0)
    X, Y = tsDf['T'], tsDf['D3']
    p, residuals, rank, singular_values, rcond = np.polyfit(X, Y, deg=5, full=True)
    Y2 = np.zeros(len(X))
    for i in range(len(X)):
        deg = len(p)-1
        for k in range(deg+1):
            Y2[i] += p[k]*(X[i]**(deg-k))
    results['R2_polyFit'] = get_R2(Y, Y2)
    return(results)



def createDataDict_ctField(list_ctFieldFiles):
    tableDict = {}
    tableDict['date'], tableDict['cellName'], tableDict['cellID'], tableDict['manipID'] = [], [], [], []
    tableDict['duration'], tableDict['medianRawB'], tableDict['medianThickness'] = [], [], []
    tableDict['1stDThickness'], tableDict['9thDThickness'], tableDict['fluctuAmpli'] = [], [], []
    tableDict['R2_polyFit'], tableDict['validated'] = [], []
    for f in list_ctFieldFiles:
        split_f = f.split('_')
        tableDict['date'].append(split_f[0])
        tableDict['cellName'].append(split_f[1] + '_' + split_f[2] + '_' + split_f[3])
        tableDict['cellID'].append(split_f[0] + '_' + split_f[1] + '_' + split_f[2] + '_' + split_f[3])
        tableDict['manipID'].append(split_f[0] + '_' + split_f[1])
        tS_DataFilePath = os.path.join(timeSeriesDataDir, f)
        current_tsDf = pd.read_csv(tS_DataFilePath, ';')
        current_resultDict = analyseTimeSeries_ctField(current_tsDf)
        for k in current_resultDict.keys():
            tableDict[k].append(current_resultDict[k])
    return(tableDict)



def computeGlobalTable_ctField(task = 'fromScratch', fileName = 'Global_CtFieldData', save = False):
    ctFieldTimeSeriesDataFiles = [f for f in os.listdir(timeSeriesDataDir) \
                                  if (os.path.isfile(os.path.join(timeSeriesDataDir, f)) and f.endswith(".csv") \
                                      and ('thickness' in f))]
#     print(ctFieldTimeSeriesDataFiles)
    if task == 'fromScratch':
        # create a dict containing the data
        tableDict = createDataDict_ctField(ctFieldTimeSeriesDataFiles) # MAIN SUBFUNCTION
        # create the table
        CtField_DF = pd.DataFrame(tableDict)
        
    elif task == 'updateExisting':
        # get existing table
        try:
            savePath = os.path.join(dataDir, (fileName + '.csv'))
            existing_CtField_DF = pd.read_csv(savePath, sep=';')
            for c in existing_CtField_DF.columns:
                if 'Unnamed' in c:
                    existing_CtField_DF = existing_CtField_DF.drop([c], axis=1)
        except:
            print('No existing table found')
        # find which of the time series files are new
        new_ctFieldTimeSeriesDataFiles = []
        for f in ctFieldTimeSeriesDataFiles:
            split_f = f.split('_')
            currentCellID = split_f[0] + '_' + split_f[1] + '_' + split_f[2] + '_' + split_f[3]
            if currentCellID not in existing_CtField_DF.cellID.values:
                new_ctFieldTimeSeriesDataFiles.append(f)
        new_tableDict = createDataDict_ctField(new_ctFieldTimeSeriesDataFiles) # MAIN SUBFUNCTION
        # create the table with new data
        new_CtField_DF = pd.DataFrame(new_tableDict)
        # fuse the two
        new_CtField_DF.index += existing_CtField_DF.shape[0]
        CtField_DF = pd.concat([existing_CtField_DF, new_CtField_DF])
    
    dateExemple = CtField_DF.loc[CtField_DF.index[0],'date']
    if re.match(dateFormatExcel, dateExemple):
        CtField_DF.loc[:,'date'] = CtField_DF.loc[:,'date'].apply(lambda x: x.split('/')[0] + '-' + x.split('/')[1] + '-' + x.split('/')[2][2:])
    
    if save:
        saveName = fileName + '.csv'
        savePath = os.path.join(dataDir, saveName)
        CtField_DF.to_csv(savePath, sep=';')
        
    return(CtField_DF)



def getGlobalTable_ctField(fileName = 'Global_CtFieldData'):
    try:
        savePath = os.path.join(dataDir, (fileName + '.csv'))
        CtField_DF = pd.read_csv(savePath, sep=';')
        for c in CtField_DF.columns:
            if 'Unnamed' in c:
                CtField_DF = CtField_DF.drop([c], axis=1)
        print('Extracted a table with ' + str(CtField_DF.shape[0]) + ' lines and ' + str(CtField_DF.shape[1]) + ' columns.')
        
    except:
        print('No existing table found')
        
    dateExemple = CtField_DF.loc[CtField_DF.index[0],'date']
    if re.match(dateFormatExcel, dateExemple):
        print('dates corrected')
        CtField_DF.loc[:,'date'] = CtField_DF.loc[:,'date'].apply(lambda x: x.split('/')[0] + '-' + x.split('/')[1] + '-' + x.split('/')[2][2:])
#         mecaDF['ManipID'] = mecaDF['ExpDay'] + '_' + mecaDF['CellName'].apply(lambda x: x.split('_')[0])
    return(CtField_DF)

In [82]:
computeGlobalTable_ctField(task='updateExisting',save=False).head()

,date,cellName,cellID,manipID,duration,medianRawB,medianThickness,1stDThickness,9thDThickness,fluctuAmpli,R2_polyFit,validated
0,21-02-10,M1_P1_C1,21-02-10_M1_P1_C1,21-02-10_M1,393.550,6.0036,183.232388,103.105084,276.671473,173.566389,0.222994,1.0
1,21-02-10,M1_P1_C2,21-02-10_M1_P1_C2,21-02-10_M1,326.388,6.0036,-58.229968,-147.203982,41.303068,188.507051,0.228033,0.0
2,21-02-10,M1_P1_C3,21-02-10_M1_P1_C3,21-02-10_M1,327.997,6.0036,204.963636,123.738478,277.782516,154.044038,0.402168,1.0
3,21-02-10,M1_P1_C4,21-02-10_M1_P1_C4,21-02-10_M1,326.929,6.0036,203.321336,165.769700,282.439284,116.669584,0.261129,1.0
4,21-02-10,M1_P1_C5,21-02-10_M1_P1_C5,21-02-10_M1,260.260,6.0036,231.480125,169.220846,301.601770,132.380924,0.248455,1.0


In [83]:
getGlobalTable_ctField().head()

Extracted a table with 40 lines and 12 columns.


,date,cellName,cellID,manipID,duration,medianRawB,medianThickness,1stDThickness,9thDThickness,fluctuAmpli,R2_polyFit,validated
0,21-02-10,M1_P1_C1,21-02-10_M1_P1_C1,21-02-10_M1,393.550,6.0036,183.232388,103.105084,276.671473,173.566389,0.222994,1.0
1,21-02-10,M1_P1_C2,21-02-10_M1_P1_C2,21-02-10_M1,326.388,6.0036,-58.229968,-147.203982,41.303068,188.507051,0.228033,0.0
2,21-02-10,M1_P1_C3,21-02-10_M1_P1_C3,21-02-10_M1,327.997,6.0036,204.963636,123.738478,277.782516,154.044038,0.402168,1.0
3,21-02-10,M1_P1_C4,21-02-10_M1_P1_C4,21-02-10_M1,326.929,6.0036,203.321336,165.769700,282.439284,116.669584,0.261129,1.0
4,21-02-10,M1_P1_C5,21-02-10_M1_P1_C5,21-02-10_M1,260.260,6.0036,231.480125,169.220846,301.601770,132.380924,0.248455,1.0


### Mechanics

Workflow
* analyseTimeSeries_meca() analyse 1 file and return the dict (with the results of the analysis)
* createMecaDataDict() call the previous function on the given list of files and concatenate the results
* computeGlobalTable_meca() call the previous function and convert the dict to a DataFrame

In [84]:
listColumnsMeca = ['date','cellName','cellID','manipID',\
                   'compNum','compDuration','compStartTime','compAbsStartTime','compStartTimeThisDay',\
                   'initialThickness','minThickness','maxIndent','previousThickness','surroundingThickness',\
                   'validatedThickness',\
                   'ctFieldThickness','ctFieldFluctuAmpli',\
                   'H0Chadwick','EChadwick','R2Chadwick','EChadwick_CIWidth',\
                   'hysteresis',\
                   'critFit', 'validatedFit','comments'] # 'fitParams',
regionFitsNames = ['f<150pN', 'f<100pN', 's<150Pa']
for rFN in regionFitsNames:
    listColumnsMeca += ['H0Chadwick_'+rFN, 'EChadwick_'+rFN, 'R2Chadwick_'+rFN, 'Npts_'+rFN, 'validatedFit_'+rFN]

def compressionFitChadwick(hCompr, fCompr, DIAMETER):
    
    error = False
    
    def chadwickModel(h, E, H0):
        R = DIAMETER/2
        f = (np.pi*E*R*((H0-h)**2))/(3*H0)
        return(f)

    def inversedChadwickModel(f, E, H0):
        R = DIAMETER/2
        h = H0 - ((3*H0*f)/(np.pi*E*R))**0.5
        return(h)

    # some initial parameter values - must be within bounds
    initH0 = max(hCompr) # H0 ~ h_max
    initE = (3*max(hCompr)*max(fCompr))/(np.pi*(DIAMETER/2)*(max(hCompr)-min(hCompr))**2) # E ~ 3*H0*F_max / pi*R*(H0-h_min)²
#     initH0, initE = initH0*(initH0>0), initE*(initE>0)
    
    initialParameters = [initE, initH0]
#     print(initialParameters)

    # bounds on parameters - initial parameters must be within these
    lowerBounds = (0, 0)
    upperBounds = (np.Inf, np.Inf)
    parameterBounds = [lowerBounds, upperBounds]
    
#     testH = inversedChadwickModel(fCompr, initE, initH0)
#     fig, ax = plt.subplots(1,1)
#     ax.plot(hCompr,fCompr,'b-', linewidth = 0.8)
#     ax.plot(testH,fCompr,'kx', linewidth = 0.8)
#     ax.set_xlabel('h (nm)')
#     ax.set_ylabel('f (pN)')
#     fig.show()
#     print(initialParameters)

    try:
        params, covM = curve_fit(inversedChadwickModel, fCompr, hCompr, initialParameters, bounds = parameterBounds)

        # Previously I fitted with y=F and x=H, but it didn't work so well cause H(t) isn't monotonous:
        # params, covM = curve_fit(chadwickModel, hCompr, fCompr, initialParameters, bounds = parameterBounds)
        # Fitting with the 'inverse Chadwick model', with y=H and x=F is more convenient

        E, H0 = params
        hPredict = inversedChadwickModel(fCompr, E, H0)

        SSR = np.sum((hCompr-hPredict)**2)
        alpha = 0.975
        df = len(fCompr)-len(params)
        q = st.t.ppf(alpha, df) # Student coefficient
        R2 = get_R2(hCompr,hPredict)

        varE = covM[0,0]
        seE = (varE)**0.5
        E, seE = E*1e6, seE*1e6
        confIntE = [E-q*seE, E+q*seE]
        confIntEWidth = 2*q*seE

        varH0 = covM[1,1]
        seH0 = (varH0)**0.5
        confIntH0 = [H0-q*seH0, H0+q*seH0]
        confIntH0Width = 2*q*seH0
        
        
    except:
        error = True
        E, H0, hPredict, R2, confIntE, confIntH0 = -1, -1, np.ones(len(hCompr))*(-1), -1, [-1,-1], [-1,-1]
    
    return(E, H0, hPredict, R2, confIntE, confIntH0, error)



def analyseTimeSeries_meca(f, tsDF, expDf, listColumnsMeca, PLOT, PLOT_SHOW):
    split_f = f.split('_')
    tsDF.dx, tsDF.dy, tsDF.dz, tsDF.D2, tsDF.D3 = tsDF.dx*1000, tsDF.dy*1000, tsDF.dz*1000, tsDF.D2*1000, tsDF.D3*1000
    thisManipID = split_f[0] + '_' + split_f[1]
    expDf['manipID'] = expDf['date'] + '_' + expDf['manip']
    thisExpDf = expDf.loc[expDf['manipID'] == thisManipID]

    # Deal with the asymmetric pair case : the diameter can be for instance 4503 (float) or '4503_2691' (string)
    diameters = str(thisExpDf.at[thisExpDf.index.values[0], 'bead diameter'])
    diameters = diameters.split('_')
    if len(diameters) == 2:
        DIAMETER = (int(diameters[0]) + int(diameters[1]))/2.
    else:
        DIAMETER = int(diameters[0])
    
    EXPTYPE = str(thisExpDf.at[thisExpDf.index.values[0], 'experimentType'])
    
#     if 'compressions' in EXPTYPE:
#         rampFields = str(thisExpDf.at[thisExpDf.index.values[0], 'ramp field'])
#         rampFields = rampFields.split('_')
#         RAMP_MIN = int(rampFields[0])
#         RAMP_MAX = int(rampFields[1])
#     print(DIAMETER, RAMP_MIN, RAMP_MAX)
        
#     try:
#         if '_' in DIAMETER:
#             diameters = DIAMETER.split('_')
#             DIAMETER = (int(diameters[0]) + int(diameters[1]))/2.
#     except:
#         pass
#     try:
#         DIAMETER = int(DIAMETER)
#     except:
#         pass

    results = {}
    for c in listColumnsMeca:
        results[c] = []

    Ncomp = max(tsDF['idxCompression'])
    loopStruct = thisExpDf.at[thisExpDf.index.values[0], 'loop structure'].split('_')
    nUplet = thisExpDf.at[thisExpDf.index.values[0], 'normal field multi images']

    loop_totalSize = int(loopStruct[0])
    if len(loopStruct) >= 2:
        loop_rampSize = int(loopStruct[1])
    else:
        loop_rampSize = 0
    if len(loopStruct) >= 3:
        loop_excludedSize = int(loopStruct[2])
    else:
        loop_excludedSize = 0
    loop_ctSize = int((loop_totalSize - (loop_rampSize+loop_excludedSize))/nUplet)

# Less proper way to do exactly like above

#     NimgComp = np.sum((tsDF['idxCompression'] != 0))/Ncomp
#     NimgCompTh = round(0.49999999999 + np.sum((tsDF['idxCompression'] != 0))/Ncomp)
#     NimgBtwComp = np.sum((tsDF['idxCompression'] == 0))/Ncomp
#     NimgBtwCompTh = round(0.4999999999 + np.sum((tsDF['idxCompression'] == 0))/Ncomp)
#     print('Ncomp : ' + str(Ncomp) + ' ; ' + 'NimgComp : ' + str(NimgComp) + '/' + str(NimgCompTh) + ' ; ' + 'NimgBtwComp : ' + str(NimgBtwComp) + '/' + str(NimgBtwCompTh))
#     if not NimgBtwComp%2 == 0:
#         print('Bug with the compressions sequence delimitation')

    Ncomp = max(tsDF['idxCompression'])
    compField = thisExpDf.at[thisExpDf.index.values[0], 'ramp field'].split('_')
    minCompField = int(compField[0])
    maxCompField = int(compField[1])


    # These values are computed once for the whole cell D3 time series, but since the table has 1 line per compression, 
    # that same value will be put in the table for each line corresponding to that cell
    ctFieldH = (tsDF.loc[tsDF['idxCompression'] == 0, 'D3'].values - DIAMETER)
    ctFieldThickness   = np.median(ctFieldH)
    ctFieldFluctuAmpli = np.percentile(ctFieldH,90) - np.percentile(ctFieldH,10)

    # First part of the plot [mainly ax1 and ax1bis]
    if PLOT:
        # First plot - fig1 & ax1, is the F(t) curves with the different compressions colored depending of the analysis success
        fig1, ax1 = plt.subplots(1,1,figsize=(tsDF.shape[0]*(1/100),5))
        color = 'blue'
        ax1.plot(tsDF['T'].values, tsDF['D3'].values-DIAMETER, color = color, ls = '-', linewidth = 1)
        ax1.set_xlabel('t (s)')
        ax1.set_ylabel('h (nm)', color=color)
        ax1.tick_params(axis='y', labelcolor=color)
        (axm, axM) = ax1.get_ylim()
        ax1.set_ylim([min(0,axm), axM])
        if (max(tsDF['D3'].values-DIAMETER) > 200):
            ax1.set_yticks(np.arange(0, max(tsDF['D3'].values-DIAMETER), 100))

        twinAxis = True
        if twinAxis:
            ax1.tick_params(axis='y', labelcolor='b')
            ax1bis = ax1.twinx()
            color = 'firebrick'
            ax1bis.set_ylabel('F (pN)', color=color)
            ax1bis.plot(tsDF['T'].values, tsDF['F'].values, color=color)
            ax1bis.tick_params(axis='y', labelcolor=color)
            ax1bis.set_yticks([0,500,1000,1500])
            minh = np.min(tsDF['D3'].values-DIAMETER)
            ratio = min(1/abs(minh/axM), 5)
#             print(ratio)
            (axmbis, axMbis) = ax1bis.get_ylim()
            ax1bis.set_ylim([0, max(axMbis*ratio, 3*max(tsDF['F'].values))])

        nColsSubplot = 5
        nRowsSubplot = ((Ncomp-1) // nColsSubplot) + 1
        # Second plot - fig2 & ax2, gather all the F(h) curves, and will be completed later in the code
        fig2, ax2 = plt.subplots(nRowsSubplot,nColsSubplot,figsize=(3*nColsSubplot,3*nRowsSubplot))
        # Third plot - fig3 & ax3, gather all the stress-strain curves, and will be completed later in the code
        fig3, ax3 = plt.subplots(nRowsSubplot,nColsSubplot,figsize=(3*nColsSubplot,3*nRowsSubplot))


    for i in range(1, Ncomp+1):#Ncomp+1):

        # (1) Identifiers
        results['date'].append(split_f[0])
        results['cellName'].append(split_f[1] + '_' + split_f[2] + '_' + split_f[3])
        results['cellID'].append(split_f[0] + '_' + split_f[1] + '_' + split_f[2] + '_' + split_f[3])
        results['manipID'].append(split_f[0] + '_' + split_f[1])

        # (2) Segment the compression n°i
        thisCompDf = tsDF.loc[tsDF['idxCompression'] == i,:]
        iStart = (findFirst(tsDF['idxCompression'], i))
        iStop = iStart+thisCompDf.shape[0]

        # Easy-to-get parameters
        results['compNum'].append(i)
        results['compDuration'].append(thisExpDf.at[thisExpDf.index.values[0], 'compression duration'])
        results['compStartTime'].append(thisCompDf['T'].values[0])
        results['compAbsStartTime'].append(thisCompDf['Tabs'].values[0])
        results['compStartTimeThisDay'].append(thisCompDf['Tabs'].values[0])

        # Test to check if most of the compression have not been deleted due to bad image quality 
        highBvalues = (thisCompDf.B.values > (maxCompField + minCompField)/2)
        N_highBvalues = np.sum(highBvalues)
        testHighVal = (N_highBvalues > 20)

        # Test to check if the range of B field is large enough
        listB = thisCompDf.B.values
        minB, maxB = min(listB), max(listB)
        testRangeB = (maxB-minB > 0.7*(maxCompField - minCompField))

        # Is the curve ok to analyse ?
        doThisCompAnalysis = testHighVal and testRangeB # Some criteria can be added here

        if doThisCompAnalysis:
            # (3) Inside the compression n°i, delimit the compression and relaxation phases

            # Delimit the start of the increase of B (typically the moment when the field decrease from 5 to 3)
            # and the end of its decrease (typically when it goes back from 3 to 5)
            listB = thisCompDf.B.values
            
            # Correct for bugs in the B data
            if 'compressions' in EXPTYPE:
                for k in range(1,len(listB)):
                    B = listB[k]
                    if B > 1.25*maxCompField:
                        listB[k] = listB[k-1]
            
            offsetStart, offsetStop = 0, 0
            minB, maxB = min(listB), max(listB)
            thresholdB = (maxB-minB)/50
            thresholdDeltaB = (maxB-minB)/400 # NEW CONDITION for the beginning of the compression : 
            # remove the first points where the steps in B are very very small
            
            k = 0
            while (listB[k] > minB+thresholdB) or (listB[k+1]-listB[k] < thresholdDeltaB):
                offsetStart += int((listB[k] > minB+thresholdB) or ((listB[k+1]-listB[k]) < thresholdDeltaB))
                k += 1
                
            k = 0
            while (listB[-1-k] > minB+thresholdB):
                offsetStop += int(listB[-1-k] > minB+thresholdB)
                k += 1
                
            jStart = offsetStart # Beginning of compression
            jMax = np.argmax(thisCompDf.B) # End of compression, beginning of relaxation
            jStop = thisCompDf.shape[0] - offsetStop # End of relaxation

            # Four arrays
            hCompr = (thisCompDf.D3.values[jStart:jMax+1] - DIAMETER)
            hRelax = (thisCompDf.D3.values[jMax+1:jStop] - DIAMETER)
            fCompr = (thisCompDf.F.values[jStart:jMax+1])
            fRelax = (thisCompDf.F.values[jMax+1:jStop])


            # Refinement of the compression delimitation.
            # Remove the 1-2 points at the begining where there is just the viscous relaxation of the cortex
            # because of the initial decrease of B and the cortex thickness increases.
            offsetStart2 = 0
            k = 0
            while (k<len(hCompr)-10) and (hCompr[k] < np.max(hCompr[k+1:min(k+10, len(hCompr))])):
                offsetStart2 += 1
                k += 1
            # Better compressions arrays
            hCompr = hCompr[offsetStart2:]
            fCompr = fCompr[offsetStart2:]

            # Get the points of constant field preceding and surrounding the current compression
            # Ex : if the labview code was set so that there is 6 points of ct field before and after each compression,
            # previousPoints will contains D3[iStart-12:iStart]
            # surroundingPoints will contains D3[iStart-6:iStart] and D3[iStop:iStop+6]
            previousPoints = (tsDF.D3.values[max(0,iStart-(loop_ctSize)):iStart]) - DIAMETER
            surroundingPoints = np.concatenate([tsDF.D3.values[max(0,iStart-(loop_ctSize//2)):iStart],tsDF.D3.values[iStop:iStop+(loop_ctSize//2)]]) - DIAMETER

            # Parameters relative to the thickness ( = D3-DIAMETER)
            results['initialThickness'].append(np.mean(hCompr[0:3]))
            results['minThickness'].append(np.min(hCompr))
            results['maxIndent'].append(results['initialThickness'][-1] - results['minThickness'][-1])
            results['previousThickness'].append(np.median(previousPoints))
            results['surroundingThickness'].append(np.median(surroundingPoints))
            results['ctFieldThickness'].append(ctFieldThickness)
            results['ctFieldFluctuAmpli'].append(ctFieldFluctuAmpli)

            validatedThickness = np.min([results['initialThickness'],results['minThickness'],results['previousThickness'],\
                                        results['surroundingThickness'],results['ctFieldThickness']]) > 0
            results['validatedThickness'].append(validatedThickness)

            # (4) Fit with Chadwick model of the force-thickness curve
            E, H0, hPredict, R2, confIntE, confIntH0, fitError = compressionFitChadwick(hCompr, fCompr, DIAMETER) # IMPORTANT SUBFUNCTION

            R2CRITERION = 0.9
            critFit = 'R2 > ' + str(R2CRITERION)
            results['critFit'].append(critFit)
            validatedFit = (R2 > R2CRITERION)
            
            if fitError:
                validatedFit = False
                results['H0Chadwick'].append(np.nan)
                results['EChadwick'].append(np.nan)
                results['R2Chadwick'].append(np.nan)
                results['EChadwick_CIWidth'].append(np.nan)
                results['validatedFit'].append(validatedFit)
                results['comments'].append('fitFailure')
            
            if not fitError:
                results['validatedFit'].append(validatedFit)
                if validatedFit:
                    results['comments'].append('ok')
                else:
                    results['comments'].append('R2 < ' + str(R2CRITERION))
                
                confIntEWidth = abs(confIntE[0] - confIntE[1])
                results['H0Chadwick'].append(H0)
                results['EChadwick'].append(E)
                results['R2Chadwick'].append(R2)
                results['EChadwick_CIWidth'].append(confIntEWidth)
                
            
            # (4.1) Fits on specific regions of the curve
            regionFitsNames = ['f<150pN', 'f<100pN', 's<150Pa']
            if fitError:
                for rFN in regionFitsNames:
                    results['H0Chadwick_'+rFN].append(np.nan)
                    results['EChadwick_'+rFN].append(np.nan)
                    results['R2Chadwick_'+rFN].append(np.nan)
                    results['Npts_'+rFN].append(np.nan)
                    results['validatedFit_'+rFN].append(False)
            
            if not fitError:
                deltaCompr = (H0 - hCompr)/1000
                stressCompr = fCompr / (np.pi * (DIAMETER/2000) * deltaCompr)
                strainCompr = deltaCompr / (3*(H0/1000))
                validDelta = (deltaCompr > 0)
                dictRegionFit = {'regionFitNames' : [], 'E' : [], 'H0' : [], 'R2' : [], 
                                 'fitError' : [], 'validatedFit' : [], 'Npts' : []}
                
                # Fit f < 150pN
                regionFitNames = 'f<150pN'
                mask_region = (fCompr < 150)
                Npts_region = np.sum(mask_region)
                if Npts_region > 10:
                    fCompr_region = fCompr[mask_region]
                    hCompr_region = hCompr[mask_region]
                    E_region, H0_region, hPredict_region, R2_region, confIntE_region, confIntH0_region, fitError_region = \
                                  compressionFitChadwick(hCompr_region, fCompr_region, DIAMETER)
                    if not fitError_region:
                        R2CRITERION = 0.9
                        validatedFit_region = (R2_region > R2CRITERION)
                else:
                    validatedFit_region, fitError_region = False, True
                
                if fitError_region:
                    validatedFit_region = False
                    E_region, H0_region, R2_region  = np.nan, np.nan, np.nan
                
                dictRegionFit['regionFitNames'].append(regionFitNames)
                dictRegionFit['Npts'].append(Npts_region)
                dictRegionFit['E'].append(E_region)
                dictRegionFit['H0'].append(H0_region)
                dictRegionFit['R2'].append(R2_region)
                dictRegionFit['fitError'].append(fitError_region)
                dictRegionFit['validatedFit'].append(validatedFit_region)

                # Fit f < 100pN
                regionFitNames = 'f<100pN'
                mask_region = (fCompr < 100)
                Npts_region = np.sum(mask_region)
                if Npts_region > 10:
                    fCompr_region = fCompr[mask_region]
                    hCompr_region = hCompr[mask_region]
                    E_region, H0_region, hPredict_region, R2_region, confIntE_region, confIntH0_region, fitError_region = \
                                  compressionFitChadwick(hCompr_region, fCompr_region, DIAMETER)
                    if not fitError_region:
                        R2CRITERION = 0.9
                        validatedFit_region = (R2_region > R2CRITERION)
                else:
                    validatedFit_region, fitError_region = False, True
                
                if fitError_region:
                    validatedFit_region = False
                    E_region, H0_region, R2_region  = np.nan, np.nan, np.nan
                
                dictRegionFit['regionFitNames'].append(regionFitNames)
                dictRegionFit['Npts'].append(Npts_region)
                dictRegionFit['E'].append(E_region)
                dictRegionFit['H0'].append(H0_region)
                dictRegionFit['R2'].append(R2_region)
                dictRegionFit['fitError'].append(fitError_region)
                dictRegionFit['validatedFit'].append(validatedFit_region)

                # Fit s < 150Pa
                regionFitNames = 's<150Pa'
                mask_region = (stressCompr < 150)
                Npts_region = np.sum(mask_region)
                if Npts_region > 10:
                    fCompr_region = fCompr[mask_region]
                    hCompr_region = hCompr[mask_region]
                    E_region, H0_region, hPredict_region, R2_region, confIntE_region, confIntH0_region, fitError_region = \
                                  compressionFitChadwick(hCompr_region, fCompr_region, DIAMETER)
                    if not fitError_region:
                        R2CRITERION = 0.9
                        validatedFit_region = (R2_region > R2CRITERION)
                else:
                    validatedFit_region, fitError_region = False, True
                
                if fitError_region:
                    validatedFit_region = False
                    E_region, H0_region, R2_region  = np.nan, np.nan, np.nan
                
                dictRegionFit['regionFitNames'].append(regionFitNames)
                dictRegionFit['Npts'].append(Npts_region)
                dictRegionFit['E'].append(E_region)
                dictRegionFit['H0'].append(H0_region)
                dictRegionFit['R2'].append(R2_region)
                dictRegionFit['fitError'].append(fitError_region)
                dictRegionFit['validatedFit'].append(validatedFit_region)
#                 dictRegionFit['hPredict_region'].append(hPredict_region)
            
                for k in range(len(dictRegionFit['regionFitNames'])):
                    rFN = dictRegionFit['regionFitNames'][k]
                    results['H0Chadwick_'+rFN].append(dictRegionFit['H0'][k])
                    results['EChadwick_'+rFN].append(dictRegionFit['E'][k])
                    results['R2Chadwick_'+rFN].append(dictRegionFit['R2'][k])
                    results['Npts_'+rFN].append(dictRegionFit['Npts'][k])
                    results['validatedFit_'+rFN].append(dictRegionFit['validatedFit'][k])

            # Complete fig 1, 2, 3 with the results of the fit
            if PLOT:
                # fig1
                if not fitError:
                    if validatedFit:
                        ax1.plot(thisCompDf['T'].values, thisCompDf['D3'].values-DIAMETER, color = 'chartreuse', linestyle = '-', linewidth = 1.25)
                    else:
                        ax1.plot(thisCompDf['T'].values, thisCompDf['D3'].values-DIAMETER, color = 'gold', linestyle = '-', linewidth = 1.25)
                else:
                    ax1.plot(thisCompDf['T'].values, thisCompDf['D3'].values-DIAMETER, color = 'crimson', linestyle = '-', linewidth = 1.25)


                fig1.suptitle(results['cellID'][-1])

                # fig2 & fig3
                colSp = (i-1) % nColsSubplot
                rowSp = (i-1) // nColsSubplot
                # ax2[i-1] with the 1 line plot
                if nRowsSubplot == 1:
                    thisAx2 = ax2[colSp]
                    thisAx3 = ax3[colSp]
                elif nRowsSubplot >= 1:
                    thisAx2 = ax2[rowSp,colSp]
                    thisAx3 = ax3[rowSp,colSp]

                thisAx2.plot(hCompr,fCompr,'b-', linewidth = 0.8)
                thisAx2.plot(hRelax,fRelax,'r-', linewidth = 0.8)
                titleText = results['cellID'][-1] + '__c' + str(i)
                legendText = ''
                thisAx2.set_xlabel('h (nm)')
                thisAx2.set_ylabel('f (pN)')


                if not fitError:
                    legendText += 'H0 = {:.1f}nm\nE = {:.2e}Pa\nR2 = {:.3f}'.format(H0, E, R2)
                    thisAx2.plot(hPredict,fCompr,'k--', linewidth = 0.8, label = legendText)
                    thisAx2.legend(loc = 'upper right', prop={'size': 6})
                    if not validatedFit:
                        titleText += '\nNON VALIDATED'
                    for i in range(len(dictRegionFit['regionFitNames'])):
                        rFN = dictRegionFit['regionFitNames'][i]

                    thisAx3.plot(stressCompr, strainCompr, 'b+')
                    thisAx3.plot([np.percentile(stressCompr,10), np.percentile(stressCompr,90)], 
                                 [np.percentile(stressCompr,10)/E, np.percentile(stressCompr,90)/E],
                                 'r--', linewidth = 1.2, label = legendText)
                    thisAx3.legend(loc = 'lower right', prop={'size': 6})
                    thisAx3.set_xlabel('sigma (Pa)')
                    thisAx3.set_ylabel('epsilon')

                else:
                    titleText += '\nFIT ERROR'

                thisAx2.title.set_text(titleText)
                thisAx3.title.set_text(titleText)

                for item in ([thisAx2.title, thisAx2.xaxis.label, \
                              thisAx2.yaxis.label] + thisAx2.get_xticklabels() + thisAx2.get_yticklabels()):
                    item.set_fontsize(9)
                for item in ([thisAx3.title, thisAx3.xaxis.label, \
                              thisAx3.yaxis.label] + thisAx3.get_xticklabels() + thisAx3.get_yticklabels()):
                    item.set_fontsize(9)

                    
            # (5) hysteresis (its definition may change)
            try:
                results['hysteresis'].append(hCompr[0] - hRelax[-1])
            except:
                print('hysteresis computation failed, see hCompr and hRelax below:')
                print(hCompr, hRelax)
                print('Details of the delimitation of the curve')
                print(jStart, jMax, jStop, offsetStop, thisCompDf.shape[0])
                print(offsetStart2)
                print(thisCompDf.B.values)
                results['hysteresis'].append(np.nan)
                
        else: # The compression curve was detected as not suitable for analysis
            generalBug = True
            validatedThickness = False
            results['initialThickness'].append(np.nan)
            results['minThickness'].append(np.nan)
            results['maxIndent'].append(np.nan)
            results['previousThickness'].append(np.nan)
            results['surroundingThickness'].append(np.nan)
            results['ctFieldThickness'].append(np.nan)
            results['ctFieldFluctuAmpli'].append(np.nan)
            results['validatedThickness'].append(validatedThickness)
            validatedFit = False
            results['critFit'].append('Not relevant')
            results['H0Chadwick'].append(np.nan)
            results['EChadwick'].append(np.nan)
            results['R2Chadwick'].append(np.nan)
            results['EChadwick_CIWidth'].append(np.nan)
            results['validatedFit'].append(validatedFit)
            results['comments'].append('Unspecified bug in the code')
            results['hysteresis'].append(np.nan)
            regionFitsNames = ['f<150pN', 'f<100pN', 's<150Pa']
            for rFN in regionFitsNames:
                results['H0Chadwick_'+rFN].append(np.nan)
                results['EChadwick_'+rFN].append(np.nan)
                results['R2Chadwick_'+rFN].append(np.nan)
                results['Npts_'+rFN].append(np.nan)
                results['validatedFit_'+rFN].append(False)

        if not doThisCompAnalysis:
            print('Curve not suitable for analysis !')
            print(results['cellID'][-1])
            print('Compression no ' + str(i))
    
    # Rescale fig3 axes
    if PLOT:
        eMin, eMax = 1, 0
        sMin, sMax = 1000, 0
        for i in range(1, Ncomp+1):
            colSp = (i-1) % nColsSubplot
            rowSp = (i-1) // nColsSubplot
            # ax2[i-1] with the 1 line plot
            if nRowsSubplot == 1:
                thisAx3 = ax3[colSp]
            elif nRowsSubplot >= 1:
                thisAx3 = ax3[rowSp,colSp]
            title = thisAx3.title.get_text()
            if not 'NON VALIDATED' in title:
                if thisAx3.get_ylim()[0] < eMin:
                    eMin = thisAx3.get_ylim()[0]
                if thisAx3.get_ylim()[1] > eMax:
                    eMax = thisAx3.get_ylim()[1]
                if thisAx3.get_xlim()[0] < sMin:
                    sMin = thisAx3.get_xlim()[0]
                if thisAx3.get_xlim()[1] > sMax:
                    sMax = thisAx3.get_xlim()[1]
        for i in range(1, Ncomp+1):
            colSp = (i-1) % nColsSubplot
            rowSp = (i-1) // nColsSubplot
            # ax2[i-1] with the 1 line plot
            if nRowsSubplot == 1:
                thisAx3 = ax3[colSp]
            elif nRowsSubplot >= 1:
                thisAx3 = ax3[rowSp,colSp]
            title = thisAx3.title.get_text()
            if not 'NON VALIDATED' in title:
                thisAx3.set_xlim([sMin, sMax])
                thisAx3.set_ylim([eMin, eMax])
    
    # Save the figures
    if PLOT:
        archiveFig(fig1, ax1, name=results['cellID'][-1] + '_h(t)', figSubDir = 'MecaAnalysis_allCells')
        archiveFig(fig2, ax2, name=results['cellID'][-1] + '_F(h)', figSubDir = 'MecaAnalysis_allCells')
        archiveFig(fig3, ax3, name=results['cellID'][-1] + '_sig(eps)', figSubDir = 'MecaAnalysis_allCells')
        if PLOT_SHOW:
            fig1.show()
            fig2.tight_layout()
            fig2.show()
        else:
            plt.close('all')

    return(results)



def createDataDict_meca(list_mecaFiles, listColumnsMeca, PLOT):
    expDf = getExperimentalConditions()
    tableDict = {}
    Nfiles = len(list_mecaFiles)
    PLOT_SHOW = (Nfiles<11)
    if not PLOT_SHOW:
        plt.ioff()
    for c in listColumnsMeca:
        tableDict[c] = []
    for f in list_mecaFiles: #[:10]:
        tS_DataFilePath = os.path.join(timeSeriesDataDir, f)
        current_tsDF = pd.read_csv(tS_DataFilePath, ';')
         # MAIN SUBFUNCTION
        current_resultDict = analyseTimeSeries_meca(f, current_tsDF, expDf, 
                                                    listColumnsMeca, PLOT, PLOT_SHOW)
        for k in current_resultDict.keys():
            tableDict[k] += current_resultDict[k]
    plt.ion()
    return(tableDict)



def computeGlobalTable_meca(task = 'fromScratch', fileName = 'Global_MecaData', save = False, PLOT = False, \
                            source = 'Matlab', listColumnsMeca=listColumnsMeca):
    """
    Compute the GlobalTable_meca from the time series data files.
    Option task='fromScratch' will analyse all the time series data files and construct a new GlobalTable from them regardless of the existing GlobalTable.
    Option task='updateExisting' will open the existing GlobalTable and determine which of the time series data files are new ones, and will append the existing GlobalTable with the data analysed from those new fils.
    listColumnsMeca have to contain all the fields of the table that will be constructed.
    """
    top = time.time()
    
#     list_mecaFiles = [f for f in os.listdir(timeSeriesDataDir) \
#                       if (os.path.isfile(os.path.join(timeSeriesDataDir, f)) and f.endswith(".csv") \
#                       and ('R40' in f))] # Change to allow different formats in the future
    
    suffixPython = '_PY'
    if source == 'Matlab':
        list_mecaFiles = [f for f in os.listdir(timeSeriesDataDir) \
                      if (os.path.isfile(os.path.join(timeSeriesDataDir, f)) and f.endswith(".csv") \
                      and ('R40' in f) and not (suffixPython in f))]
        
    elif source == 'Python':
        list_mecaFiles = [f for f in os.listdir(timeSeriesDataDir) \
                      if (os.path.isfile(os.path.join(timeSeriesDataDir, f)) and f.endswith(".csv") \
                      and ('R40' in f) and (suffixPython in f))]
    
#     print(list_mecaFiles)
    
    if task == 'fromScratch':
        # create a dict containing the data
        tableDict = createDataDict_meca(list_mecaFiles, listColumnsMeca, PLOT) # MAIN SUBFUNCTION
        # create the dataframe from it
        meca_DF = pd.DataFrame(tableDict)
        
        # last step: now that the dataFrame is complete, one can use "compStartTimeThisDay" col to compute the start time of each compression relative to the first one done this day.
        allDates = list(meca_DF['date'].unique())
        for d in allDates:
            subDf = meca_DF.loc[meca_DF['date'] == d]
            experimentStartTime = np.min(subDf['compStartTimeThisDay'])
            meca_DF['compStartTimeThisDay'].loc[meca_DF['date'] == d] = meca_DF['compStartTimeThisDay'] - experimentStartTime
        
    elif task == 'updateExisting':
        # get existing table
        try:
            savePath = os.path.join(dataDir, (fileName + '.csv'))
            existing_meca_DF = pd.read_csv(savePath, sep=';')
        except:
            print('No existing table found')
            
        # find which of the time series files are new
        new_list_mecaFiles = []
        for f in list_mecaFiles:
            split_f = f.split('_')
            currentCellID = split_f[0] + '_' + split_f[1] + '_' + split_f[2] + '_' + split_f[3]
            if currentCellID not in existing_meca_DF.cellID.values:
                new_list_mecaFiles.append(f)
                
        # create the dict with new data
        new_tableDict = createDataDict_meca(new_list_mecaFiles, listColumnsMeca, PLOT) # MAIN SUBFUNCTION
        # create the dataframe from it
        new_meca_DF = pd.DataFrame(new_tableDict)
        # fuse the existing table with the new one
        meca_DF = pd.concat([existing_meca_DF, new_meca_DF])
        
        # Reset the relative time in the day for all rows and recompute it (not elegant but simpler to write)
#         meca_DF['compStartTimeThisDay'] = meca_DF['compAbsStartTime']
#         allDates = list(meca_DF['date'].unique())
#         for d in allDates:
#             subDf = meca_DF.loc[meca_DF['date'] == d]
#             experimentStartTime = np.min(subDf['compStartTimeThisDay'])
#             meca_DF['compStartTimeThisDay'].loc[meca_DF['date'] == d] = meca_DF['compStartTimeThisDay'] - experimentStartTime
    
    for c in meca_DF.columns:
            if 'Unnamed' in c:
                meca_DF = meca_DF.drop([c], axis=1)
    
    if save:
        saveName = fileName + '.csv'
        savePath = os.path.join(dataDir, saveName)
        meca_DF.to_csv(savePath, sep=';')
    
    delta = time.time() - top
    print(delta)
    
    return(meca_DF)
            

    
def getGlobalTable_meca(fileName = 'Global_mecaData'):
    try:
        savePath = os.path.join(dataDir, (fileName + '.csv'))
        meca_DF = pd.read_csv(savePath, sep=';')
        print('Extracted a table with ' + str(meca_DF.shape[0]) + ' lines and ' + str(meca_DF.shape[1]) + ' columns.')
    except:
        print('No existing table found')
    for c in meca_DF.columns:
            if 'Unnamed' in c:
                meca_DF = meca_DF.drop([c], axis=1)
    
    if 'ExpDay'in meca_DF.columns:
        dateExemple = meca_DF.loc[meca_DF.index[0],'ExpDay']
        if not ('manipID' in meca_DF.columns):
            meca_DF['manipID'] = meca_DF['ExpDay'] + '_' + meca_DF['CellID'].apply(lambda x: x.split('_')[0])
            
    elif 'date'in meca_DF.columns:
        dateExemple = meca_DF.loc[meca_DF.index[0],'date']
        if not ('manipID' in meca_DF.columns):
            meca_DF['manipID'] = meca_DF['date'] + '_' + meca_DF['cellName'].apply(lambda x: x.split('_')[0])
    
    if re.match(dateFormatExcel, dateExemple):
        print('bad date')
    return(meca_DF)

In [85]:
# computeGlobalTable_meca(task = 'updateExisting', fileName = 'Global_MecaData_Py', save = False, PLOT = False) # task = 'updateExisting'

In [86]:
computeGlobalTable_meca(task = 'fromScratch', fileName = 'Global_MecaData_Py2', save = True, PLOT = False, source = 'Python') # task = 'updateExisting'

Extracted a table with 57 lines and 24 columns.
optical index correction already in Float64 type.
hysteresis computation failed, see hCompr and hRelax below:
[609.01062542 606.23363109 599.10968367 597.7937514  582.05095772
 566.89394871 554.02164151 545.27685556 535.16568107 532.95398637
 530.94988558 524.86593934 523.5706601  509.84235789 505.93209438
 505.70486107 497.34036414 497.88895314 500.41447123 492.99887269
 491.26384879 489.04073457 483.11514792 483.14037761 479.46277491
 478.83199599 469.57095728 473.1642715  465.7272313  459.65930033
 453.65486843] []
Details of the delimitation of the curve
3 34 6 57 63
1
[ 5.573342  5.571114  5.57557   5.580026  5.923138  6.197182  8.063132
  8.72262  11.176762 15.531388 16.816944 18.883414 19.615312 20.36392
 21.129238 21.916836 22.722258 25.261064 26.15115  27.049034 27.975882
 28.918326 29.879708 30.860028 31.864856 32.88528  33.923528 34.981828
 36.06575  37.161926 39.417776 40.574108 41.749378 42.9447   44.162302
 43.743438 42.5380

,date,cellName,cellID,manipID,compNum,compDuration,compStartTime,compAbsStartTime,compStartTimeThisDay,initialThickness,minThickness,maxIndent,previousThickness,surroundingThickness,validatedThickness,ctFieldThickness,ctFieldFluctuAmpli,H0Chadwick,EChadwick,R2Chadwick,EChadwick_CIWidth,hysteresis,critFit,validatedFit,comments,H0Chadwick_f<150pN,EChadwick_f<150pN,R2Chadwick_f<150pN,Npts_f<150pN,validatedFit_f<150pN,H0Chadwick_f<100pN,EChadwick_f<100pN,R2Chadwick_f<100pN,Npts_f<100pN,validatedFit_f<100pN,H0Chadwick_s<150Pa,EChadwick_s<150Pa,R2Chadwick_s<150Pa,Npts_s<150Pa,validatedFit_s<150Pa
0,21-01-21,M1_P1_C1,21-01-21_M1_P1_C1,21-01-21_M1,1,1s,3.788,9174.132,0.000,1320.097601,774.632475,545.465126,1297.673227,1269.270179,True,1258.650163,329.524206,1491.247456,847.688047,0.993476,64.523881,157.312571,R2 > 0.9,True,ok,1442.106609,1296.957308,0.991545,25.0,True,1432.767648,1473.341639,0.981308,19.0,True,1491.247456,847.688047,0.993476,59.0,True
1,21-01-21,M1_P1_C1,21-01-21_M1_P1_C1,21-01-21_M1,2,1s,18.305,9188.649,14.517,1434.268846,789.558000,644.710845,1278.626098,1316.648649,True,1258.650163,329.524206,1641.226993,635.092948,0.995567,40.347161,168.170843,R2 > 0.9,True,ok,1602.889589,795.752765,0.994938,22.0,True,1599.336637,821.860990,0.985975,16.0,True,1641.226993,635.092948,0.995567,56.0,True
2,21-01-21,M1_P1_C1,21-01-21_M1_P1_C1,21-01-21_M1,3,1s,32.323,9202.667,28.535,1577.487620,705.006996,872.480623,1419.361019,1335.793396,True,1258.650163,329.524206,1828.166418,421.123920,0.997080,20.689111,377.696212,R2 > 0.9,True,ok,1774.478159,561.749505,0.991846,26.0,True,1757.349718,646.439844,0.984394,20.0,True,1828.166418,421.123920,0.997080,59.0,True
3,21-01-21,M1_P1_C1,21-01-21_M1_P1_C1,21-01-21_M1,4,1s,46.342,9216.686,42.554,1246.273543,550.385065,695.888478,1191.377539,1176.121921,True,1258.650163,329.524206,1471.211373,572.493406,0.996684,31.375317,267.765065,R2 > 0.9,True,ok,1392.621397,1022.137154,0.987761,22.0,True,1373.646191,1271.969266,0.991884,16.0,True,1471.211373,572.493406,0.996684,58.0,True
4,21-01-21,M1_P1_C1,21-01-21_M1_P1_C1,21-01-21_M1,5,1s,60.368,9230.712,56.580,1366.423208,670.595212,695.827997,1231.052130,1284.735203,True,1258.650163,329.524206,1597.711691,592.130346,0.989430,55.881729,227.360639,R2 > 0.9,True,ok,1508.317836,1116.218815,0.979425,24.0,True,1482.797201,1528.157224,0.975894,18.0,True,1597.711691,592.130346,0.989430,58.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618,21-10-25,M2_P1_C9,21-10-25_M2_P1_C9,21-10-25_M2,6,1s,64.468,26428.095,12402.519,558.248525,408.525720,149.722805,446.153122,444.344024,True,443.612135,70.493108,637.953584,948.654137,0.986734,108.487860,32.398797,R2 > 0.9,True,ok,637.953584,948.654137,0.986734,54.0,True,636.903835,963.877980,0.985638,52.0,True,637.953584,948.654137,0.986734,54.0,True
619,21-10-25,M2_P1_C9,21-10-25_M2_P1_C9,21-10-25_M2,7,1s,76.256,26439.883,12414.307,535.747783,396.373165,139.374618,443.448131,435.254696,True,443.612135,70.493108,602.448433,1085.070480,0.987972,114.287115,31.026078,R2 > 0.9,True,ok,602.448433,1085.070480,0.987972,58.0,True,603.790823,1060.210075,0.986981,54.0,True,602.448433,1085.070480,0.987972,58.0,True
620,21-10-25,M2_P1_C9,21-10-25_M2_P1_C9,21-10-25_M2,8,1s,88.037,26451.664,12426.088,384.734201,254.001703,130.732498,426.000252,414.557844,True,443.612135,70.493108,452.490067,1191.317754,0.995302,77.179047,36.095780,R2 > 0.9,True,ok,452.490067,1191.317754,0.995302,55.0,True,452.923092,1184.664403,0.990072,39.0,True,454.364534,1154.073662,0.992315,43.0,True
621,21-10-25,M2_P1_C9,21-10-25_M2_P1_C9,21-10-25_M2,9,1s,99.831,26463.458,12437.882,379.204066,252.474800,126.729265,412.498575,419.464858,True,443.612135,70.493108,446.777864,1228.536227,0.993143,95.593629,8.941257,R2 > 0.9,True,ok,446.777864,1228.536227,0.993143,56.0,True,448.943241,1184.439826,0.986858,40.0,True,449.965900,1162.062359,0.989288,43.0,Tr

In [41]:
getGlobalTable_meca().tail()

Extracted a table with 6150 lines and 20 columns.


,ExpType,ExpDay,CellID,CellName,CompNum,TpsComp,MaxIndent,MinThickness,Hysteresis,CompTime,...,SurroundingThickness,PreviousThickness,H0Chadwick,EChadwick,CiEChadwick,R2Chadwick,FitParams,Validated,Comments,manipID
6145,3T3aSFL-A8_nodrug,21-06-17,M2_P1_C12,21-06-17_M2_P1_C12,6,1s,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Strain100-R20.9,0,Couldn't be fitted,21-06-17_M2
6146,3T3aSFL-A8_nodrug,21-06-17,M2_P1_C12,21-06-17_M2_P1_C12,7,1s,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Strain100-R20.9,0,Couldn't be fitted,21-06-17_M2
6147,3T3aSFL-A8_nodrug,21-06-17,M2_P1_C12,21-06-17_M2_P1_C12,8,1s,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Strain100-R20.9,0,Couldn't be fitted,21-06-17_M2
6148,3T3aSFL-A8_nodrug,21-06-17,M2_P1_C12,21-06-17_M2_P1_C12,9,1s,64.512472,118.090149,-0.121251,99.541673,...,187.426750,172.161996,237.445377,7956.419593,4358.645354,0.731828,Strain100-R20.9,0,Has large jumps & Bad R2 on fit,21-06-17_M2
6149,3T3aSFL-A8_nodrug,21-06-17,M2_P1_C12,21-06-17_M2_P1_C12,10,1s,73.110975,132.492617,0.357010,111.458527,...,202.847877,204.894050,242.264687,9038.707611,2822.429026,0.912460,Strain100-R20.9,1,Succes!,21-06-17_M2


In [42]:
getGlobalTable_meca('Global_MecaData_Py').tail()

Extracted a table with 3790 lines and 41 columns.


,date,cellName,cellID,manipID,compNum,compDuration,compStartTime,compAbsStartTime,compStartTimeThisDay,initialThickness,...,H0Chadwick_f<100pN,EChadwick_f<100pN,R2Chadwick_f<100pN,Npts_f<100pN,validatedFit_f<100pN,H0Chadwick_s<150Pa,EChadwick_s<150Pa,R2Chadwick_s<150Pa,Npts_s<150Pa,validatedFit_s<150Pa
3785,21-09-09,M2_P1_C9,21-09-09_M2_P1_C9,21-09-09_M2,6,1s,67.163,78498.851,8908.816,223.193493,...,NaN,NaN,NaN,0.0,False,NaN,NaN,NaN,0.0,False
3786,21-09-09,M2_P1_C9,21-09-09_M2_P1_C9,21-09-09_M2,7,1s,79.867,78511.555,8921.520,248.390286,...,NaN,NaN,NaN,5.0,False,NaN,NaN,NaN,0.0,False
3787,21-09-09,M2_P1_C9,21-09-09_M2_P1_C9,21-09-09_M2,8,1s,92.576,78524.264,8934.229,244.999992,...,NaN,NaN,NaN,8.0,False,NaN,NaN,NaN,0.0,False
3788,21-09-09,M2_P1_C9,21-09-09_M2_P1_C9,21-09-09_M2,9,1s,105.273,78536.961,8946.926,260.173735,...,NaN,NaN,NaN,3.0,False,NaN,NaN,NaN,0.0,False
3789,21-09-09,M2_P1_C9,21-09-09_M2_P1_C9,21-09-09_M2,10,1s,117.969,78549.657,8959.622,246.932128,...,NaN,NaN,NaN,8.0,False,NaN,NaN,NaN,0.0,False


### Fluorescence

In [71]:
def getFluoData(save = False):
    # Getting the table
    fluoDataFile = 'FluoQuantification.csv'
    fluoDataFilePath = os.path.join(dataDir, fluoDataFile)
    fluoDF = pd.read_csv(fluoDataFilePath, sep=';',header=0)
    print('Extracted a table with ' + str(fluoDF.shape[0]) + ' lines and ' + str(fluoDF.shape[1]) + ' columns.')
    # Cleaning the table
    try:
        for c in fluoDF.columns:
            if 'Unnamed' in c:
                fluoDF = fluoDF.drop([c], axis=1)
        
    except:
        print('Unexpected bug with the cleaning step')

    if save:
        saveName = 'FluoQuantification.csv'
        savePath = os.path.join(dataDir, saveName)
        fluoDF.to_csv(savePath, sep=';')

    
    return(fluoDF)

In [72]:
getFluoData().head()

Extracted a table with 209 lines and 2 columns.


,cellID,meanFluoPeakAmplitude
0,21-01-18_M1-2_P1_C5,1798.786253
1,21-01-18_M1-2_P1_C6,768.192944
2,21-01-18_M1-2_P1_C7,686.701203
3,21-01-18_M1-2_P1_C8,1655.534620
4,21-01-18_M1-2_P1_C9,187.560043


### Comparison meca tables from matlab vs python

* Comparison - Per compression

Problems with an offset in the indexation of compression in the matlab table

In [45]:
# Comparison of values obtained with the python code vs matlab

# table_ExpConditions = getExperimentalConditions()
# table_fluo = getFluoData()

# mecaPy = getGlobalTable_meca('Global_MecaData_Py')
# mecaPy = pd.merge(table_ExpConditions, mecaPy, how="inner", on='manipID')
# mecaPy = pd.merge(mecaPy, table_fluo, how="left", on='cellID')
# mecaPy = mecaPy.sort_values(["cellID", "compNum"], ascending = (True, True),ignore_index=True)
# # mecaPy = mecaPy.loc[(mecaPy['cell subtype'] == 'aSFL') | (mecaPy['cell subtype'] == 'aSFL-6FP')]
# mecaMat = getGlobalTable_meca('Global_MecaData')
# mecaMat = pd.merge(table_ExpConditions, mecaMat, how="inner", on='manipID')
# mecaMat = pd.merge(mecaMat, table_fluo, how="left", left_on='CellName', right_on='cellID')
# mecaMat = mecaMat.sort_values(["CellName", "CompNum"], ascending = (True, True),ignore_index=True)

# FiltersPy = [(mecaMat['Validated'] == True), (mecaPy['validatedFit'] == True)]
# filterGlobal = np.ones((mecaPy.shape[0]), dtype=bool)
# for fltr in FiltersPy:
#     filterGlobal = filterGlobal & fltr
# mecaPyF = mecaPy.loc[filterGlobal]
# mecaPyF_light = mecaPyF[['compNum','H0Chadwick','EChadwick','R2Chadwick','EChadwick_CIWidth']]
# mecaPyF.tail()
# np.sum(mecaPyF['validatedFit'].values)
# mecaPyF

In [46]:
# FiltersMat = [(mecaPy['validatedFit'] == True), (mecaMat['Validated'] == True)]
# filterGlobal = np.ones((mecaMat.shape[0]), dtype=bool)
# for fltr in FiltersPy:
#     filterGlobal = filterGlobal & fltr
# mecaMatF = mecaMat.loc[filterGlobal]
# mecaMatF = mecaMatF.rename(columns={'CompNum': 'compNum', 'CiEChadwick': 'EChadwick_CIWidth'})
# mecaMatF_light = mecaMatF[['compNum','H0Chadwick','EChadwick','R2Chadwick','EChadwick_CIWidth']]
# # mecaMat_light.index = [i for i in range(Bbis.shape[0])]
# mecaMatF

In [47]:
# mecaCompare = (mecaPyF_light-mecaMatF_light)/mecaMatF_light
# Th = 0.1
# mecaCompare['Evaries'] = (np.abs(mecaCompare.EChadwick) > Th)
# mecaCompare['H0varies'] = (np.abs(mecaCompare.H0Chadwick) > Th)
# mecaCompare['cellID'] = mecaPyF['cellID']
# mecaCompare['compNum'] = mecaPyF['compNum']
# mecaCompare['cellID2'] = mecaMatF['CellName']
# mecaCompare['compNum2'] = mecaMatF['compNum']
# mecaCompare = mecaCompare[['H0Chadwick','EChadwick','R2Chadwick','EChadwick_CIWidth','Evaries','H0varies','cellID','compNum','cellID2','compNum2']]
# print('Nb of signif changes in E : {:.0f} / {:.0f} ; in H0 : {:.0f} / {:.0f}'.format(np.sum(mecaCompare['Evaries']), mecaCompare.shape[0], np.sum(mecaCompare['H0varies']), mecaCompare.shape[0]))
# mecaCompare

* Comparison - Per cell

In [48]:
# # Comparison of values obtained with the python code vs matlab

# table_ExpConditions = getExperimentalConditions()
# table_fluo = getFluoData()

# mecaPy = getGlobalTable_meca('Global_MecaData_Py')
# mecaPy = pd.merge(table_ExpConditions, mecaPy, how="inner", on='manipID')
# mecaPy = pd.merge(mecaPy, table_fluo, how="left", on='cellID')
# mecaPy = mecaPy.sort_values(["cellID", "compNum"], ascending = (True, True),ignore_index=True)
# # mecaPy = mecaPy.loc[(mecaPy['cell subtype'] == 'aSFL') | (mecaPy['cell subtype'] == 'aSFL-6FP')]
# mecaMat = getGlobalTable_meca('Global_MecaData')
# mecaMat = pd.merge(table_ExpConditions, mecaMat, how="inner", on='manipID')
# mecaMat = pd.merge(mecaMat, table_fluo, how="left", left_on='CellName', right_on='cellID')
# mecaMat = mecaMat.sort_values(["CellName", "CompNum"], ascending = (True, True),ignore_index=True)

# FiltersPy = [(mecaMat['Validated'] == True), (mecaPy['validatedFit'] == True)]
# filterGlobal = np.ones((mecaPy.shape[0]), dtype=bool)
# for fltr in FiltersPy:
#     filterGlobal = filterGlobal & fltr
# mecaPyF = mecaPy.loc[filterGlobal]

# cellID = 'cellID'
# group = mecaPyF.groupby(cellID)
# dictAggMean = getDictAggMean(mecaPyF)
# mecaPyF_perCell = group.agg(dictAggMean)

# mecaPyF_perCell_light = mecaPyF_perCell[['H0Chadwick','EChadwick','R2Chadwick','EChadwick_CIWidth']]
# mecaPyF_perCell_light.head()

In [49]:
# FiltersMat = [(mecaPy['validatedFit'] == True), (mecaMat['Validated'] == True)]
# filterGlobal = np.ones((mecaMat.shape[0]), dtype=bool)
# for fltr in FiltersPy:
#     filterGlobal = filterGlobal & fltr
# mecaMatF = mecaMat.loc[filterGlobal]
# mecaMatF = mecaMatF.rename(columns={'CompNum': 'compNum', 'CiEChadwick': 'EChadwick_CIWidth'})

# cellID = 'CellName'
# group = mecaMatF.groupby(cellID)
# dictAggMean = getDictAggMean(mecaMatF)
# mecaMatF_perCell = group.agg(dictAggMean)


# mecaMatF_perCell_light = mecaMatF_perCell[['H0Chadwick','EChadwick','R2Chadwick','EChadwick_CIWidth']]
# # mecaMat_light.index = [i for i in range(Bbis.shape[0])]
# # mecaMatF_perCell
# mecaMatF_perCell_light.head()

In [50]:
# mecaCompare = (mecaPyF_perCell_light-mecaMatF_perCell_light)/mecaMatF_perCell_light
# listE = []
# listH0 = []
# for i in range(1,20):
#     Th = 0.05*i
#     mecaCompare['Evaries'] = (np.abs(mecaCompare.EChadwick) > Th)
#     mecaCompare['H0varies'] = (np.abs(mecaCompare.H0Chadwick) > Th)
#     mecaCompare['cellID'] = mecaPyF_perCell['cellID']
#     mecaCompare['cellID2'] = mecaMatF_perCell['CellName']
#     listE.append(np.sum(mecaCompare['Evaries']))
#     listH0.append(np.sum(mecaCompare['H0varies']))
#     print('For T = {:.0f}%, E : {:.0f}/{:.0f} ; H0 : {:.0f}/{:.0f}'\
#           .format(Th*100, np.sum(mecaCompare['Evaries']), mecaCompare.shape[0], np.sum(mecaCompare['H0varies']), mecaCompare.shape[0]))
# # mecaCompare #.loc[mecaCompare['Evaries']]

## Data import & DataFrame formatting

In [87]:
pd.set_option('max_columns', None)
# pd.reset_option('max_columns')
pd.set_option('max_rows', None)
pd.reset_option('max_rows')

In [88]:
# getExperimentalConditions().head()
# getGlobalTable_ctField().head()
# getGlobalTable_meca().head()
# getFluoData().head()

In [89]:
# GlobalTable_ctField

GlobalTable_ctField = getGlobalTable_ctField()
table_ExpConditions = getExperimentalConditions()
table_fluo = getFluoData()
GlobalTable_ctField = pd.merge(table_ExpConditions, GlobalTable_ctField, how="inner", on='manipID',
#     left_on=None,right_on=None,left_index=False,right_index=False,sort=True,
#     suffixes=("_x", "_y"),copy=True,indicator=False,validate=None,
)
GlobalTable_ctField = pd.merge(GlobalTable_ctField, table_fluo, how="left", on='cellID',
#     left_on=None,right_on=None,left_index=False,right_index=False,sort=True,
#     suffixes=("_x", "_y"),copy=True,indicator=False,validate=None,
)
print('Merged table has ' + str(GlobalTable_ctField.shape[0]) + ' lines and ' + str(GlobalTable_ctField.shape[1]) + ' columns.')

GlobalTable_ctField.head()

Extracted a table with 40 lines and 12 columns.
Extracted a table with 57 lines and 24 columns.
optical index correction already in Float64 type.
Extracted a table with 209 lines and 2 columns.
Merged table has 40 lines and 36 columns.


,date_x,manip,experimentType,drug,substrate,objective magnification,scale pixel per um,objective immersion,optical index correction,magnetic field correction,cell type,cell subtype,bead type,bead diameter,beads bright spot delta,normal field,ramp field,compression duration,with fluo images,normal field multi images,multi image Z step,loop structure,comments,manipID,date_y,cellName,cellID,duration,medianRawB,medianThickness,1stDThickness,9thDThickness,fluctuAmpli,R2_polyFit,validated,meanFluoPeakAmplitude
0,21-02-10,M1,constant field,none,20um fibronectin discs,100X,15.8,oil,0.875,1.2,3T3,aSFL,M450,4503,0,5,<NA>,<NA>,True,3,500,298_0_1,<NA>,21-02-10_M1,21-02-10,M1_P1_C1,21-02-10_M1_P1_C1,393.550,6.0036,183.232388,103.105084,276.671473,173.566389,0.222994,1.0,NaN
1,21-02-10,M1,constant field,none,20um fibronectin discs,100X,15.8,oil,0.875,1.2,3T3,aSFL,M450,4503,0,5,<NA>,<NA>,True,3,500,298_0_1,<NA>,21-02-10_M1,21-02-10,M1_P1_C2,21-02-10_M1_P1_C2,326.388,6.0036,-58.229968,-147.203982,41.303068,188.507051,0.228033,0.0,NaN
2,21-02-10,M1,constant field,none,20um fibronectin discs,100X,15.8,oil,0.875,1.2,3T3,aSFL,M450,4503,0,5,<NA>,<NA>,True,3,500,298_0_1,<NA>,21-02-10_M1,21-02-10,M1_P1_C3,21-02-10_M1_P1_C3,327.997,6.0036,204.963636,123.738478,277.782516,154.044038,0.402168,1.0,NaN
3,21-02-10,M1,constant field,none,20um fibronectin discs,100X,15.8,oil,0.875,1.2,3T3,aSFL,M450,4503,0,5,<NA>,<NA>,True,3,500,298_0_1,<NA>,21-02-10_M1,21-02-10,M1_P1_C4,21-02-10_M1_P1_C4,326.929,6.0036,203.321336,165.769700,282.439284,116.669584,0.261129,1.0,NaN
4,21-02-10,M1,constant field,none,20um fibronectin discs,100X,15.8,oil,0.875,1.2,3T3,aSFL,M450,4503,0,5,<NA>,<NA>,True,3,500,298_0_1,<NA>,21-02-10_M1,21-02-10,M1_P1_C5,21-02-10_M1_P1_C5,260.260,6.0036,231.480125,169.220846,301.601770,132.380924,0.248455,1.0,NaN


In [90]:
# GlobalTable_meca

GlobalTable_meca = getGlobalTable_meca()
table_ExpConditions = getExperimentalConditions()
table_fluo = getFluoData()
GlobalTable_meca = pd.merge(GlobalTable_meca, table_ExpConditions, how="inner", on='manipID',
#     left_on=None,right_on=None,left_index=False,right_index=False,sort=True,
#     suffixes=("_x", "_y"),copy=True,indicator=False,validate=None,
)
GlobalTable_meca = pd.merge(GlobalTable_meca, table_fluo, how="left", left_on='CellName', right_on='cellID'
#     left_on=None,right_on=None,left_index=False,right_index=False,sort=True,
#     suffixes=("_x", "_y"),copy=True,indicator=False,validate=None,
)
print('Merged table has ' + str(GlobalTable_meca.shape[0]) + ' lines and ' + str(GlobalTable_meca.shape[1]) + ' columns.')

GlobalTable_meca.tail()

Extracted a table with 6150 lines and 20 columns.
Extracted a table with 57 lines and 24 columns.
optical index correction already in Float64 type.
Extracted a table with 209 lines and 2 columns.
Merged table has 1992 lines and 46 columns.


,ExpType,ExpDay,CellID,CellName,CompNum,TpsComp,MaxIndent,MinThickness,Hysteresis,CompTime,InitialThickness,SurroundingThickness,PreviousThickness,H0Chadwick,EChadwick,CiEChadwick,R2Chadwick,FitParams,Validated,Comments,manipID,date,manip,experimentType,drug,substrate,objective magnification,scale pixel per um,objective immersion,optical index correction,magnetic field correction,cell type,cell subtype,bead type,bead diameter,beads bright spot delta,normal field,ramp field,compression duration,with fluo images,normal field multi images,multi image Z step,loop structure,comments,cellID,meanFluoPeakAmplitude
1987,3T3aSFL-A8_nodrug,21-06-17,M2_P1_C12,21-06-17_M2_P1_C12,6,1s,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Strain100-R20.9,0,Couldn't be fitted,21-06-17_M2,21-06-17,M2,compressions,none,20um fibronectin discs,100X,15.8,oil,0.875,1.1,3T3,aSFL-A8,M450,4503,0,5,3_40,1s,True,3,500,169_133,<NA>,21-06-17_M2_P1_C12,115.022849
1988,3T3aSFL-A8_nodrug,21-06-17,M2_P1_C12,21-06-17_M2_P1_C12,7,1s,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Strain100-R20.9,0,Couldn't be fitted,21-06-17_M2,21-06-17,M2,compressions,none,20um fibronectin discs,100X,15.8,oil,0.875,1.1,3T3,aSFL-A8,M450,4503,0,5,3_40,1s,True,3,500,169_133,<NA>,21-06-17_M2_P1_C12,115.022849
1989,3T3aSFL-A8_nodrug,21-06-17,M2_P1_C12,21-06-17_M2_P1_C12,8,1s,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Strain100-R20.9,0,Couldn't be fitted,21-06-17_M2,21-06-17,M2,compressions,none,20um fibronectin discs,100X,15.8,oil,0.875,1.1,3T3,aSFL-A8,M450,4503,0,5,3_40,1s,True,3,500,169_133,<NA>,21-06-17_M2_P1_C12,115.022849
1990,3T3aSFL-A8_nodrug,21-06-17,M2_P1_C12,21-06-17_M2_P1_C12,9,1s,64.512472,118.090149,-0.121251,99.541673,182.602621,187.426750,172.161996,237.445377,7956.419593,4358.645354,0.731828,Strain100-R20.9,0,Has large jumps & Bad R2 on fit,21-06-17_M2,21-06-17,M2,compressions,none,20um fibronectin discs,100X,15.8,oil,0.875,1.1,3T3,aSFL-A8,M450,4503,0,5,3_40,1s,True,3,500,169_133,<NA>,21-06-17_M2_P1_C12,115.022849
1991,3T3aSFL-A8_nodrug,21-06-17,M2_P1_C12,21-06-17_M2_P1_C12,10,1s,73.110975,132.492617,0.357010,111.458527,205.603592,202.847877,204.894050,242.264687,9038.707611,2822.429026,0.912460,Strain100-R20.9,1,Succes!,21-06-17_M2,21-06-17,M2,compressions,none,20um fibronectin discs,100X,15.8,oil,0.875,1.1,3T3,aSFL-A8,M450,4503,0,5,3_40,1s,True,3,500,169_133,<NA>,21-06-17_M2_P1_C12,115.022849


In [91]:
# GlobalTable_mecaBis

GlobalTable_meca_Py = getGlobalTable_meca('Global_MecaData_Py')

table_ExpConditions = getExperimentalConditions()
table_fluo = getFluoData()
GlobalTable_meca_Py = pd.merge(GlobalTable_meca_Py, table_ExpConditions, how="inner", on='manipID',
#     left_on=None,right_on=None,left_index=False,right_index=False,sort=True,
#     suffixes=("_x", "_y"),copy=True,indicator=False,validate=None,
)
GlobalTable_meca_Py = pd.merge(GlobalTable_meca_Py, table_fluo, how="left", left_on='cellID', right_on='cellID'
#     left_on=None,right_on=None,left_index=False,right_index=False,sort=True,
#     suffixes=("_x", "_y"),copy=True,indicator=False,validate=None,
)
print('Merged table has ' + str(GlobalTable_meca_Py.shape[0]) + ' lines and ' + str(GlobalTable_meca_Py.shape[1]) + ' columns.')

Extracted a table with 3790 lines and 41 columns.
Extracted a table with 57 lines and 24 columns.
optical index correction already in Float64 type.
Extracted a table with 209 lines and 2 columns.
Merged table has 3790 lines and 64 columns.


In [92]:
GlobalTable_meca_Py.tail()

,date_x,cellName,cellID,manipID,compNum,compDuration,compStartTime,compAbsStartTime,compStartTimeThisDay,initialThickness,minThickness,maxIndent,previousThickness,surroundingThickness,validatedThickness,ctFieldThickness,ctFieldFluctuAmpli,H0Chadwick,EChadwick,R2Chadwick,EChadwick_CIWidth,hysteresis,critFit,validatedFit,comments_x,H0Chadwick_f<150pN,EChadwick_f<150pN,R2Chadwick_f<150pN,Npts_f<150pN,validatedFit_f<150pN,H0Chadwick_f<100pN,EChadwick_f<100pN,R2Chadwick_f<100pN,Npts_f<100pN,validatedFit_f<100pN,H0Chadwick_s<150Pa,EChadwick_s<150Pa,R2Chadwick_s<150Pa,Npts_s<150Pa,validatedFit_s<150Pa,date_y,manip,experimentType,drug,substrate,objective magnification,scale pixel per um,objective immersion,optical index correction,magnetic field correction,cell type,cell subtype,bead type,bead diameter,beads bright spot delta,normal field,ramp field,compression duration,with fluo images,normal field multi images,multi image Z step,loop structure,comments_y,meanFluoPeakAmplitude
3785,21-09-09,M2_P1_C9,21-09-09_M2_P1_C9,21-09-09_M2,6,1s,67.163,78498.851,8908.816,223.193493,138.233656,84.959837,207.373865,220.363596,True,225.772358,51.898487,267.287393,6806.895179,0.982692,879.655184,28.934547,R2 > 0.9,True,ok,NaN,NaN,NaN,4.0,False,NaN,NaN,NaN,0.0,False,NaN,NaN,NaN,0.0,False,21-09-09,M2,compressions,none,20um fibronectin discs,100X,15.8,oil,0.875,1.154,3T3,aSFL-A8-2,M450,4503,0,5,3_40,1s,True,3,500,170_133_1,<NA>,102.414382
3786,21-09-09,M2_P1_C9,21-09-09_M2_P1_C9,21-09-09_M2,7,1s,79.867,78511.555,8921.520,248.390286,156.632105,91.758181,239.047874,239.047874,True,225.772358,51.898487,282.760144,6929.677091,0.948259,1498.857628,42.730638,R2 > 0.9,True,ok,271.95618,14151.903088,0.513392,11.0,False,NaN,NaN,NaN,5.0,False,NaN,NaN,NaN,0.0,False,21-09-09,M2,compressions,none,20um fibronectin discs,100X,15.8,oil,0.875,1.154,3T3,aSFL-A8-2,M450,4503,0,5,3_40,1s,True,3,500,170_133_1,<NA>,102.414382
3787,21-09-09,M2_P1_C9,21-09-09_M2_P1_C9,21-09-09_M2,8,1s,92.576,78524.264,8934.229,244.999992,168.592016,76.407976,253.618743,255.975130,True,225.772358,51.898487,269.633303,11045.782241,0.991127,976.508246,21.117942,R2 > 0.9,True,ok,267.60991,14132.767980,0.746489,13.0,False,NaN,NaN,NaN,8.0,False,NaN,NaN,NaN,0.0,False,21-09-09,M2,compressions,none,20um fibronectin discs,100X,15.8,oil,0.875,1.154,3T3,aSFL-A8-2,M450,4503,0,5,3_40,1s,True,3,500,170_133_1,<NA>,102.414382
3788,21-09-09,M2_P1_C9,21-09-09_M2_P1_C9,21-09-09_M2,9,1s,105.273,78536.961,8946.926,260.173735,169.727001,90.446734,258.859365,255.234736,True,225.772358,51.898487,290.955108,8178.952820,0.982428,1046.340702,17.807194,R2 > 0.9,True,ok,NaN,NaN,NaN,8.0,False,NaN,NaN,NaN,3.0,False,NaN,NaN,NaN,0.0,False,21-09-09,M2,compressions,none,20um fibronectin discs,100X,15.8,oil,0.875,1.154,3T3,aSFL-A8-2,M450,4503,0,5,3_40,1s,True,3,500,170_133_1,<NA>,102.414382
3789,21-09-09,M2_P1_C9,21-09-09_M2_P1_C9,21-09-09_M2,10,1s,117.969,78549.657,8959.622,246.932128,160.104213,86.827915,238.745033,231.639213,True,225.772358,51.898487,264.588424,9322.043140,0.935663,2251.793871,40.875118,R2 > 0.9,True,ok,282.14400,5960.169580,0.945277,13.0,True,NaN,NaN,NaN,8.0,False,NaN,NaN,NaN,0.0,False,21-09-09,M2,compressions,none,20um fibronectin discs,100X,15.8,oil,0.875,1.154,3T3,aSFL-A8-2,M450,4503,0,5,3_40,1s,True,3,500,170_133_1,<NA>,102.414382


In [113]:
GlobalTable_meca_Py2 = getGlobalTable_meca('Global_MecaData_Py2')

table_ExpConditions = getExperimentalConditions()
table_fluo = getFluoData()
GlobalTable_meca_Py2 = pd.merge(GlobalTable_meca_Py2, table_ExpConditions, how="inner", on='manipID',
#     left_on=None,right_on=None,left_index=False,right_index=False,sort=True,
#     suffixes=("_x", "_y"),copy=True,indicator=False,validate=None,
)
GlobalTable_meca_Py2 = pd.merge(GlobalTable_meca_Py2, table_fluo, how="left", left_on='cellID', right_on='cellID'
#     left_on=None,right_on=None,left_index=False,right_index=False,sort=True,
#     suffixes=("_x", "_y"),copy=True,indicator=False,validate=None,
)
print('Merged table has ' + str(GlobalTable_meca_Py2.shape[0]) + ' lines and ' + str(GlobalTable_meca_Py2.shape[1]) + ' columns.')

Extracted a table with 623 lines and 41 columns.
Extracted a table with 57 lines and 24 columns.
optical index correction already in Float64 type.
Extracted a table with 209 lines and 2 columns.
Merged table has 623 lines and 64 columns.


In [115]:
GlobalTable_meca_Py2.tail()

,date_x,cellName,cellID,manipID,compNum,compDuration,compStartTime,compAbsStartTime,compStartTimeThisDay,initialThickness,minThickness,maxIndent,previousThickness,surroundingThickness,validatedThickness,ctFieldThickness,ctFieldFluctuAmpli,H0Chadwick,EChadwick,R2Chadwick,EChadwick_CIWidth,hysteresis,critFit,validatedFit,comments_x,H0Chadwick_f<150pN,EChadwick_f<150pN,R2Chadwick_f<150pN,Npts_f<150pN,validatedFit_f<150pN,H0Chadwick_f<100pN,EChadwick_f<100pN,R2Chadwick_f<100pN,Npts_f<100pN,validatedFit_f<100pN,H0Chadwick_s<150Pa,EChadwick_s<150Pa,R2Chadwick_s<150Pa,Npts_s<150Pa,validatedFit_s<150Pa,date_y,manip,experimentType,drug,substrate,objective magnification,scale pixel per um,objective immersion,optical index correction,magnetic field correction,cell type,cell subtype,bead type,bead diameter,beads bright spot delta,normal field,ramp field,compression duration,with fluo images,normal field multi images,multi image Z step,loop structure,comments_y,meanFluoPeakAmplitude
618,21-10-25,M2_P1_C9,21-10-25_M2_P1_C9,21-10-25_M2,6,1s,64.468,26428.095,12402.519,558.248525,408.525720,149.722805,446.153122,444.344024,True,443.612135,70.493108,637.953584,948.654137,0.986734,108.487860,32.398797,R2 > 0.9,True,ok,637.953584,948.654137,0.986734,54.0,True,636.903835,963.877980,0.985638,52.0,True,637.953584,948.654137,0.986734,54.0,True,21-10-25,M2,compressions,none,20um fibronectin discs,100X,15.8,oil,0.875,1.1,3T3,aSFL,M270,2691,0,5,5_40,1s,False,3,500,169_133,<NA>,NaN
619,21-10-25,M2_P1_C9,21-10-25_M2_P1_C9,21-10-25_M2,7,1s,76.256,26439.883,12414.307,535.747783,396.373165,139.374618,443.448131,435.254696,True,443.612135,70.493108,602.448433,1085.070480,0.987972,114.287115,31.026078,R2 > 0.9,True,ok,602.448433,1085.070480,0.987972,58.0,True,603.790823,1060.210075,0.986981,54.0,True,602.448433,1085.070480,0.987972,58.0,True,21-10-25,M2,compressions,none,20um fibronectin discs,100X,15.8,oil,0.875,1.1,3T3,aSFL,M270,2691,0,5,5_40,1s,False,3,500,169_133,<NA>,NaN
620,21-10-25,M2_P1_C9,21-10-25_M2_P1_C9,21-10-25_M2,8,1s,88.037,26451.664,12426.088,384.734201,254.001703,130.732498,426.000252,414.557844,True,443.612135,70.493108,452.490067,1191.317754,0.995302,77.179047,36.095780,R2 > 0.9,True,ok,452.490067,1191.317754,0.995302,55.0,True,452.923092,1184.664403,0.990072,39.0,True,454.364534,1154.073662,0.992315,43.0,True,21-10-25,M2,compressions,none,20um fibronectin discs,100X,15.8,oil,0.875,1.1,3T3,aSFL,M270,2691,0,5,5_40,1s,False,3,500,169_133,<NA>,NaN
621,21-10-25,M2_P1_C9,21-10-25_M2_P1_C9,21-10-25_M2,9,1s,99.831,26463.458,12437.882,379.204066,252.474800,126.729265,412.498575,419.464858,True,443.612135,70.493108,446.777864,1228.536227,0.993143,95.593629,8.941257,R2 > 0.9,True,ok,446.777864,1228.536227,0.993143,56.0,True,448.943241,1184.439826,0.986858,40.0,True,449.965900,1162.062359,0.989288,43.0,True,21-10-25,M2,compressions,none,20um fibronectin discs,100X,15.8,oil,0.875,1.1,3T3,aSFL,M270,2691,0,5,5_40,1s,False,3,500,169_133,<NA>,NaN
622,21-10-25,M2_P1_C9,21-10-25_M2_P1_C9,21-10-25_M2,10,1s,111.745,26475.372,12449.796,403.395397,244.482066,158.913332,422.698029,439.401100,True,443.612135,70.493108,484.565886,910.401029,0.993325,67.463801,48.554004,R2 > 0.9,True,ok,484.873292,906.413542,0.992674,56.0,True,482.158619,945.268841,0.983860,42.0,True,484.873292,906.413542,0.992674,56.0,True,21-10-25,M2,compressions,none,20um fibronectin discs,100X,15.8,oil,0.875,1.1,3T3,aSFL,M270,2691,0,5,5_40,1s,False,3,500,169_133,<NA>,NaN


In [94]:
# Experiment counter - Matlab table

# count ct field cells
cellID = 'cellID'
GlobalTable_ctField_CountCell = GlobalTable_ctField.groupby(['cell type', 'cell subtype', 'bead type', 'drug', 'substrate']).count()
GlobalTable_ctField_CountCell = GlobalTable_ctField_CountCell.loc[:, [cellID]].rename(columns={cellID : 'Count cells - ctField'})

# count meca compressions
cellID = 'CellName'
GlobalTable_meca_CountComp = GlobalTable_meca.groupby(['cell type', 'cell subtype', 'bead type', 'drug', 'substrate']).count()
GlobalTable_meca_CountComp = GlobalTable_meca_CountComp.loc[:, [cellID]].rename(columns={cellID : 'Count compressions'})

# count valid meca compressions
cellID = 'CellName'
GlobalTable_meca_CountCompOK = GlobalTable_meca[GlobalTable_meca['Validated'] == True].groupby(['cell type', 'cell subtype', 'bead type', 'drug', 'substrate']).count()
GlobalTable_meca_CountCompOK = GlobalTable_meca_CountCompOK.loc[:, [cellID]].rename(columns={cellID : 'Count OK compressions'})

# count meca cells
cellID = 'CellName'
group = GlobalTable_meca.groupby(cellID)
dictAggMean = getDictAggMean(GlobalTable_meca)
GlobalTable_meca_perCell = group.agg(dictAggMean)
GlobalTable_meca_CountCell = GlobalTable_meca_perCell.groupby(['cell type', 'cell subtype', 'bead type', 'drug', 'substrate']).count()
GlobalTable_meca_CountCell = GlobalTable_meca_CountCell.loc[:, [cellID]].rename(columns={cellID : 'Count cells - meca'})


# Fuse all the previous tables
GlobalTable_CountAll = pd.concat([GlobalTable_ctField_CountCell, GlobalTable_meca_CountCell, GlobalTable_meca_CountComp, GlobalTable_meca_CountCompOK], axis=1)
GlobalTable_CountAll = GlobalTable_CountAll.fillna(0)
GlobalTable_CountAll = GlobalTable_CountAll.loc[:,:].astype(int)
GlobalTable_CountAll

Count cells - ctField  \
cell type cell subtype bead type drug       substrate                                       
3T3       aSFL         M450      doxycyclin 20um fibronectin discs                     20   
                                            BSA coated glass                            0   
                                 none       20um fibronectin discs                     20   
                                            BSA coated glass                            0   
          aSFL-6FP     M450      doxycyclin 20um fibronectin discs                      0   
                                 none       20um fibronectin discs                      0   
          aSFL-A8      M450      doxycyclin 20um fibronectin discs                      0   
                                 none       20um fibronectin discs                      0   

                                                                    Count cells - meca  \
cell type cell subtype bead type drug       substrate                                    
3T3       aSFL         M450      doxycyclin 20um fibronectin discs                  19   
                                            BSA coated glass                        16   
                                 none       20um fibronectin discs                  32   
                                            BSA coated glass                        13   
          aSFL-6FP     M450      doxycyclin 20um fibronectin discs                  26   
                                 none       20um fibronectin discs                  29   
          aSFL-A8      M450      doxycyclin 20um fibronectin discs                  25   
                                 none       20um fibronectin discs                  23   

                                                                    Count compressions  \
cell type cell subtype bead type drug       substrate                                    
3T3       aSFL         M450      doxycyclin 20um fibronectin discs                 255   
                                            BSA coated glass                       161   
                                 none       20um fibronectin discs                 392   
                                            BSA coated glass                       117   
          aSFL-6FP     M450      doxycyclin 20um fibronectin discs                 254   
                                 none       20um fibronectin discs                 293   
          aSFL-A8      M450      doxycyclin 20um fibronectin discs                 260   
                                 none       20um fibronectin discs                 260   

                                                                    Count OK compressions  
cell type cell subtype bead type drug       substrate                                      
3T3       aSFL         M450      doxycyclin 20um fibronectin discs                    152  
                                            BSA coated glass                          105  
                                 none       20um fibronectin discs                    314  
                                            BSA coated glass                           60  
          aSFL-6FP     M450      doxycyclin 20um fibronectin discs                    161  
                                 none       20um fibronectin discs                    160  
          aSFL-A8      M450      doxycyclin 20um fibronectin discs                    168  
                                 none       20um fibronectin discs                    183

In [95]:
# Experiment counter - Python table

# count ct field cells
cellID = 'cellID'
GlobalTable_ctField_CountCell = GlobalTable_ctField.groupby(['cell type', 'cell subtype', 'bead type', 'drug', 'substrate']).count()
GlobalTable_ctField_CountCell = GlobalTable_ctField_CountCell.loc[:, [cellID]].rename(columns={cellID : 'Count cells - ctField'})

# count meca compressions
cellID = 'cellID'
GlobalTable_meca_CountComp = GlobalTable_meca_Py.groupby(['cell type', 'cell subtype', 'bead type', 'drug', 'substrate']).count()
GlobalTable_meca_CountComp = GlobalTable_meca_CountComp.loc[:, [cellID]].rename(columns={cellID : 'Count compressions'})

# count valid meca compressions
cellID = 'cellID'
GlobalTable_meca_CountCompOK = GlobalTable_meca_Py[GlobalTable_meca_Py['validatedFit'] == True].groupby(['cell type', 'cell subtype', 'bead type', 'drug', 'substrate']).count()
GlobalTable_meca_CountCompOK = GlobalTable_meca_CountCompOK.loc[:, [cellID]].rename(columns={cellID : 'Count OK compressions'})

# count meca cells
cellID = 'cellID'
group = GlobalTable_meca_Py.groupby(cellID)
dictAggMean = getDictAggMean(GlobalTable_meca_Py)
GlobalTable_meca_perCell = group.agg(dictAggMean)
GlobalTable_meca_CountCell = GlobalTable_meca_perCell.groupby(['cell type', 'cell subtype', 'bead type', 'drug', 'substrate']).count()
GlobalTable_meca_CountCell = GlobalTable_meca_CountCell.loc[:, [cellID]].rename(columns={cellID : 'Count cells - meca'})


# Fuse all the previous tables
GlobalTable_CountAll = pd.concat([GlobalTable_ctField_CountCell, GlobalTable_meca_CountCell, GlobalTable_meca_CountComp, GlobalTable_meca_CountCompOK], axis=1)
GlobalTable_CountAll = GlobalTable_CountAll.fillna(0)
GlobalTable_CountAll = GlobalTable_CountAll.loc[:,:].astype(int)
GlobalTable_CountAll

Count cells - ctField  \
cell type cell subtype bead type drug               substrate                                       
3T3       aSFL         M270      none               20um fibronectin discs                      0   
                       M270_M450 none               20um fibronectin discs                      0   
                       M450      dmso               20um fibronectin discs                      0   
                                 dmso. doxycyclin   20um fibronectin discs                      0   
                                 doxycyclin         20um fibronectin discs                     20   
                                                    BSA coated glass                            0   
                                 none               20um fibronectin discs                     20   
                                                    BSA coated glass                            0   
                                 smifh2             20um fibronectin discs                      0   
                                 smifh2. doxycyclin 20um fibronectin discs                      0   
                       M450_M270 none               20um fibronectin discs                      0   
          aSFL-6FP     M450      doxycyclin         20um fibronectin discs                      0   
                                 none               20um fibronectin discs                      0   
          aSFL-6FP-2   M450      doxycyclin         20um fibronectin discs                      0   
                                 none               20um fibronectin discs                      0   
          aSFL-A8      M450      doxycyclin         20um fibronectin discs                      0   
                                 none               20um fibronectin discs                      0   
          aSFL-A8-2    M450      doxycyclin         20um fibronectin discs                      0   
                                 none               20um fibronectin discs                      0   

                                                                            Count cells - meca  \
cell type cell subtype bead type drug               substrate                                    
3T3       aSFL         M270      none               20um fibronectin discs                   7   
                       M270_M450 none               20um fibronectin discs                   7   
                       M450      dmso               20um fibronectin discs                  23   
                                 dmso. doxycyclin   20um fibronectin discs                   9   
                                 doxycyclin         20um fibronectin discs                  28   
                                                    BSA coated glass                        16   
                                 none               20um fibronectin discs                  37   
                                                    BSA coated glass                        13   
                                 smifh2             20um fibronectin discs                  24   
                                 smifh2. doxycyclin 20um fibronectin discs                   9   
                       M450_M270 none               20um fibronectin discs                   9   
          aSFL-6FP     M450      doxycyclin         20um fibronectin discs                  26   
                                 none               20um fibronectin discs                  29   
          aSFL-6FP-2   M450      doxycyclin         20um fibronectin discs                  20   
                                 none               20um fibronectin discs                  22   
          aSFL-A8      M450      doxycyclin         20um fibronectin discs                  25   
                                 none               20um fibronectin discs                  23   
          aSFL-A8-2    M450      doxycyclin         20um fibronectin discs                  

In [96]:
# Experiment counter - Python table

# count ct field cells
cellID = 'cellID'
GlobalTable_ctField_CountCell = GlobalTable_ctField.groupby(['cell type', 'cell subtype', 'bead type', 'drug', 'substrate']).count()
GlobalTable_ctField_CountCell = GlobalTable_ctField_CountCell.loc[:, [cellID]].rename(columns={cellID : 'Count cells - ctField'})

# count meca compressions
cellID = 'cellID'
GlobalTable_meca_CountComp = GlobalTable_meca_Py2.groupby(['cell type', 'cell subtype', 'bead type', 'drug', 'substrate']).count()
GlobalTable_meca_CountComp = GlobalTable_meca_CountComp.loc[:, [cellID]].rename(columns={cellID : 'Count compressions'})

# count valid meca compressions
cellID = 'cellID'
GlobalTable_meca_CountCompOK = GlobalTable_meca_Py2[GlobalTable_meca_Py2['validatedFit'] == True].groupby(['cell type', 'cell subtype', 'bead type', 'drug', 'substrate']).count()
GlobalTable_meca_CountCompOK = GlobalTable_meca_CountCompOK.loc[:, [cellID]].rename(columns={cellID : 'Count OK compressions'})

# count meca cells
cellID = 'cellID'
group = GlobalTable_meca_Py2.groupby(cellID)
dictAggMean = getDictAggMean(GlobalTable_meca_Py2)
GlobalTable_meca_perCell = group.agg(dictAggMean)
GlobalTable_meca_CountCell = GlobalTable_meca_perCell.groupby(['cell type', 'cell subtype', 'bead type', 'drug', 'substrate']).count()
GlobalTable_meca_CountCell = GlobalTable_meca_CountCell.loc[:, [cellID]].rename(columns={cellID : 'Count cells - meca'})


# Fuse all the previous tables
GlobalTable_CountAll = pd.concat([GlobalTable_ctField_CountCell, GlobalTable_meca_CountCell, GlobalTable_meca_CountComp, GlobalTable_meca_CountCompOK], axis=1)
GlobalTable_CountAll = GlobalTable_CountAll.fillna(0)
GlobalTable_CountAll = GlobalTable_CountAll.loc[:,:].astype(int)
GlobalTable_CountAll

Count cells - ctField  \
cell type cell subtype bead type drug       substrate                                       
3T3       aSFL         M270      none       20um fibronectin discs                      0   
                       M450      doxycyclin 20um fibronectin discs                     20   
                                 none       20um fibronectin discs                     20   
          aSFL-A8-2    M450      doxycyclin 20um fibronectin discs                      0   

                                                                    Count cells - meca  \
cell type cell subtype bead type drug       substrate                                    
3T3       aSFL         M270      none       20um fibronectin discs                  27   
                       M450      doxycyclin 20um fibronectin discs                   0   
                                 none       20um fibronectin discs                  27   
          aSFL-A8-2    M450      doxycyclin 20um fibronectin discs                   3   

                                                                    Count compressions  \
cell type cell subtype bead type drug       substrate                                    
3T3       aSFL         M270      none       20um fibronectin discs                 299   
                       M450      doxycyclin 20um fibronectin discs                   0   
                                 none       20um fibronectin discs                 292   
          aSFL-A8-2    M450      doxycyclin 20um fibronectin discs                  32   

                                                                    Count OK compressions  
cell type cell subtype bead type drug       substrate                                      
3T3       aSFL         M270      none       20um fibronectin discs                    231  
                       M450      doxycyclin 20um fibronectin discs                      0  
                                 none       20um fibronectin discs                    289  
          aSFL-A8-2    M450      doxycyclin 20um fibronectin discs                     17

In [59]:
# Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), (GlobalTable_meca_Py['cell subtype'] == 'aSFL-A8')]
# GlobalTable_meca_PyF = GlobalTable_meca_Py
# for fltr in Filters:
#         GlobalTable_meca_PyF = GlobalTable_meca_PyF.loc[fltr]
# # GlobalTable_mecaBis
# cellID = 'cellID'
# group = GlobalTable_meca_PyF.groupby(cellID)
# dictAggMean = getDictAggMean(GlobalTable_meca_PyF)
# GlobalTable_meca_PyF_perCell = group.agg(dictAggMean)
# GlobalTable_meca_PyF_perCell

### Custom data export

In [60]:
# 21-06-28 Export of E - h data for Julien
# GlobalTable_meca_Py_expJ = GlobalTable_meca_Py.loc[GlobalTable_meca_Py['validatedFit'] & GlobalTable_meca_Py['validatedThickness']]\
#                                                [['cell type', 'cell subtype', 'bead type', 'drug', 'substrate', 'compNum', \
#                                                  'EChadwick', 'H0Chadwick', 'surroundingThickness', 'ctFieldThickness']]
# GlobalTable_meca_Py_expJ = GlobalTable_meca_Py_expJ.reset_index()
# GlobalTable_meca_Py_expJ = GlobalTable_meca_Py_expJ.drop('index', axis=1)
# savePath = os.path.join(dataDir, 'mecanicsData_3T3.csv')
# GlobalTable_meca_Py_expJ.to_csv(savePath, sep=';')

## Plots

### Functions

In [97]:
Styles = {''} # Project of automatic formatting according to the type of data

renameDict1 = {'SurroundingThickness':'Thickness (nm) [b&a]',\
               'surroundingThickness':'Thickness (nm) [b&a]',\
               'ctFieldThickness':'Thickness at low force (nm)',\
               'EChadwick': 'E Chadwick (Pa)',\
               'medianThickness': 'Median Thickness (nm)',\
               'fluctuAmpli': 'Fluctuations Amplitude (nm)',\
               'meanFluoPeakAmplitude' : 'Fluo Intensity (a.u.)',\
               'none':'control',\
               'doxycyclin':'expressing iMC linker',\
               'none & BSA coated glass':'control & non adherent',\
               'doxycyclin & BSA coated glass':'iMC & non adherent',\
               'none & 20um fibronectin discs':'control & adherent on fibro',\
               'doxycyclin & 20um fibronectin discs':'iMC & adherent on fibro',\
               'BSA coated glass & none':'control & non adherent',\
               'BSA coated glass & doxycyclin':'iMC & non adherent',\
               '20um fibronectin discs & none':'control & adherent on fibro',\
               '20um fibronectin discs & doxycyclin':'iMC & adherent on fibro',\
               'aSFL & none':'aSFL control',\
               'aSFL & doxycyclin':'aSFL iMC',\
               'aSFL-6FP & none':'aSFL-6FP control',\
               'aSFL-6FP & doxycyclin':'aSFL-6FP long-iMC',\
               'aSFL-6FP-2 & none':'aSFL-6FP-2 control',\
               'aSFL-6FP-2 & doxycyclin':'aSFL-6FP-2 long-iMC',\
               'aSFL-A8 & none':'aSFL-A8 control',\
               'aSFL-A8 & doxycyclin':'aSFL-A8 iMC',\
               'aSFL-A8-2 & none':'aSFL-A8-2 control',\
               'aSFL-A8-2 & doxycyclin':'aSFL-A8-2 iMC',\
               'dmso' : 'DMSO, no linker', 
               'smifh2' : 'SMIFH2, no linker', 
               'dmso, doxycyclin' : 'DMSO, iMC linker', 
               'smifh2, doxycyclin' : 'SMIFH2, iMC linker'}

styleDict1 =  {'none & BSA coated glass':{'color':'#ff9896','marker':'^'},\
               'doxycyclin & BSA coated glass':{'color':'#d62728','marker':'^'},\
               'none & 20um fibronectin discs':{'color':'#aec7e8','marker':'o'},\
               'doxycyclin & 20um fibronectin discs':{'color':'#1f77b4','marker':'o'},\
               'none':{'color':'#aec7e8','marker':'o'},\
               'doxycyclin':{'color':'#1f77b4','marker':'o'},\
               'BSA coated glass & none':{'color':'#ff9896','marker':'^'},\
               'BSA coated glass & doxycyclin':{'color':'#d62728','marker':'^'},\
               '20um fibronectin discs & none':{'color':'#aec7e8','marker':'o'},\
               '20um fibronectin discs & doxycyclin':{'color':'#1f77b4','marker':'o'},\
               'aSFL':{'color':'#1f77b4','marker':'o'},\
               'aSFL-6FP':{'color':'#2ca02c','marker':'o'},\
               'aSFL-A8':{'color':'#ff7f0e','marker':'o'}, \
               'aSFL & none':{'color':'#aec7e8','marker':'o'},\
               'aSFL & doxycyclin':{'color':'#1f77b4','marker':'o'},\
               'aSFL-6FP & none':{'color':'#98df8a','marker':'o'},\
               'aSFL-6FP & doxycyclin':{'color':'#2ca02c','marker':'o'},\
               'aSFL-A8 & none':{'color':'#ffbb78','marker':'o'},\
               'aSFL-A8 & doxycyclin':{'color':'#ff7f0e','marker':'o'}}



These functions use matplotlib.pyplot and seaborn libraries to display 1D categorical or 2D plots

In [98]:
def D1Plot(data, CondCol=[], Parameters=[], Filters=[], Boxplot=True, AvgPerCell=False, cellID='cellID', co_order=[],\
           stats=True, statMethod='Mann-Whitney', box_pairs=[], figSizeFactor = 1, markerSizeFactor=1, orientation = 'h', useHue = False):
    data_filtered = data
    for fltr in Filters:
        data_filtered = data_filtered.loc[fltr]    
    NCond = len(CondCol)
    
    if NCond == 1:
        CondCol = CondCol[0]
        
    elif NCond > 1:
        newColName = ''
        for i in range(NCond):
            newColName += CondCol[i]
            newColName += ' & '
        newColName = newColName[:-3]
        data_filtered[newColName] = ''
        for i in range(NCond):
            data_filtered[newColName] += data_filtered[CondCol[i]].astype(str)
            data_filtered[newColName] = data_filtered[newColName].apply(lambda x : x + ' & ')
        data_filtered[newColName] = data_filtered[newColName].apply(lambda x : x[:-3])
        CondCol = newColName
    
    if AvgPerCell:
        group = data_filtered.groupby(cellID)
        dictAggMean = getDictAggMean(data_filtered)
#         dictAggMean['EChadwick'] = 'median'
        data_filtered = group.agg(dictAggMean)
        
    data_filtered.sort_values(CondCol, axis=0, ascending=True, inplace=True)
    
    if len(co_order) > 0:
        p = getStyleLists(co_order, styleDict1)
    else:
        p = sns.color_palette()
    
    NPlots = len(Parameters)
    Conditions = list(data_filtered[CondCol].unique())
    if len(co_order) == 0:
        co_order = Conditions
    
    if orientation == 'h':
        fig, ax = plt.subplots(1, NPlots, figsize = (5*NPlots*NCond*figSizeFactor, 5))
    elif orientation == 'v':
        fig, ax = plt.subplots(NPlots, 1, figsize = (5*NCond*figSizeFactor, 5*NPlots))
    
    markerSize = 5*markerSizeFactor
    
    if NPlots == 1:
        ax = np.array([ax])
    
    for k in range(NPlots):

        if 'EChadwick' in Parameters[k]:
            ax[k].set_yscale('log')


        if Boxplot:
            sns.boxplot(x=CondCol, y=Parameters[k], data=data_filtered, ax=ax[k], 
                        color='w', linewidth = 2, width = 0.5, showfliers = False, order= co_order)
            # data_filtered.boxplot(column=Parameters[k], by = CondCol, ax=ax[k],showfliers = False) # linewidth = 2, width = 0.5

        if stats:
            if len(box_pairs) == 0:
                box_pairs = makeBoxPairs(co_order)
            addStat(ax[k], data_filtered, box_pairs, Parameters[k], CondCol, test = statMethod)
            # add_stat_annotation(ax[k], x=CondCol, y=Parameters[k], data=data_filtered,box_pairs = box_pairs,test=statMethod, text_format='star',loc='inside', verbose=2)
        
        if not useHue:
            sns.swarmplot(x=CondCol, y=Parameters[k], data=data_filtered, ax=ax[k], order = co_order,
                          size=markerSize, edgecolor='k', linewidth = 1*markerSizeFactor, palette = p)
        else:
            sns.swarmplot(x=CondCol, y=Parameters[k], data=data_filtered, ax=ax[k], order = co_order,
                          size=markerSize, edgecolor='k', linewidth = 1*markerSizeFactor, palette = p, hue = 'manipID')
            legend = ax[k].legend()
            legend.remove()

        ax[k].set_xlabel('')
        ax[k].set_ylabel(Parameters[k])
        ax[k].tick_params(axis='x', labelrotation = 10)
        ax[k].yaxis.grid(True)           
            
#     else:
#         if Parameters[0] == 'EChadwick':
#             ax.set_yscale('log')
                
#         if Boxplot:
#             sns.boxplot(x=CondCol, y=Parameters[0], data=data_filtered, ax=ax, 
#                         color='w', linewidth = 2, width = 0.5, showfliers = False, order= co_order)
#             # data_filtered.boxplot(column=Parameters[k], by = CondCol, ax=ax[k],showfliers = False) # linewidth = 2, width = 0.5

#         if stats:
#             if len(box_pairs) == 0:
#                 box_pairs = makeBoxPairs(co_order)
#             addStat(ax, data_filtered, box_pairs, Parameters[0], CondCol, test = statMethod)
#             # add_stat_annotation(ax, x=CondCol, y=Parameters[0], data=data_filtered,box_pairs = box_pairs,test=statMethod, text_format='star',loc='inside', verbose=2)

#         sns.swarmplot(x=CondCol, y=Parameters[0], data=data_filtered, ax=ax, order= co_order, 
#                       size=markerSize, edgecolor='k',linewidth = 1*markerSizeFactor, palette = p)
    
#         ax.set_xlabel('')
#         ax.set_ylabel(Parameters[0])
#         ax.tick_params(axis='x', labelrotation = 10)
#         ax.yaxis.grid(True)
    
    plt.rcParams['axes.prop_cycle'] = my_default_color_cycle
    return(fig, ax)

In [99]:
def D2Plot(data, XCol='',YCol='',CondCol='',Filters=[], cellID='cellID', AvgPerCell=False, modelFit=False, modelType='y=ax+b',\
           xscale = 'lin', yscale = 'lin', figSizeFactor = 1):
    data_filtered = data
    for fltr in Filters:
        data_filtered = data_filtered.loc[fltr]
    
    NCond = len(CondCol)    
    if NCond == 1:
        CondCol = CondCol[0]
    elif NCond > 1:
        newColName = ''
        for i in range(NCond):
            newColName += CondCol[i]
            newColName += ' & '
        newColName = newColName[:-3]
        data_filtered[newColName] = ''
        for i in range(NCond):
            data_filtered[newColName] += data_filtered[CondCol[i]].astype(str)
            data_filtered[newColName] = data_filtered[newColName].apply(lambda x : x + ' & ')
        data_filtered[newColName] = data_filtered[newColName].apply(lambda x : x[:-3])
        CondCol = newColName
    
    if AvgPerCell:
        group = data_filtered.groupby(cellID)
        dictAggMean = getDictAggMean(data_filtered)
        data_filtered = group.agg(dictAggMean.pop(cellID)) #.reset_index(level=0, inplace=True)
        data_filtered.reset_index(level=0, inplace=True)
        
    Conditions = list(data_filtered[CondCol].unique())
    fig, ax = plt.subplots(1, 1, figsize = (8*figSizeFactor,5))
    markerSize = 5
    
    if xscale == 'log':
        ax.set_xscale('log')
    if yscale == 'log':
        ax.set_yscale('log')
    
    current_color_list = getStyleLists(Conditions, styleDict1).as_hex()
    cc = cycler(color=current_color_list)
    ax.set_prop_cycle(cc)
    
    if modelFit:
        # Tweak the style cycle to plot for each condition: the points ('o') and then the fit ('-') with the same color.
#         current_prop_cycle = plt.rcParams['axes.prop_cycle']
#         current_color_list = prop_cycle.by_key()['color']
        ncustom_color_list = list(np.array([current_color_list, current_color_list]).flatten(order='F'))
        # if new_color_list was ['red', 'green', blue'], custom_color_list is now ['red', 'red', 'green', 'green', blue', blue']
        cc = cycler(color=ncustom_color_list)
        ax.set_prop_cycle(cc)
    
    for c in Conditions:
        Xraw = data_filtered[data_filtered[CondCol] == c][XCol].values
        Yraw = data_filtered[data_filtered[CondCol] == c][YCol].values
        XYraw = np.array([Xraw,Yraw]).T
        XY = XYraw[~np.isnan(XYraw).any(axis=1), :]
        X, Y = XY[:,0], XY[:,1]
        if len(X) == 0:
            ax.plot([], [])
            if modelFit:
                ax.plot([], [])
                
        elif len(X) > 0:
            eqnText = ''

            if modelFit:
                print('Fitting condition: ' + c + ' with model ' + modelType)
                if modelType == 'y=ax+b':
                    params, results = fitLine(X, Y) # Y=a*X+b ; params[0] = b,  params[1] = a
                    pval = results.pvalues[1] # pvalue on the param 'a'
                    eqnText += " ; Y = {:.1f} X + {:.1f}".format(params[1], params[0])
                    eqnText += " ; p-val = {:.2e}".format(pval)
                    print("Y = {:.5} X + {:.5}".format(params[1], params[0]))
                    print("p-value on the 'a' coefficient: {:.4e}".format(pval))
                    print('\n')
                    fitY = params[1]*X + params[0]
                    imin = np.argmin(X)
                    imax = np.argmax(X)
                    ax.plot([X[imin],X[imax]], [fitY[imin],fitY[imax]], '--', lw = '1')

                elif modelType == 'y=A*exp(kx)':
                    params, results = fitLine(X, np.log(Y)) # Y=a*X+b ; params[0] = b,  params[1] = a
                    pval = results.pvalues[1] # pvalue on the param 'a'
                    eqnText += " ; Y = {:.1f}*exp({:.1f}*X)".format(params[0], params[1])
                    eqnText += " ; p-val = {:.2e}".format(pval)
                    print("Y = {:.5}*exp({:.5}*X)".format(np.exp(params[0]), params[1]))
                    print("p-value on the 'k' coefficient: {:.4e}".format(pval))
                    print('\n')
                    fitY = np.exp(params[0])*np.exp(params[1]*X)
                    imin = np.argmin(X)
                    imax = np.argmax(X)
                    ax.plot([X[imin],X[imax]], [fitY[imin],fitY[imax]], '--', lw = '1')
                    
            ax.plot(X, Y, 'o', markersize = markerSize, markeredgecolor='k', markeredgewidth = 1, label=c + eqnText)
            
            
    ax.set_xlabel(XCol)
    ax.set_xlim([min(0,1.1*np.min(data_filtered[XCol])), 1.1*np.max(data_filtered[XCol])])
    ax.set_ylabel(YCol)
    if not yscale == 'log':
        ax.set_ylim([min(0,1.1*np.min(data_filtered[YCol])), 1.1*np.max(data_filtered[YCol])])
    ax.legend(loc='upper left')
    return(fig, ax)

These functions use the Bokeh library to display 1D categorical or 2D plots with interactive plots. They are less flexible but can be nice to explore the data set since you can display the cellID which is the source of each point by passing your pointer over it.

In [100]:
def D1PlotInteractive(data, CondCol='',Parameters=[],Filters=[],AvgPerCell=False,cellID='cellID'):
    data_filtered = data
    for fltr in Filters:
        data_filtered = data_filtered.loc[fltr]
        
#     print(data_filtered[cellID])
    if AvgPerCell:
        group = data_filtered.groupby(cellID)
        dictAggMean = getDictAggMean(data_filtered)
        data_filtered = group.agg(dictAggMean.pop(cellID)) #.reset_index(level=0, inplace=True)
        data_filtered.reset_index(level=0, inplace=True)
    
#     return(data_filtered)
    
    NPlots = len(Parameters)
    Conditions = list(data_filtered[CondCol].unique())
    if NPlots > 1:
        plots = []
        NCond = len(Conditions)
        data_filtered['X'] = 0
        data_filtered['X_jitter'] = 0.
        dictTicks = {}
        for i in range(NCond):
            mask = data_filtered[CondCol] == Conditions[i]
            data_filtered.loc[mask, 'X'] = i+1
            dictTicks[i+1] = Conditions[i]
        for i in data_filtered.index:
            data_filtered.loc[i, 'X_jitter'] = data_filtered.loc[i, 'X'] + 0.4*(np.random.rand(1)[0]-0.5)
        source = ColumnDataSource(
            data=data_filtered[[cellID]+[CondCol]+Parameters+['X','X_jitter']]
        )        
        
        for k in range(NPlots):
            hover = HoverTool(
                tooltips=[
                    ('Cell ID', "@"+cellID),
                    (Parameters[k], "@"+Parameters[k]),
                ]
            )
            index_cmap = factor_cmap(CondCol, palette=Category10[10], factors=sorted(data_filtered[CondCol].unique()), end=1)
            p = figure(plot_width=450, plot_height=500, tools=[hover], title="InteractivePlot") # 
            p.circle('X_jitter', Parameters[k], size=8, alpha = 0.6, source=source,fill_color=index_cmap,line_color='black')
            # Format
            p.x_range = Range1d(0, NCond+1)
            p.y_range = Range1d(min(0,1.1*np.min(data_filtered[Parameters[0]])), 1.1*np.max(data_filtered[Parameters[k]]))
            p.xaxis.ticker = [i for i in range(1,NCond+1)]
            p.xaxis.major_label_overrides = dictTicks
            p.xaxis.axis_label = CondCol
            p.xaxis.axis_label_text_font_size = '18pt'
            p.xaxis.major_label_text_font_size = '16pt'
            p.yaxis.axis_label = Parameters[k]
            p.yaxis.axis_label_text_font_size = '18pt'
            p.yaxis.major_label_text_font_size = '16pt'
            
            plots.append(p)
            
        p = gridplot(plots, ncols=2, toolbar_location=None)
        
        
    else:
        hover = HoverTool(
            tooltips=[
                ('Cell ID', "@"+cellID),
                (Parameters[0], "@"+Parameters[0]),
            ]
        )
        
        NCond = len(Conditions)
        data_filtered['X'] = 0
        data_filtered['X_jitter'] = 0.
        dictTicks = {}
        for i in range(NCond):
            mask = data_filtered[CondCol] == Conditions[i]
            data_filtered.loc[mask, 'X'] = i+1
            dictTicks[i+1] = Conditions[i]
        for i in data_filtered.index:
            data_filtered.loc[i, 'X_jitter'] = data_filtered.loc[i, 'X'] + 0.4*(np.random.rand(1)[0]-0.5)
        source = ColumnDataSource(
            data=data_filtered[[cellID]+[CondCol]+Parameters+['X','X_jitter']]
        )
        index_cmap = factor_cmap(CondCol, palette=Category10[10], factors=sorted(data_filtered[CondCol].unique()), end=1)
        TOOLS = "hover,pan,box_zoom,wheel_zoom,reset,save,help"
        p = figure(plot_width=500, plot_height=500, tools=TOOLS, title="InteractivePlot") # 
        p.circle('X_jitter', Parameters[0], size=8, alpha = 0.6, source=source,fill_color=index_cmap,line_color='black')
        # Format
        p.x_range = Range1d(0, NCond+1)
        p.y_range = Range1d(min(0,1.1*np.min(data_filtered[Parameters[0]])), 1.1*np.max(data_filtered[Parameters[0]]))
        p.xaxis.ticker = [i for i in range(1,NCond+1)]
        p.xaxis.major_label_overrides = dictTicks
        p.xaxis.axis_label = CondCol
        p.xaxis.axis_label_text_font_size = '18pt'
        p.xaxis.major_label_text_font_size = '16pt'
        p.yaxis.axis_label = Parameters[0]
        p.yaxis.axis_label_text_font_size = '18pt'
        p.yaxis.major_label_text_font_size = '16pt'
    return(p)

In [101]:
def D2PlotInteractive(data, XCol='',YCol='',CondCol='',Filters=[], cellID='cellID',AvgPerCell=False):
    
    data_filtered = data
    for fltr in Filters:
        data_filtered = data_filtered.loc[fltr]
        
    if AvgPerCell:
        group = data_filtered.groupby(cellID)
        dictAggMean = getDictAggMean(data_filtered)
        data_filtered = group.agg(dictAggMean.pop(cellID)) #.reset_index(level=0, inplace=True)
        data_filtered.reset_index(level=0, inplace=True)
    
    Conditions = list(data_filtered[CondCol].unique())

    NCond = len(Conditions)
    dictTicks = {}
    for i in range(NCond):
        dictTicks[i+1] = Conditions[i]
    
    source = ColumnDataSource(
        data=data_filtered[[cellID,CondCol,XCol,YCol]]
    )
    
    hover = HoverTool(
        tooltips=[
            ('Cell ID', "@"+cellID),
            (XCol, "@"+XCol),
            (YCol, "@"+YCol),
            (CondCol, "@"+CondCol),
        ]
    )
    
    index_cmap = factor_cmap(CondCol, palette=Category10[10], factors=sorted(data_filtered[CondCol].unique()), end=1)
    TOOLS = "pan,box_zoom,wheel_zoom,reset,save,help"
    p = figure(plot_width=900, plot_height=500, tools=TOOLS, title="InteractivePlot",toolbar_location="below") # 
    p.circle(XCol, YCol, size=8, alpha = 0.6, source=source,fill_color=index_cmap,line_color='black')
    p.add_tools(hover)
    # Format
    p.x_range = Range1d(0, 1.1*np.max(data_filtered[XCol]))
    p.y_range = Range1d(0, 1.1*np.max(data_filtered[YCol]))
    p.xaxis.axis_label = XCol
    p.xaxis.axis_label_text_font_size = '18pt'
    p.xaxis.major_label_text_font_size = '16pt'
    p.yaxis.axis_label = YCol
    p.yaxis.axis_label_text_font_size = '18pt'
    p.yaxis.major_label_text_font_size = '16pt'
    return(p)

Other subfunctions useful to plot

In [102]:
def makeOrder(*args):
    order = []
    listeTuple = list(itertools.product(*args, repeat=1))
    for tup in listeTuple:
        tmpText = ''
        for word in tup:
            tmpText += word
            tmpText += ' & '
        tmpText = tmpText[:-3]
        order.append(tmpText)
    return(order)


def makeBoxPairs(O):
    return(list(itertools.combinations(O, 2)))


def renameAxes(axes, rD):
    try:
        N = len(axes)
    except:
        axes = [axes]
        N = 1
    for i in range(N):
        # set xticks
        xticksTextObject = axes[i].get_xticklabels()
        xticksList = [xticksTextObject[j].get_text() for j in range(len(xticksTextObject))]
        test_hasXLabels = (len(''.join(xticksList)) > 0)
        if test_hasXLabels:
            newXticksList = [rD.get(k, k) for k in xticksList]
            axes[i].set_xticklabels(newXticksList)
        
        # set xlabel
        xlabel = axes[i].get_xlabel()
        newXlabel = rD.get(xlabel, xlabel)
        axes[i].set_xlabel(newXlabel)
        # set ylabel
        ylabel = axes[i].get_ylabel()
        newYlabel = rD.get(ylabel, ylabel)
        axes[i].set_ylabel(newYlabel)
        

def addStat(ax, data, box_pairs, param, cond, test = 'MannWhitney', percentHeight = 98):
    refHeight = np.percentile(data[param].values, percentHeight)
    currentHeight = refHeight
    scale = ax.get_yscale()
    xTicks = ax.get_xticklabels()
    dictXTicks = {xTicks[i].get_text() : xTicks[i].get_position()[0] for i in range(len(xTicks))}
    for bp in box_pairs:
        c1 = data[data[cond] == bp[0]][param].values
        c2 = data[data[cond] == bp[1]][param].values
        if test=='Mann-Whitney':
            statistic, pval = st.mannwhitneyu(c1,c2)
        elif test=='t-test':
            statistic, pval = st.ttest_ind(c1,c2)
        text = 'ns'
        if pval < 0.05 and pval > 0.01:
            text = '*'
        elif pval < 0.01 and pval > 0.001:
            text = '**'
        elif pval < 0.001 and pval < 0.001:
            text = '***'
        elif pval < 0.0001:
            text = '****'
        ax.plot([bp[0], bp[1]], [currentHeight, currentHeight], 'k-', lw = 1)
        XposText = (dictXTicks[bp[0]]+dictXTicks[bp[1]])/2
        if scale == 'log':
            power = 0.01* (text=='ns') + 0.000 * (text!='ns')
            YposText = currentHeight*(refHeight**power)
        else:
            factor = 0.03 * (text=='ns') + 0.000 * (text!='ns')
            YposText = currentHeight + factor*refHeight
        ax.text(XposText, YposText, text, ha = 'center')
#         if text=='ns':
#             ax.text(posText, currentHeight + 0.025*refHeight, text, ha = 'center')
#         else:
#             ax.text(posText, currentHeight, text, ha = 'center')
        if scale == 'log':
            currentHeight = currentHeight*(refHeight**0.05)
        else:
            currentHeight =  currentHeight + 0.15*refHeight
    ax.set_ylim([ax.get_ylim()[0], currentHeight])
    

def getStyleCycle(co_order, styleDict):
    colors = []
    markers = []
    linestyles = []
    linewidth = []

    for co in co_order:
        coStyle = styleDict[co]
        if 'color' in coStyle.keys():
            colors.append(coStyle['color'])
        else:
            colors.append('')
        if 'marker' in coStyle.keys():
            markers.append(coStyle['marker'])
        else:
            markers.append('')
        if 'linestyle' in coStyle.keys():
            linestyles.append(coStyle['marker'])
        else:
            linestyles.append('')
        if 'linewidth' in coStyle.keys():
            linewidth.append(coStyle['linewidth'])
        else:
            linewidth.append(1)
            
    cc = (cycler(color=colors) +
          cycler(linestyle=linestyles) +
          cycler(marker=markers) +
          cycler(linewidth=linewidth))
            
    return(cc)

def getStyleLists(co_order, styleDict):
    colors = []
    markers = []
    try:
        for co in co_order:
            coStyle = styleDict[co]
            if 'color' in coStyle.keys():
                colors.append(coStyle['color'])
            else:
                colors.append('')
            if 'marker' in coStyle.keys():
                markers.append(coStyle['marker'])
            else:
                markers.append('')
        palette = sns.color_palette(colors)
    except:
        palette = sns.color_palette()
    return(palette)


def buildStyleDictMCA():
    # TBC
    styleDict = {}
    return(styleDict)

In [103]:
# Test of the averaging per cell routine

# data = GlobalTable_meca
# CondCol='drug'
# Parameters=['SurroundingThickness','EChadwick']
# Filters = [(GlobalTable_meca['Validated'] == 1)]
# AvgPerCell=True
# cellID='CellName'

# data_filtered = data
# for fltr in Filters:
#     data_filtered = data_filtered.loc[fltr]

# group = data_filtered.groupby(cellID)
# dictAggMean = getDictAggMean(data_filtered)
# data_filtered = group.agg(dictAggMean.pop(cellID)) #.reset_index(level=0, inplace=True)
# data_filtered.reset_index(level=0, inplace=True)
# data_filtered=data_filtered[[cellID]+[CondCol]+Parameters]
# print(data_filtered)

In [104]:
# Test of a routine to remove points of a list of XY positions where at least 1 of the coordinates is 'nan'

# XYraw = np.array([[np.nan, 2, 3, np.nan, 5], [10,20,30,40,50]])
# XYraw = XYraw.T
# XY = XYraw[~np.isnan(XYraw).any(axis=1), :]
# X, Y = XY[:,0], XY[:,1]
# X, Y

In [105]:
# Test of a routine to double each element in a list ; example [1, 2, 3] -> [1, 1, 2, 2, 3, 3]

# newnew_color_list = np.array([new_color_list, new_color_list])
# custom_color_list = list(np.array([new_color_list, new_color_list]).flatten(order='F'))
# custom_color_list

In [106]:
# Test of makeOrder function

# print(makeOrder(['none','doxycyclin'],['BSA coated glass','20um fibronectin discs']))
# print(makeOrder(['A','B']))
# print(makeOrder(['A','B'], ['C','D']))
# print(makeOrder(['A','B'], ['C','D'], ['E','F']))

In [107]:
# Test of makeBoxPairs function

# O = makeOrder(['none','doxycyclin'],['BSA coated glass','20um fibronectin discs'])
# makeBoxPairs(O)

In [108]:
# Test of custom cycles

cc = (cycler(color=list('rgb')) +
      cycler(linestyle=['', '', '']) +
      cycler(marker=['o','*','<']) +
     cycler(linewidth=[1,1,1]))
cc

# plt.rcParams['axes.prop_cycle'] = cc

# fig, ax = plt.subplots()
# for i in range(10):
#     ax.plot([0,1], [0,i])
# fig.show()

'r','',1,'o'
'g','',1,'*'
'b','',1,'<'


In [109]:
p = sns.color_palette("tab20").as_hex()
p[0]

'#1f77b4'

### Exploratory Graphs

#### Classic

In [ ]:
rawMecaTable = getGlobalTable_meca()
# rawMecaTable.head()

In [ ]:
# rawMecaTable.loc[]
# Agréger tous les M450 WT sous un même nom

In [ ]:
Filters = [(rawMecaTable['Validated'] == 1), ((rawMecaTable['ExpType'] == 'DictyDB_M450'))] #  | (rawMecaTable['ExpType'] == 'DictyDB_M450-Multi')
co_order = makeOrder(['02s', '05s', '1s', '2s', '4s', '7s', '12s']) # co_order = co_order,
fig, ax00 = D1Plot(rawMecaTable, CondCol=['TpsComp'],Parameters=['SurroundingThickness','EChadwick'], co_order = co_order,\
                   Filters=Filters,AvgPerCell=False,cellID='CellName',stats=False,figSizeFactor = 1.8,markerSizeFactor=0.5)
fig.suptitle('M450, various rates')
renameAxes(ax00,renameDict1)
fig.show()

In [ ]:
co_order

In [ ]:
Filters = [(rawMecaTable['Validated'] == 1), ((rawMecaTable['ExpType'] == 'DictyDB_M270'))]
fig, ax01 = D1Plot(rawMecaTable, CondCol=['TpsComp'],Parameters=['SurroundingThickness','EChadwick'],Filters=Filters,\
                   AvgPerCell=False,cellID='CellName',stats=False,figSizeFactor = 1.8,markerSizeFactor=0.5)
fig.suptitle('M270, various rates')
fig.show()

In [ ]:
Filters = [(GlobalTable_ctField['validated'] == True), (GlobalTable_ctField['medianThickness'] <= 1000)]
fig, ax = D1Plot(GlobalTable_ctField, CondCol=['drug','substrate'],Parameters=['medianThickness','fluctuAmpli'],Filters=Filters)
fig.suptitle('3T3aSFL on patterns: Ct Field')
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == 1), (GlobalTable_meca['cell subtype'] == 'aSFL')]
co_order = makeOrder(['none','doxycyclin'],['BSA coated glass','20um fibronectin discs'])
fig, ax = D1Plot(GlobalTable_meca, CondCol=['drug','substrate'],Parameters=['SurroundingThickness','EChadwick'],Filters=Filters,AvgPerCell=True,cellID='CellName', co_order=co_order)
fig.suptitle('3T3aSFL on diverse substrates: Compressions')
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == 1), (GlobalTable_meca['cell subtype'] == 'aSFL-6FP')]
fig, ax = D1Plot(GlobalTable_meca, CondCol=['drug','substrate'],Parameters=['SurroundingThickness','EChadwick'],Filters=Filters,AvgPerCell=True,cellID='CellName')
fig.suptitle('3T3aSFL on patterns: Compressions')
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == 1), (GlobalTable_meca['substrate'] == '20um fibronectin discs')]
co_order = makeOrder([['aSFL','aSFL-6FP'],['none','doxycyclin']])
fig, ax = D1Plot(GlobalTable_meca, CondCol=['cell subtype','drug'],Parameters=['SurroundingThickness','EChadwick'],Filters=Filters,AvgPerCell=True,cellID='CellName',co_order=co_order)
fig.suptitle('3T3aSFL on diverse substrates: Compressions')
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == 1), (GlobalTable_meca['substrate'] == '20um fibronectin discs')]
co_order = makeOrder([['aSFL','aSFL-6FP'],['none','doxycyclin']])
fig, ax = D1Plot(GlobalTable_meca, CondCol=['cell subtype','drug'],Parameters=['SurroundingThickness','EChadwick'],Filters=Filters,AvgPerCell=True,cellID='CellName',co_order=co_order)
fig.suptitle('3T3aSFL SHORT vs LONG linker: Compressions')
fig.show()

In [ ]:
Filters = [(GlobalTable_ctField['validated'] == True), (GlobalTable_ctField['cell subtype'] == 'aSFL')]
fig, ax = D2Plot(GlobalTable_ctField, XCol='medianThickness',YCol='fluctuAmpli',CondCol = ['drug'], Filters=Filters, modelFit=True)
fig.suptitle('3T3aSFL - Dh = f(H)')
archiveFig(fig, ax, name='aSFL_Dh(h)_drug', figDir = todayFigDir, figSubDir='ThicknessPlots')
fig.show()

In [ ]:
# Same as above without the 2 thickest cells
Filters = [(GlobalTable_ctField['validated'] == True), (GlobalTable_ctField['cell subtype'] == 'aSFL'), (GlobalTable_ctField['medianThickness'] <= 700)]
fig, ax = D2Plot(GlobalTable_ctField, XCol='medianThickness',YCol='fluctuAmpli',CondCol = ['drug'], Filters=Filters, modelFit=True)
fig.suptitle('3T3aSFL - Dh = f(H)')
archiveFig(fig, ax, name='aSFL_Dh(h)_drug_wo2LastPoints', figDir = todayFigDir, figSubDir='ThicknessPlots')
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL')]
fig, ax = D2Plot(GlobalTable_meca, XCol='SurroundingThickness',YCol='EChadwick',CondCol = ['substrate','drug'],\
           Filters=Filters, cellID = 'CellName', AvgPerCell=True, modelFit=False, modelType='y=A*exp(kx)', yscale = 'log')
fig.suptitle('3T3aSFL: E(h)')
renameAxes(ax,renameDict1)
ax.legend(loc='upper right')
archiveFig(fig, ax, name='aSFL_E(h)_drug&substrate_01', figDir = todayFigDir, figSubDir='')
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL'), (GlobalTable_meca['substrate'] == '20um fibronectin discs')]
fig, ax = D2Plot(GlobalTable_meca, XCol='SurroundingThickness',YCol='EChadwick',CondCol = ['substrate','drug'],\
           Filters=Filters, cellID = 'CellName', AvgPerCell=False, modelFit=False, modelType='y=A*exp(kx)', xscale = 'log', \
                 yscale = 'log')
fig.suptitle('3T3aSFL: E(h)')
renameAxes(ax,renameDict1)
ax.set_xlim([90, 1500])
ax.legend(loc='upper right')
archiveFig(fig, ax, name='aSFL_E(h)_drug&substrate_00_allComp', figDir = todayFigDir, figSubDir='')
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['substrate'] == '20um fibronectin discs')]
fig, ax = D2Plot(GlobalTable_meca, XCol='SurroundingThickness',YCol='EChadwick',CondCol = ['cell subtype','drug'],\
           Filters=Filters, cellID = 'CellName', AvgPerCell=True, modelFit=False, modelType='y=A*exp(kx)', yscale = 'log')
fig.suptitle('3T3aSFL: E(h)')
renameAxes(ax,renameDict1)
ax.legend(loc='upper right')
archiveFig(fig, ax, name='aSFL_E(h)_drug&substrate_02', figDir = todayFigDir, figSubDir='')
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['substrate'] == '20um fibronectin discs')]
fig, ax = D2Plot(GlobalTable_meca, XCol='SurroundingThickness',YCol='EChadwick',CondCol = ['cell subtype','drug'],\
           Filters=Filters, cellID = 'CellName', AvgPerCell=False, modelFit=False, modelType='y=A*exp(kx)', yscale = 'log')
fig.suptitle('3T3aSFL: E(h)')
renameAxes(ax,renameDict1)
ax.legend(loc='upper right')
archiveFig(fig, ax, name='aSFL_E(h)_drug&substrate_02_allComp', figDir = todayFigDir, figSubDir='')
fig.show()

In [ ]:
co_o = ['BSA coated glass & none', 'BSA coated glass & doxycyclin', '20um fibronectin discs & doxycyclin', '20um fibronectin discs & none']
# getStyleLists(co_o, styleDict1)
sns.color_palette(['#ff9896', '#d62728', '#1f77b4', '#aec7e8'])

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL')]
fig, ax = D2Plot(GlobalTable_meca, XCol='SurroundingThickness',YCol='EChadwick',CondCol = ['substrate','drug'],\
           Filters=Filters, cellID = 'CellName', AvgPerCell=True, modelFit=False, modelType='y=A*exp(kx)', yscale = 'log')
fig.suptitle('3T3aSFL: E(h)')
ax.legend(loc='upper right')
archiveFig(fig, ax, name='aSFL_E(h)_drug&substrate', figDir = todayFigDir, figSubDir='')
fig.show()

In [ ]:
# xticksTextObject = ax.get_xticklabels()
# xticksList = [xticksTextObject[j].get_text() for j in range(len(xticksTextObject))]
# newXticksList = [renameDict1.get(k, k) for k in xticksList]
# newXticksList

In [ ]:
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs')]
co_order = makeOrder(['aSFL','aSFL-A8','aSFL-6FP'], ['none','doxycyclin'])
box_pairs=[('aSFL & none', 'aSFL & doxycyclin'),
 ('aSFL-A8 & none', 'aSFL-A8 & doxycyclin'),
 ('aSFL-6FP & none', 'aSFL-6FP & doxycyclin'),
 ('aSFL & none', 'aSFL-A8 & none'),
 ('aSFL & none', 'aSFL-6FP & none')]
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['cell subtype','drug'],Parameters=['surroundingThickness','EChadwick'],\
                 Filters=Filters,AvgPerCell=False,cellID='cellID',co_order=co_order,box_pairs=box_pairs,stats=True)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL - All linker types: All Compressions')
fig.show()

In [130]:
GlobalTable_meca_Py2.tail()

,date_x,cellName,cellID,manipID,compNum,compDuration,compStartTime,compAbsStartTime,compStartTimeThisDay,initialThickness,minThickness,maxIndent,previousThickness,surroundingThickness,validatedThickness,ctFieldThickness,ctFieldFluctuAmpli,H0Chadwick,EChadwick,R2Chadwick,EChadwick_CIWidth,hysteresis,critFit,validatedFit,comments_x,H0Chadwick_f<150pN,EChadwick_f<150pN,R2Chadwick_f<150pN,Npts_f<150pN,validatedFit_f<150pN,H0Chadwick_f<100pN,EChadwick_f<100pN,R2Chadwick_f<100pN,Npts_f<100pN,validatedFit_f<100pN,H0Chadwick_s<150Pa,EChadwick_s<150Pa,R2Chadwick_s<150Pa,Npts_s<150Pa,validatedFit_s<150Pa,date_y,manip,experimentType,drug,substrate,objective magnification,scale pixel per um,objective immersion,optical index correction,magnetic field correction,cell type,cell subtype,bead type,bead diameter,beads bright spot delta,normal field,ramp field,compression duration,with fluo images,normal field multi images,multi image Z step,loop structure,comments_y,meanFluoPeakAmplitude
618,21-10-25,M2_P1_C9,21-10-25_M2_P1_C9,21-10-25_M2,6,1s,64.468,26428.095,12402.519,558.248525,408.525720,149.722805,446.153122,444.344024,True,443.612135,70.493108,637.953584,948.654137,0.986734,108.487860,32.398797,R2 > 0.9,True,ok,637.953584,948.654137,0.986734,54.0,True,636.903835,963.877980,0.985638,52.0,True,637.953584,948.654137,0.986734,54.0,True,21-10-25,M2,compressions,none,20um fibronectin discs,100X,15.8,oil,0.875,1.1,3T3,aSFL,M270,2691,0,5,5_40,1s,False,3,500,169_133,<NA>,NaN
619,21-10-25,M2_P1_C9,21-10-25_M2_P1_C9,21-10-25_M2,7,1s,76.256,26439.883,12414.307,535.747783,396.373165,139.374618,443.448131,435.254696,True,443.612135,70.493108,602.448433,1085.070480,0.987972,114.287115,31.026078,R2 > 0.9,True,ok,602.448433,1085.070480,0.987972,58.0,True,603.790823,1060.210075,0.986981,54.0,True,602.448433,1085.070480,0.987972,58.0,True,21-10-25,M2,compressions,none,20um fibronectin discs,100X,15.8,oil,0.875,1.1,3T3,aSFL,M270,2691,0,5,5_40,1s,False,3,500,169_133,<NA>,NaN
620,21-10-25,M2_P1_C9,21-10-25_M2_P1_C9,21-10-25_M2,8,1s,88.037,26451.664,12426.088,384.734201,254.001703,130.732498,426.000252,414.557844,True,443.612135,70.493108,452.490067,1191.317754,0.995302,77.179047,36.095780,R2 > 0.9,True,ok,452.490067,1191.317754,0.995302,55.0,True,452.923092,1184.664403,0.990072,39.0,True,454.364534,1154.073662,0.992315,43.0,True,21-10-25,M2,compressions,none,20um fibronectin discs,100X,15.8,oil,0.875,1.1,3T3,aSFL,M270,2691,0,5,5_40,1s,False,3,500,169_133,<NA>,NaN
621,21-10-25,M2_P1_C9,21-10-25_M2_P1_C9,21-10-25_M2,9,1s,99.831,26463.458,12437.882,379.204066,252.474800,126.729265,412.498575,419.464858,True,443.612135,70.493108,446.777864,1228.536227,0.993143,95.593629,8.941257,R2 > 0.9,True,ok,446.777864,1228.536227,0.993143,56.0,True,448.943241,1184.439826,0.986858,40.0,True,449.965900,1162.062359,0.989288,43.0,True,21-10-25,M2,compressions,none,20um fibronectin discs,100X,15.8,oil,0.875,1.1,3T3,aSFL,M270,2691,0,5,5_40,1s,False,3,500,169_133,<NA>,NaN
622,21-10-25,M2_P1_C9,21-10-25_M2_P1_C9,21-10-25_M2,10,1s,111.745,26475.372,12449.796,403.395397,244.482066,158.913332,422.698029,439.401100,True,443.612135,70.493108,484.565886,910.401029,0.993325,67.463801,48.554004,R2 > 0.9,True,ok,484.873292,906.413542,0.992674,56.0,True,482.158619,945.268841,0.983860,42.0,True,484.873292,906.413542,0.992674,56.0,True,21-10-25,M2,compressions,none,20um fibronectin discs,100X,15.8,oil,0.875,1.1,3T3,aSFL,M270,2691,0,5,5_40,1s,False,3,500,169_133,<NA>,NaN


In [132]:
Filters = [(GlobalTable_meca_Py2['validatedFit'] == True), 
           (GlobalTable_meca_Py2['validatedThickness'] == True), 
           (GlobalTable_meca_Py2['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py2['date_x'].apply(lambda x : x in ['21-10-18', '21-10-25']))]
co_order = makeOrder(['M270','M450'])

fig, ax = D1Plot(GlobalTable_meca_Py2, CondCol=['bead type'],Parameters=['H0Chadwick_f<150pN','EChadwick_f<150pN'],\
                 Filters=Filters,AvgPerCell=True,cellID='cellID',co_order=co_order,stats=True, useHue = True)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL - Oct 21 experiment, bead types')
fig.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [128]:
Filters = [(GlobalTable_meca_Py2['validatedFit'] == True), 
           (GlobalTable_meca_Py2['validatedThickness'] == True), 
           (GlobalTable_meca_Py2['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py2['date_x'].apply(lambda x : x in ['21-10-18', '21-10-25']))]
fig, ax = D2Plot(GlobalTable_meca_Py2, XCol='surroundingThickness',YCol='EChadwick_f<150pN',CondCol = ['bead type'],\
           Filters=Filters, cellID = 'cellID', AvgPerCell=True, modelFit=False, modelType='y=A*exp(kx)',xscale = 'log', yscale = 'log')
fig.suptitle('3T3aSFL: E(h)')
# archiveFig(fig, ax, name='aSFL_E(h)_drug&substrate', figDir = todayFigDir, figSubDir='')
fig.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\JosephVermeil\anaconda3\lib\site-packages\ipykernel_launcher.py:97: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.


#### Plots that exlore the impact of the delay before starting experiment

In [ ]:
figSubDir = 'ExploratoryPlots'
GlobalTable_meca_Py['compStartTimeThisDay_min'] = GlobalTable_meca_Py['compStartTimeThisDay']/60
GlobalTable_meca_Py

In [ ]:
figSubDir = 'ExploratoryPlots'
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True),\
           (GlobalTable_meca_Py['cell subtype'] == 'aSFL'), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs')]
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['manipID','drug'],Parameters=['surroundingThickness','EChadwick'],\
                 Filters=Filters,AvgPerCell=True,cellID='cellID',stats=False)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL - Compressions - Differences with manip')
for i in range(len(ax)):
    xTicks = ax[i].get_xticklabels()
    for j in range(len(xTicks)):
        xTicks[j].set_fontsize(9)
archiveFig(fig, ax, name='3T3aSFL-Compressions-DifferencesWithManip_PYTHONTABLE', figSubDir = figSubDir)
fig.show()

In [ ]:
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True),\
           (GlobalTable_meca_Py['cell subtype'] == 'aSFL'), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs')]
fig, ax = D2Plot(GlobalTable_meca_Py, XCol='compStartTimeThisDay_min',YCol='EChadwick', CondCol = ['date_x','drug'],\
           Filters=Filters, cellID = 'cellID', AvgPerCell=False, modelFit=False)
renameAxes(ax,renameDict1)
ax.legend(loc='upper right')
ax.set_xlabel('Time since start of experiment (min)')
fig.suptitle('3T3aSFL - Compressions - Evolution with time')
archiveFig(fig, ax, name='3T3aSFL-Compressions-EvolWithTime_PYTHONTABLE', figSubDir = figSubDir)
fig.show()

In [ ]:
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True),\
           (GlobalTable_meca_Py['cell subtype'] == 'aSFL-A8'), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs')]
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['manipID','drug'],Parameters=['surroundingThickness','EChadwick'],\
                 Filters=Filters,AvgPerCell=True,cellID='cellID',stats=True)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL-A8 - Compressions - Differences with manip')
for i in range(len(ax)):
    xTicks = ax[i].get_xticklabels()
    for j in range(len(xTicks)):
        xTicks[j].set_fontsize(9)
archiveFig(fig, ax, name='3T3aSFL-A8-Compressions-DifferencesWithManip_PYTHONTABLE', figSubDir = figSubDir)
fig.show()

In [ ]:
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True),\
           (GlobalTable_meca_Py['cell subtype'] == 'aSFL-A8'), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs')]
fig, ax = D2Plot(GlobalTable_meca_Py, XCol='compStartTimeThisDay_min',YCol='EChadwick', CondCol = ['date_x','drug'],\
           Filters=Filters, cellID = 'cellID', AvgPerCell=False, modelFit=False)
renameAxes(ax,renameDict1)
ax.legend(loc='upper right')
ax.set_xlabel('Time since start of experiment (min)')
fig.suptitle('3T3aSFL-A8 - Compressions - Evolution with time')
archiveFig(fig, ax, name='3T3aSFL-A8-Compressions-EvolWithTime_PYTHONTABLE', figSubDir = figSubDir)
fig.show()

In [ ]:
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True),\
           (GlobalTable_meca_Py['cell subtype'] == 'aSFL-6FP'), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs')]
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['manipID','drug'],Parameters=['surroundingThickness','EChadwick'],\
                 Filters=Filters,AvgPerCell=True,cellID='cellID',stats=True)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL-6FP - Compressions - Differences with manip')
for i in range(len(ax)):
    xTicks = ax[i].get_xticklabels()
    for j in range(len(xTicks)):
        xTicks[j].set_fontsize(9)
archiveFig(fig, ax, name='3T3aSFL-6FP-Compressions-DifferencesWithManip_PYTHONTABLE', figSubDir = figSubDir)
fig.show()

In [ ]:
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True),\
           (GlobalTable_meca_Py['cell subtype'] == 'aSFL-6FP'), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs')]
fig, ax = D2Plot(GlobalTable_meca_Py, XCol='compStartTimeThisDay_min',YCol='EChadwick', CondCol = ['date_x','drug'],\
           Filters=Filters, cellID = 'cellID', AvgPerCell=False, modelFit=False)
renameAxes(ax,renameDict1)
ax.legend(loc='upper right')
ax.set_xlabel('Time since start of experiment (min)')
fig.suptitle('3T3aSFL-6FP - Compressions - Evolution with time')
archiveFig(fig, ax, name='3T3aSFL-6FP-Compressions-EvolWithTime_PYTHONTABLE', figSubDir = figSubDir)
fig.show()

#### Interactive

In [ ]:
Filters = [(GlobalTable_ctField['validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL')]

p = D1PlotInteractive(GlobalTable_ctField, CondCol='drug',Parameters=['medianThickness','fluctuAmpli'],Filters=Filters)
p.children[0][0].title.text = '3T3aSFL on patterns: Ct Field'
p.children[0][0].title.text_font_size = '16pt'
p.children[1][0].title.text = ''
show(p)

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == 1), (GlobalTable_meca['cell subtype'] == 'aSFL')]
p = D1PlotInteractive(GlobalTable_meca, CondCol='drug',Parameters=['SurroundingThickness','EChadwick'],Filters=Filters,AvgPerCell=True,cellID='CellName')
p.children[0][0].title.text = '3T3aSFL on patterns: Compressions'
p.children[0][0].title.text_font_size = '14pt'
p.children[1][0].title.text = ''
show(p)

In [ ]:
Filters = [(GlobalTable_ctField['validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL')]
p = D2PlotInteractive(GlobalTable_ctField, XCol='medianThickness',YCol='fluctuAmpli',CondCol = 'drug', Filters=Filters)
show(p)

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL')]
p = D2PlotInteractive(GlobalTable_meca, XCol='SurroundingThickness',YCol='EChadwick',CondCol = 'drug', Filters=Filters, cellID = 'CellName')
show(p)

In [ ]:
Filters = [(GlobalTable_ctField['validated'] == True)]
p = D2PlotInteractive(GlobalTable_ctField, XCol='meanFluoPeakAmplitude',YCol='medianThickness',CondCol = 'drug', Filters=Filters, cellID = 'cellID',AvgPerCell=True)
show(p)

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True)]
p = D2PlotInteractive(GlobalTable_meca, XCol='meanFluoPeakAmplitude',YCol='SurroundingThickness',CondCol = 'drug', Filters=Filters, cellID = 'CellName',AvgPerCell=True)
p.title.text = '3T3aSFL expressing linker: H(fluo)'
p.title.text_font_size = '16pt'
show(p)

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL')]
p = D2PlotInteractive(GlobalTable_meca, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = 'drug', Filters=Filters, cellID = 'CellName',AvgPerCell=True)
p.title.text = 'aSFL expressing linker: E(fluo)'
p.title.text_font_size = '16pt'
show(p)

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL')]
p = D2PlotInteractive(GlobalTable_meca, XCol='meanFluoPeakAmplitude',YCol='SurroundingThickness',CondCol = 'drug', Filters=Filters, cellID = 'CellName',AvgPerCell=True)
p.title.text = 'aSFL expressing linker: H(fluo)'
p.title.text_font_size = '16pt'
show(p)

In [ ]:
Outliers = []
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True),
           (GlobalTable_meca_Py['drug'].apply(lambda x : x in ['none', 'doxycyclin'])),
           (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'), 
           (GlobalTable_meca_Py['cell subtype'].apply(lambda x : x in ['aSFL-A8-2'])),
           (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers))]
p = D2PlotInteractive(GlobalTable_meca_Py, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = 'drug', 
                      Filters=Filters, cellID = 'cellID', AvgPerCell=True)
p.title.text = 'aSFL-6FP expressing long linker: E(fluo)'
p.title.text_font_size = '16pt'
show(p)

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL-6FP')]
p = D2PlotInteractive(GlobalTable_meca, XCol='meanFluoPeakAmplitude', YCol='SurroundingThickness', CondCol = 'drug', Filters=Filters, cellID = 'CellName',AvgPerCell=True)
p.title.text = 'aSFL-6FP expressing long linker: H(fluo)'
p.title.text_font_size = '16pt'
show(p)

#### Region Fits

In [ ]:
figSubDir = 'RegionFits'

In [ ]:
Outliers = ['21-07-08_M3_P1_C10', '21-07-08_M3_P1_C5', '21-07-08_M3_P1_C6']
Filters = [(GlobalTable_meca_Py['validatedFit_f<100pN'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), 
           (GlobalTable_meca_Py['drug'] == 'none'), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py['cell subtype'] == 'aSFL'), (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers)),
           (GlobalTable_meca_Py['bead type'].apply(lambda x : x in ['M270', 'M270_M450', 'M450_M270', 'M450']))]
co_order = makeOrder(['M270', 'M270_M450', 'M450_M270', 'M450'])
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['bead type'],\
                 Parameters=['surroundingThickness','EChadwick_f<100pN'],Filters=Filters,\
                 AvgPerCell=True, cellID='cellID', co_order=co_order, figSizeFactor = 2, orientation = 'v', useHue = False)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL asym bead pairs - all comps - f<100pN')
# archiveFig(fig, ax, name='3T3aSFL_asymBeadsAllComp_SurroundingThickness&EChadwick', figSubDir = figSubDir)
fig.show()

In [ ]:
Outliers = ['21-07-08_M3_P1_C10', '21-07-08_M3_P1_C5', '21-07-08_M3_P1_C6']
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), 
           (GlobalTable_meca_Py['drug'] == 'none'), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py['cell subtype'] == 'aSFL'), (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers)),
           (GlobalTable_meca_Py['bead type'].apply(lambda x : x in ['M270', 'M270_M450', 'M450_M270', 'M450']))]
co_order = makeOrder(['M270', 'M270_M450', 'M450_M270', 'M450'])
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['bead type'],\
                 Parameters=['surroundingThickness','EChadwick'],Filters=Filters,\
                 AvgPerCell=True, cellID='cellID', co_order=co_order, figSizeFactor = 2, orientation = 'v', useHue = False)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL asym bead pairs')
archiveFig(fig, ax, name='3T3aSFL_asymBeads_SurroundingThickness&EChadwick_allDates', figSubDir = figSubDir)
fig.show()

In [ ]:
Outliers = ['21-07-08_M3_P1_C10', '21-07-08_M3_P1_C5', '21-07-08_M3_P1_C6']
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), 
           (GlobalTable_meca_Py['date_x'] == '21-07-08'),
           (GlobalTable_meca_Py['drug'] == 'none'), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py['cell subtype'] == 'aSFL'), (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers)),
           (GlobalTable_meca_Py['bead type'].apply(lambda x : x in ['M270', 'M270_M450', 'M450_M270', 'M450']))]
co_order = makeOrder(['M270', 'M270_M450', 'M450_M270', 'M450'])
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['bead type'],\
                 Parameters=['surroundingThickness','EChadwick'],Filters=Filters,\
                 AvgPerCell=True, cellID='cellID', co_order=co_order, figSizeFactor = 2, orientation = 'v', useHue = False)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL asym bead pairs')
archiveFig(fig, ax, name='3T3aSFL_asymBeads_SurroundingThickness&EChadwick_21-07-08', figSubDir = )
fig.show()

In [ ]:
Outliers = ['21-07-08_M3_P1_C10', '21-07-08_M3_P1_C5', '21-07-08_M3_P1_C6']
Filters = [(GlobalTable_meca_Py['validatedFit_f<100pN'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), 
           (GlobalTable_meca_Py['drug'] == 'none'), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py['cell subtype'] == 'aSFL'), (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers)),
           (GlobalTable_meca_Py['bead type'].apply(lambda x : x in ['M270', 'M450']))]
fig, ax = D2Plot(GlobalTable_meca_Py, XCol='surroundingThickness',YCol='EChadwick_f<100pN', CondCol = ['bead type'],\
           Filters=Filters, cellID = 'cellID', AvgPerCell=True, modelFit=False, modelType='y=A*exp(kx)')
fig.suptitle('3T3aSFL: E(h)')
# archiveFig(fig, ax, name='aSFL_E(h)_drug&substrate', figDir = todayFigDir, figSubDir='')
fig.show()

In [ ]:
Outliers = ['21-07-08_M3_P1_C10', '21-07-08_M3_P1_C5', '21-07-08_M3_P1_C6']
Filters = [(GlobalTable_meca_Py['validatedFit_f<150pN'] == True), (GlobalTable_meca_Py['validatedThickness'] == True),
           (GlobalTable_meca_Py['date_x'].apply(lambda x : x in ['21-07-08'])), 
           (GlobalTable_meca_Py['drug'] == 'none'), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py['cell subtype'] == 'aSFL'), (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers)),
           (GlobalTable_meca_Py['bead type'].apply(lambda x : x in ['M270', 'M450_M270', 'M270_M450', 'M450']))]
fig, ax = D2Plot(GlobalTable_meca_Py, XCol='surroundingThickness',YCol='EChadwick_f<150pN', CondCol = ['bead type'],\
           Filters=Filters, cellID = 'cellID', AvgPerCell=False, modelFit=False, modelType='y=A*exp(kx)')
fig.suptitle('3T3aSFL: E(h) _f<150pN_21-07-08')
archiveFig(fig, ax, name='aSFL_beadTypes01_E(h)_21-07-08', figDir = todayFigDir, figSubDir=figSubDir)
fig.show()

In [ ]:
Outliers = ['21-07-08_M3_P1_C10', '21-07-08_M3_P1_C5', '21-07-08_M3_P1_C6']
Filters = [(GlobalTable_meca_Py['validatedFit_f<150pN'] == True), (GlobalTable_meca_Py['validatedThickness'] == True),
           (GlobalTable_meca_Py['date_x'].apply(lambda x : x in ['21-07-08'])), 
           (GlobalTable_meca_Py['drug'] == 'none'), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py['cell subtype'] == 'aSFL'), (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers)),
           (GlobalTable_meca_Py['bead type'].apply(lambda x : x in ['M270', 'M450']))]
fig, ax = D2Plot(GlobalTable_meca_Py, XCol='surroundingThickness',YCol='EChadwick_f<150pN', CondCol = ['bead type'],\
           Filters=Filters, cellID = 'cellID', AvgPerCell=False, modelFit=False, modelType='y=A*exp(kx)')
fig.suptitle('3T3aSFL: E(h) _f<150pN_21-07-08')
archiveFig(fig, ax, name='aSFL_beadTypes02_E(h)_21-07-08', figDir = todayFigDir, figSubDir=figSubDir)
fig.show()

### Clean Graphs

In [63]:
figSubDir = 'CleanPlots'

print(pairedPalette)
pairedPalette

['#1f77b4', '#aec7e8', '#ff7f0e', '#ffbb78', '#2ca02c', '#98df8a', '#d62728', '#ff9896', '#9467bd', '#c5b0d5', '#8c564b', '#c49c94', '#e377c2', '#f7b6d2', '#7f7f7f', '#c7c7c7', '#bcbd22', '#dbdb8d', '#17becf', '#9edae5']


['#1f77b4',
 '#aec7e8',
 '#ff7f0e',
 '#ffbb78',
 '#2ca02c',
 '#98df8a',
 '#d62728',
 '#ff9896',
 '#9467bd',
 '#c5b0d5',
 '#8c564b',
 '#c49c94',
 '#e377c2',
 '#f7b6d2',
 '#7f7f7f',
 '#c7c7c7',
 '#bcbd22',
 '#dbdb8d',
 '#17becf',
 '#9edae5']

#### 3T3 aSFL (A11) on and off pattern, with or without doxy -- (january 2021)

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == 1), (GlobalTable_meca['cell subtype'] == 'aSFL'), (GlobalTable_meca['substrate'] == 'BSA coated glass'), (GlobalTable_meca['SurroundingThickness'] > 0)]
co_order = makeOrder(['BSA coated glass'],['none','doxycyclin'])
fig, ax = D1Plot(GlobalTable_meca, CondCol=['substrate','drug'],\
                 Parameters=['SurroundingThickness','EChadwick'], Filters=Filters,\
                 AvgPerCell=False, cellID='CellName', co_order=co_order, figSizeFactor = 0.5)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL non-adherent: Compressions')
# archiveFig(fig, ax, name='3T3aSFLonBSA_drug_SurroundingThickness&EChadwick_allComp', figSubDir = figSubDir)
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == 1), (GlobalTable_meca['cell subtype'] == 'aSFL'), (GlobalTable_meca['SurroundingThickness'] > 0)]
co_order = makeOrder(['BSA coated glass','20um fibronectin discs'],['none','doxycyclin'])
fig, ax = D1Plot(GlobalTable_meca, CondCol=['substrate','drug'],\
                 Parameters=['SurroundingThickness','EChadwick'],Filters=Filters,\
                 AvgPerCell=True, cellID='CellName', co_order=co_order, figSizeFactor = 0.8)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL on diverse substrates: Compressions')
# archiveFig(fig, ax, name='3T3aSFL_substrate&drug_SurroundingThickness&EChadwick', figSubDir = figSubDir)
fig.show()

In [ ]:
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), (GlobalTable_meca_Py['cell subtype'] == 'aSFL')]
co_order = makeOrder(['BSA coated glass','20um fibronectin discs'],['none','doxycyclin'])
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['substrate','drug'],\
                 Parameters=['surroundingThickness','EChadwick'],Filters=Filters,\
                 AvgPerCell=True, cellID='cellID', co_order=co_order, figSizeFactor = 0.8,
                 useHue = False, orientation = 'v')
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL on diverse substrates: Compressions')
# archiveFig(fig, ax, name='3T3aSFL_substrate&drug_SurroundingThickness&EChadwick_NEWTABLE', figSubDir = figSubDir)
fig.show()


In [89]:
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), 
           (GlobalTable_meca_Py['validatedThickness'] == True), 
           (GlobalTable_meca_Py['cell subtype'] == 'aSFL'),
          (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py['bead type'] == 'M450'),
          (GlobalTable_meca_Py['ctFieldThickness'] <= 600)]
co_order = makeOrder(['none','doxycyclin'])
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['drug'],\
                 Parameters=['ctFieldThickness','EChadwick'],Filters=Filters,\
                 AvgPerCell=True, cellID='cellID', co_order=co_order, figSizeFactor = 1.0,
                 useHue = False, orientation = 'h', markerSizeFactor=1.2)
ax[0].set_ylim([0, ax[0].get_ylim()[1]])
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL on fibronectin discs: Compressions')
# archiveFig(fig, ax, name='3T3aSFL_substrate&drug_SurroundingThickness&EChadwick_NEWTABLE', figSubDir = figSubDir)
fig.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
Filters = [(GlobalTable_ctField['validated'] == True), (GlobalTable_ctField['medianThickness'] <= 1000)]
co_order = makeOrder(['20um fibronectin discs'],['none','doxycyclin'])
fig, ax = D1Plot(GlobalTable_ctField, CondCol=['substrate','drug'],Parameters=['medianThickness','fluctuAmpli'],\
                 Filters=Filters,stats=True,co_order=co_order,figSizeFactor=0.5)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL on patterns: Constant Field')
# archiveFig(fig, ax, name='3T3aSFL_drug_medianThickness', figSubDir = figSubDir)

fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL')]
fig, ax = D2Plot(GlobalTable_meca, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = ['cell subtype', 'drug'], \
                 Filters=Filters, cellID = 'CellName', AvgPerCell=True, modelFit=True)

renameAxes(ax,renameDict1)
fig.suptitle('aSFL expressing linker: E(fluo)')
yt = ax.get_yticks()
ax.set_yticklabels((yt/1000).astype(int))
ax.set_ylabel('E Chadwick (kPa)')

# archiveFig(fig, ax, name='aSFL_iMC_E(fluo)', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL')]
fig, ax = D2Plot(GlobalTable_meca, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = ['cell subtype', 'drug'], \
                 Filters=Filters, cellID = 'CellName', AvgPerCell=True, modelFit=True)
renameAxes(ax,renameDict1)
fig.suptitle('aSFL expressing linker: E(fluo)')
# archiveFig(fig, ax, name='aSFL_iMC_E(fluo)', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

In [ ]:
#Same as above without the lonely point
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL'), (GlobalTable_meca['EChadwick'] <= 80000)]
fig, ax = D2Plot(GlobalTable_meca, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = ['cell subtype', 'drug'], \
                 Filters=Filters, cellID = 'CellName', AvgPerCell=True, modelFit=True)
renameAxes(ax,renameDict1)
fig.suptitle('aSFL expressing linker: E(fluo)')
# archiveFig(fig, ax, name='aSFL_iMC_E(fluo)_woLastPoint', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL')]
fig, ax = D2Plot(GlobalTable_meca, XCol='meanFluoPeakAmplitude',YCol='SurroundingThickness',CondCol = ['cell subtype', 'drug'], Filters=Filters, cellID = 'CellName',AvgPerCell=True)
renameAxes(ax,renameDict1)
fig.suptitle('aSFL expressing linker: H(fluo)')
# archiveFig(fig, ax, name='aSFL_iMC_H(fluo)', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

In [ ]:
Filters = [(GlobalTable_ctField['validated'] == True)]
fig, ax = D2Plot(GlobalTable_ctField, XCol='meanFluoPeakAmplitude',YCol='medianThickness',CondCol = ['cell subtype', 'drug'],\
                 Filters=Filters, cellID = 'cellID', AvgPerCell=True)
renameAxes(ax,renameDict1)
fig.suptitle('aSFL expressing linker: medianH(fluo)')
# archiveFig(fig, ax, name='aSFL_iMC_medianH(fluo)', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

##### Older graphs

In [ ]:
# Filters = [(GlobalTable_meca['Validated'] == 1), (GlobalTable_meca['cell subtype'] == 'aSFL')]
# co_order = makeOrder(['BSA coated glass','20um fibronectin discs'],['none','doxycyclin'])
# fig, ax = D1Plot(GlobalTable_meca, CondCol=['substrate','drug'],\
#                  Parameters=['EChadwick'],Filters=Filters,\
#                  AvgPerCell=True, cellID='CellName', co_order=co_order,statMethod = 'Mann-Whitney', statVerbose = 2)
# renameAxes(ax,renameDict1)
# fig.suptitle('3T3aSFL on diverse substrates: Compressions')
# fig.show()

# # test value should be one of the following: t-test_ind, t-test_welch, t-test_paired, Mann-Whitney, Mann-Whitney-gt, Mann-Whitney-ls, Levene, Wilcoxon, Kruskal.

In [ ]:
# Filters = [(GlobalTable_meca['Validated'] == 1), (GlobalTable_meca['cell subtype'] == 'aSFL')]
# co_order = makeOrder(['BSA coated glass','20um fibronectin discs'],['none','doxycyclin'])
# fig, ax = D1Plot(GlobalTable_meca, CondCol=['substrate','drug'],\
#                  Parameters=['SurroundingThickness','EChadwick'],Filters=Filters,\
#                  AvgPerCell=True, cellID='CellName', co_order=co_order, stats=False)
# renameAxes(ax,renameDict1)
# fig.suptitle('3T3aSFL on diverse substrates: Compressions')
# fig.show()
# # fig.savefig(todayFigDir + '//' + 'compressionsBSAvsFibro_woStats.png')

#### 3T3 aSFL : standard line (A11) versus different clones (A8 - high exp ; 6FP - long linker) -- First round (april - june 2021)

In [ ]:
Filters = [(GlobalTable_meca_Py['validatedThickness'] == True), (GlobalTable_meca_Py['ctFieldThickness'] <= 1000)]
co_order = makeOrder(['aSFL','aSFL-A8','aSFL-6FP'],['none','doxycyclin'])
box_pairs=[('aSFL & none', 'aSFL & doxycyclin'),
 ('aSFL-A8 & none', 'aSFL-A8 & doxycyclin'),
 ('aSFL-6FP & none', 'aSFL-6FP & doxycyclin'),
 ('aSFL & none', 'aSFL-A8 & none'),
 ('aSFL & none', 'aSFL-6FP & none')]
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['cell subtype','drug'],Parameters=['ctFieldThickness','ctFieldFluctuAmpli'],\
                 Filters=Filters,AvgPerCell=True,stats=True,co_order=co_order,box_pairs=box_pairs,figSizeFactor=1)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL on patterns: H and DH from meca expe')
# archiveFig(fig, ax, name='3T3aSFL_drug_medianThickness_fromMeca_PYTHONTABLE', figSubDir = figSubDir)

fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == 1), (GlobalTable_meca['substrate'] == '20um fibronectin discs')]
co_order = makeOrder(['aSFL','aSFL-6FP'],['none','doxycyclin'])
fig, ax = D1Plot(GlobalTable_meca, CondCol=['cell subtype','drug'],Parameters=['SurroundingThickness','EChadwick'],\
                 Filters=Filters,AvgPerCell=True,cellID='CellName',co_order=co_order,stats=True)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL short vs long linker: Compressions')
# archiveFig(fig, ax, name='3T3aSFL_likerType&drug_SurroundingThickness&EChadwick', figSubDir = figSubDir)
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['substrate'] == '20um fibronectin discs')]
co_order = makeOrder(['aSFL','aSFL-A8','aSFL-6FP'],['none','doxycyclin'])
box_pairs=[('aSFL & none', 'aSFL & doxycyclin'),
 ('aSFL-A8 & none', 'aSFL-A8 & doxycyclin'),
 ('aSFL-6FP & none', 'aSFL-6FP & doxycyclin'),
 ('aSFL & none', 'aSFL-A8 & none'),
 ('aSFL & none', 'aSFL-6FP & none')]
fig, ax = D1Plot(GlobalTable_meca, CondCol=['cell subtype','drug'],Parameters=['SurroundingThickness','EChadwick'],\
                 Filters=Filters,AvgPerCell=True,cellID='CellName',co_order=co_order,box_pairs=box_pairs,stats=True)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL - All linker types: Compressions')
# archiveFig(fig, ax, name='3T3aSFL_likerType&drug_SurroundingThickness&EChadwick', figSubDir = figSubDir)
fig.show()

In [ ]:
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs')]
co_order = makeOrder(['aSFL','aSFL-A8','aSFL-6FP'],['none','doxycyclin'])
box_pairs=[('aSFL & none', 'aSFL & doxycyclin'),
 ('aSFL-A8 & none', 'aSFL-A8 & doxycyclin'),
 ('aSFL-6FP & none', 'aSFL-6FP & doxycyclin'),
 ('aSFL & none', 'aSFL-A8 & none'),
 ('aSFL & none', 'aSFL-6FP & none')]
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['cell subtype','drug'],Parameters=['ctFieldThickness','EChadwick'],\
                 Filters=Filters,AvgPerCell=True,cellID='cellID',co_order=co_order,box_pairs=box_pairs,stats=True)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL - All linker types: Compressions')
# archiveFig(fig, ax, name='3T3aSFL_likerType&drug_ctFThickness&EChadwick_PYTHONTABLE', figSubDir = figSubDir)
fig.show()

In [ ]:
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True),\
           (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'), (GlobalTable_meca_Py['drug'] == 'doxycyclin'),\
           (pd.isna(GlobalTable_meca_Py['meanFluoPeakAmplitude']) != True)]
co_order = makeOrder(['aSFL','aSFL-A8','aSFL-6FP'])
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['cell subtype'],Parameters=['meanFluoPeakAmplitude'],\
                 Filters=Filters,AvgPerCell=True,cellID='cellID',co_order=co_order,stats=True,figSizeFactor=1.25)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL different cell lines\nLinker expression quantif by fluo')
archiveFig(fig, ax, name='3T3aSFL_likerType&_fluoExp_PYTHONTABLE', figSubDir = figSubDir)
fig.show()

In [ ]:
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), ((GlobalTable_meca_Py['cell subtype'] == 'aSFL') | (GlobalTable_meca_Py['cell subtype'] == 'aSFL-A8')), (GlobalTable_meca_Py['drug'] == 'doxycyclin')]
fig, ax = D2Plot(GlobalTable_meca_Py, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = ['cell subtype'], \
                 Filters=Filters, cellID = 'cellID', AvgPerCell=True, modelFit=True)
renameAxes(ax,renameDict1)
fig.suptitle('aSFL & aSFL-A8 expressing linker: E(fluo)')
# archiveFig(fig, ax, name='aSFL&A8_iMC_E(fluo)', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

In [ ]:
#Same as above without the lonely point
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['drug'] == 'doxycyclin'), \
           ((GlobalTable_meca_Py['cell subtype'] == 'aSFL') | (GlobalTable_meca_Py['cell subtype'] == 'aSFL-A8')), \
           (GlobalTable_meca_Py['meanFluoPeakAmplitude'] <= 1200), (GlobalTable_meca_Py['EChadwick'] >= 2000)]
fig, ax = D2Plot(GlobalTable_meca_Py, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = ['cell subtype'], \
                 Filters=Filters, cellID = 'cellID', AvgPerCell=True, modelFit=True)
renameAxes(ax,renameDict1)
fig.suptitle('aSFL & aSFL-A8 expressing linker: E(fluo)')
# archiveFig(fig, ax, name='aSFL&A8_iMC_E(fluo)_fluo-1200_&_E+2000', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL-6FP')]
fig, ax = D2Plot(GlobalTable_meca, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = ['cell subtype', 'drug'], \
                 Filters=Filters, cellID = 'CellName', AvgPerCell=True, modelFit=True)
renameAxes(ax,renameDict1)
fig.suptitle('aSFL-6FP expressing long linker: E(fluo)')
# archiveFig(fig, ax, name='aSFL-6FP_iMC_E(fluo)', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

In [ ]:
#Same as above without the lonely point
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL-6FP'), (GlobalTable_meca['meanFluoPeakAmplitude'] <= 1000)]
fig, ax = D2Plot(GlobalTable_meca, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = ['cell subtype', 'drug'], \
                 Filters=Filters, cellID = 'CellName', AvgPerCell=True, modelFit=True)
renameAxes(ax,renameDict1)
fig.suptitle('aSFL-6FP expressing long linker: E(fluo)')
# archiveFig(fig, ax, name='aSFL-6FP_iMC_E(fluo)_woLastPoint', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL-6FP')]
fig, ax = D2Plot(GlobalTable_meca, XCol='meanFluoPeakAmplitude', YCol='SurroundingThickness', CondCol = ['cell subtype', 'drug'],\
                 Filters=Filters, cellID = 'CellName',AvgPerCell=True)
renameAxes(ax,renameDict1)
fig.suptitle('aSFL-6FP expressing long linker: H(fluo)')
# archiveFig(fig, ax, name='aSFL-6FP_iMC_H(fluo)', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

#### Diverse 2D plots

In [ ]:
Filters = [(GlobalTable_ctField['validated'] == True), (GlobalTable_ctField['cell subtype'] == 'aSFL')]
fig, ax = D2Plot(GlobalTable_ctField, XCol='medianThickness',YCol='fluctuAmpli',CondCol = ['cell subtype', 'drug'],\
                 Filters=Filters, modelFit=True, figSizeFactor = 0.8)
fig.suptitle('3T3aSFL - Dh = f(H)')
# archiveFig(fig, ax, name='aSFL_Dh(h)_drug', figDir = todayFigDir, figSubDir='ThicknessPlots')
fig.show()

In [ ]:
# Same as above without the 2 thickest cells
Filters = [(GlobalTable_ctField['validated'] == True), (GlobalTable_ctField['cell subtype'] == 'aSFL'),\
           (GlobalTable_ctField['medianThickness'] <= 700)]
fig, ax = D2Plot(GlobalTable_ctField, XCol='medianThickness',YCol='fluctuAmpli',CondCol = ['cell subtype', 'drug'],\
                 Filters=Filters, modelFit=True, figSizeFactor = 0.8)
fig.suptitle('3T3aSFL - Dh = f(H)')
# archiveFig(fig, ax, name='aSFL_Dh(h)_drug_wo2LastPoints', figDir = todayFigDir, figSubDir='ThicknessPlots')
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL')]
fig, ax = D2Plot(GlobalTable_meca, XCol='SurroundingThickness',YCol='EChadwick',CondCol = ['substrate','drug'],\
           Filters=Filters, cellID = 'CellName', AvgPerCell=True, modelFit=False, modelType='y=A*exp(kx)')
fig.suptitle('3T3aSFL: E(h)')
# archiveFig(fig, ax, name='aSFL_E(h)_drug&substrate', figDir = todayFigDir, figSubDir='')
fig.show()

#### Asym bead pairs -- (july 2021)

In [ ]:
rawMecaTable = getGlobalTable_meca()
# rawMecaTable.head()

In [ ]:
#  GlobalTable_meca_Py['bead type'].apply(lambda x : x in ['M270_M270', 'M270_M450', 'M450_M270', 'M450_M450'])

In [ ]:
Outliers = ['21-07-08_M3_P1_C10', '21-07-08_M3_P1_C5', '21-07-08_M3_P1_C6']
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), (GlobalTable_meca_Py['drug'] == 'none'), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'),\
           (GlobalTable_meca_Py['cell subtype'] == 'aSFL'), (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers)), \
           (GlobalTable_meca_Py['bead type'].apply(lambda x : x in ['M270', 'M270_M450', 'M450_M270', 'M450']))]
co_order = makeOrder(['M270', 'M270_M450', 'M450_M270', 'M450'])
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['bead type'],\
                 Parameters=['surroundingThickness','EChadwick'],Filters=Filters,\
                 AvgPerCell=False, cellID='cellID', co_order=co_order, figSizeFactor = 2, orientation = 'v', useHue = False)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL asym bead pairs - all comps')
# archiveFig(fig, ax, name='3T3aSFL_asymBeadsAllComp_SurroundingThickness&EChadwick', figSubDir = figSubDir)
fig.show()

In [ ]:
Outliers = ['21-07-08_M3_P1_C10','21-07-08_M3_P1_C5','21-07-08_M3_P1_C6']
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), (GlobalTable_meca_Py['drug'] == 'none'), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'), \
           (GlobalTable_meca_Py['cell subtype'] == 'aSFL'), (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers)), \
           (GlobalTable_meca_Py['bead type'].apply(lambda x : x in ['M270', 'M270_M450', 'M450_M270', 'M450']))]
co_order = makeOrder(['M270', 'M270_M450', 'M450_M270', 'M450'])
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['bead type'],\
                 Parameters=['surroundingThickness','EChadwick'],Filters=Filters,\
                 AvgPerCell=True, cellID='cellID', co_order=co_order, figSizeFactor = 2, orientation = 'v', useHue = True)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL asym bead pairs - avg per cell')
# archiveFig(fig, ax, name='3T3aSFL_asymBeadsPerCell_SurroundingThickness&EChadwick', figSubDir = figSubDir)
fig.show()

In [ ]:
Filters = [(rawMecaTable['Validated'] == 1), ((rawMecaTable['ExpType'] == 'DictyDB_M270') | (rawMecaTable['ExpType'] == 'DictyDB_M450')), (rawMecaTable['TpsComp'] == '1s')]
# Filters = [(rawMecaTable['Validated'] == 1), ((rawMecaTable['ExpType'] == 'DictyDB_M450')), (rawMecaTable['TpsComp'] == '1s')]
fig, ax = D1Plot(rawMecaTable, CondCol=['ExpType'],Parameters=['SurroundingThickness','EChadwick'],\
                 Filters=Filters,AvgPerCell=True,cellID='CellName', useHue = False)
fig.suptitle('M450 vs M270 pour compressions de 1s')
renameAxes(ax,renameDict1)
archiveFig(fig, ax, name='Dictys_beadTypes_SurroundingThickness&EChadwick', figSubDir = figSubDir)
fig.show()
# rawMecaTable[Filters[0] & Filters[1] & Filters[2]]

#### Smifh2 vs Dmso -- (Sergio's visit september 2021)

In [60]:
pd.set_option('max_columns', None)
# pd.reset_option('max_columns')
pd.set_option('max_rows', None)
# pd.reset_option('max_rows')

In [65]:
Outliers = ['21-09-02_M4_P1_C3', '21-09-02_M3_P1_C2', '21-09-02_M3_P1_C9']
Filters = [(GlobalTable_meca_Py['drug'].apply(lambda x : x in ['dmso', 'smifh2', 'dmso. doxycyclin', 'smifh2. doxycyclin'])), \
           (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'), (GlobalTable_meca_Py['cell subtype'] == 'aSFL'), \
           (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers))]
data_filtered = GlobalTable_meca_Py
for fltr in Filters:
    data_filtered = data_filtered.loc[fltr]
data_filtered

,date_x,cellName,cellID,manipID,compNum,compDuration,compStartTime,compAbsStartTime,compStartTimeThisDay,initialThickness,minThickness,maxIndent,previousThickness,surroundingThickness,validatedThickness,ctFieldThickness,ctFieldFluctuAmpli,H0Chadwick,EChadwick,R2Chadwick,EChadwick_CIWidth,hysteresis,critFit,validatedFit,comments_x,H0Chadwick_f<150pN,EChadwick_f<150pN,R2Chadwick_f<150pN,Npts_f<150pN,validatedFit_f<150pN,H0Chadwick_f<100pN,EChadwick_f<100pN,R2Chadwick_f<100pN,Npts_f<100pN,validatedFit_f<100pN,H0Chadwick_s<150Pa,EChadwick_s<150Pa,R2Chadwick_s<150Pa,Npts_s<150Pa,validatedFit_s<150Pa,date_y,manip,experimentType,drug,substrate,objective magnification,scale pixel per um,objective immersion,optical index correction,magnetic field correction,cell type,cell subtype,bead type,bead diameter,beads bright spot delta,normal field,ramp field,compression duration,with fluo images,normal field multi images,multi image Z step,loop structure,comments_y,meanFluoPeakAmplitude
2398,21-09-01,M1_P1_C10,21-09-01_M1_P1_C10,21-09-01_M1,1,1s,3.814,19481.087,3563.336,314.146708,160.940833,153.205876,295.679765,253.193684,True,286.756502,142.006751,3.560010e+02,3.471007e+03,9.687476e-01,5.432252e+02,74.220245,R2 > 0.9,True,ok,380.254270,2.292635e+03,9.745929e-01,15.0,True,NaN,NaN,NaN,10.0,False,NaN,NaN,NaN,0.0,False,21-09-01,M1,compressions,dmso,20um fibronectin discs,100X,15.8,oil,0.875,1.095,3T3,aSFL,M450,4503,0,5,3_40,1s,False,3,500,169_133,<NA>,NaN
2399,21-09-01,M1_P1_C10,21-09-01_M1_P1_C10,21-09-01_M1,2,1s,16.357,19493.630,3575.879,255.389836,163.547245,91.842591,213.358000,239.187318,True,286.756502,142.006751,2.837966e+02,7.568728e+03,9.886121e-01,7.499952e+02,3.577904,R2 > 0.9,True,ok,291.686895,6.095532e+03,9.313737e-01,13.0,True,NaN,NaN,NaN,8.0,False,NaN,NaN,NaN,0.0,False,21-09-01,M1,compressions,dmso,20um fibronectin discs,100X,15.8,oil,0.875,1.095,3T3,aSFL,M450,4503,0,5,3_40,1s,False,3,500,169_133,<NA>,NaN
2400,21-09-01,M1_P1_C10,21-09-01_M1_P1_C10,21-09-01_M1,3,1s,28.326,19505.599,3587.848,342.759625,206.695548,136.064077,293.318174,310.110660,True,286.756502,142.006751,3.892711e+02,4.467928e+03,9.809705e-01,5.721053e+02,48.936882,R2 > 0.9,True,ok,380.085490,7.616251e+03,9.367729e-01,14.0,True,NaN,NaN,NaN,8.0,False,NaN,NaN,NaN,0.0,False,21-09-01,M1,compressions,dmso,20um fibronectin discs,100X,15.8,oil,0.875,1.095,3T3,aSFL,M450,4503,0,5,3_40,1s,False,3,500,169_133,<NA>,NaN
2401,21-09-01,M1_P1_C10,21-09-01_M1_P1_C10,21-09-01_M1,4,1s,40.299,19517.572,3599.821,416.723260,258.792702,157.930559,336.898069,383.277065,True,286.756502,142.006751,4.497465e+02,5.447116e+03,7.222392e-01,3.204562e+03,50.896133,R2 > 0.9,False,R2 < 0.9,473.381538,3.652844e+03,9.810962e-01,14.0,True,NaN,NaN,NaN,9.0,False,NaN,NaN,NaN,0.0,False,21-09-01,M1,compressions,dmso,20um fibronectin discs,100X,15.8,oil,0.875,1.095,3T3,aSFL,M450,4503,0,5,3_40,1s,False,3,500,169_133,<NA>,NaN
2402,21-09-01,M1_P1_C10,21-09-01_M1_P1_C10,21-09-01_M1,5,1s,52.175,19529.448,3611.697,359.429915,216.813810,142.616105,378.120422,336.828889,True,286.756502,142.006751,4.059330e+02,4.563055e+03,9.968307e-01,2.342958e+02,50.568901,R2 > 0.9,True,ok,405.710110,4.747793e+03,9.537103e-01,16.0,True,NaN,NaN,NaN,10.0,False,NaN,NaN,NaN,0.0,False,21-09-01,M1,compressions,dmso,20um fibronectin discs,100X,15.8,oil,0.875,1.095,3T3,aSFL,M450,4503,0,5,3_40,1s,False,3,500,169_133,<NA>,NaN
2403,21-09-01,M1_P1_C10,21-09-01_M1_P1_C10,21-09-01_M1,6,1s,64.055,19541.328,3623.577,324.362738,191.994634,132.368104,306.423652,296.845144,True,286.756502,142.006751,3.608391e+02,4.749573e+03,9.815207e-01,5.923475e+02,12.728826,R2 > 0.9,True,ok,376.630873,3.427169e+03,9.735841e-01,14.0,True,NaN,NaN,NaN,9.0,False,NaN,NaN,NaN,0.0,False,21-09-01,M1,compressions,dmso,20um fibronectin discs,100X,15.8,oil,0.875,1.095,3T3,aSFL,M450,4503,0,5,3_40,1s,False,3,500,169_133,<NA>,NaN
2404,21-09-01,M1_P1_C10,21-09-01_M1_P1_C10,21-09-01_M1,7,1s,75.936,19553.209,3635.458,336.743970,245.004937,91.739033,299.159822,294.167105,True,28

In [68]:
Outliers = ['21-09-02_M4_P1_C3', '21-09-02_M3_P1_C2', '21-09-02_M3_P1_C9']
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), \
           (GlobalTable_meca_Py['drug'].apply(lambda x : x in ['dmso', 'smifh2', 'dmso. doxycyclin', 'smifh2. doxycyclin'])), \
           (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'), (GlobalTable_meca_Py['cell subtype'] == 'aSFL'), \
           (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers))]
co_order = makeOrder(['dmso', 'smifh2', 'dmso. doxycyclin', 'smifh2. doxycyclin'])
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['drug'],\
                 Parameters=['ctFieldThickness','EChadwick'], Filters=Filters,\
                 AvgPerCell=True, cellID='cellID', co_order=co_order, figSizeFactor = 1.8, orientation = 'v')
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL smifh2')
archiveFig(fig, ax, name='3T3aSFL_smifh2&doxy_Thickness&EChadwick', figSubDir = figSubDir)
fig.show()
# 'ctFieldThickness', 'surroundingThickness', 'none', 'doxycyclin', 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Long linker second clone (3T3 aSFL-6FP-2) -- (Sergio's visit september 2021)

In [ ]:
pd.set_option('max_columns', None)
pd.reset_option('max_columns')
pd.set_option('max_rows', None)
pd.reset_option('max_rows')

In [69]:
# Outliers = []
# Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), \
#            (GlobalTable_meca_Py['drug'].apply(lambda x : x in ['none', 'doxycyclin'])), \
#            (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'), (GlobalTable_meca_Py['cell subtype'].apply(lambda x : x in ['aSFL', 'aSFL-6FP', 'aSFL-6FP-2'])), \
#            (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers))]
# co_order = makeOrder(['aSFL', 'aSFL-6FP', 'aSFL-6FP-2'], ['none', 'doxycyclin'])
# box_pairs=[('aSFL & none', 'aSFL & doxycyclin'),
#  ('aSFL-6FP & none', 'aSFL-6FP & doxycyclin'),
#  ('aSFL-6FP-2 & none', 'aSFL-6FP-2 & doxycyclin'),
#  ('aSFL & none', 'aSFL-6FP & none'),
#  ('aSFL & none', 'aSFL-6FP-2 & none'),
#  ('aSFL-6FP & none', 'aSFL-6FP-2 & none')]
# fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['cell subtype', 'drug'],\
#                  Parameters=['ctFieldThickness','EChadwick'], Filters=Filters,\
#                  AvgPerCell=True, cellID='cellID', co_order=co_order, box_pairs = box_pairs, figSizeFactor = 1, orientation = 'v')
# ax[0].set_ylim([0, ax[0].get_ylim()[1]])
# renameAxes(ax,renameDict1)
# fig.suptitle('3T3aSFL 6FP-2')
# archiveFig(fig, ax, name='3T3aSFL-6FP-2_Thickness&EChadwick', figSubDir = figSubDir)
# fig.show()
# # 'ctFieldThickness', 'surroundingThickness', 'none', 'doxycyclin', 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [73]:
Outliers = []
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), \
           (GlobalTable_meca_Py['drug'].apply(lambda x : x in ['none', 'doxycyclin'])), \
           (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'), (GlobalTable_meca_Py['cell subtype'].apply(lambda x : x in ['aSFL', 'aSFL-6FP-2'])), \
           (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers))]
co_order = makeOrder(['aSFL', 'aSFL-6FP-2'], ['none', 'doxycyclin'])
box_pairs=[('aSFL & none', 'aSFL & doxycyclin'),
 ('aSFL-6FP-2 & none', 'aSFL-6FP-2 & doxycyclin'),
 ('aSFL & none', 'aSFL-6FP-2 & none')]
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['cell subtype', 'drug'],\
                 Parameters=['ctFieldThickness','EChadwick'], Filters=Filters,\
                 AvgPerCell=True, cellID='cellID', co_order=co_order, box_pairs = box_pairs, figSizeFactor = 1, orientation = 'v')
ax[0].set_ylim([0, ax[0].get_ylim()[1]])
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL 6FP-2')
archiveFig(fig, ax, name='3T3aSFL-6FP-2_Thickness&EChadwick', figSubDir = figSubDir)
fig.show()
# 'ctFieldThickness', 'surroundingThickness', 'none', 'doxycyclin', 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [70]:
Outliers = []
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), \
           (GlobalTable_meca_Py['drug'].apply(lambda x : x in ['none', 'doxycyclin'])), \
           (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'), (GlobalTable_meca_Py['cell subtype'].apply(lambda x : x in ['aSFL-6FP-2'])), \
           (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers))]
fig, ax = D2Plot(GlobalTable_meca_Py, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = ['cell subtype', 'drug'], \
                 Filters=Filters, cellID = 'cellID', AvgPerCell=True, modelFit=True)

renameAxes(ax,renameDict1)
fig.suptitle('aSFL-6FP-2 expressing linker: E(fluo)')
yt = ax.get_yticks()
ax.set_yticklabels((yt/1000).astype(int))
ax.set_ylabel('E Chadwick (kPa)')

# archiveFig(fig, ax, name='aSFL_iMC_E(fluo)', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

Fitting condition: aSFL-6FP-2 & doxycyclin with model y=ax+b
Y = -8.4787 X + 2.0066e+04
p-value on the 'a' coefficient: 5.2955e-01


Fitting condition: aSFL-6FP-2 & none with model y=ax+b
Y = -65.665 X + 2.1336e+04
p-value on the 'a' coefficient: 5.7114e-01




C:\Users\JosephVermeil\anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: FixedFormatter should only be used together with FixedLocator
  if sys.path[0] == '':


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [71]:
Outliers = []
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), \
           (GlobalTable_meca_Py['drug'].apply(lambda x : x in ['none', 'doxycyclin'])), \
           (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'), 
           (GlobalTable_meca_Py['cell subtype'].apply(lambda x : x in ['aSFL-6FP'])), \
           (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers))]
fig, ax = D2Plot(GlobalTable_meca_Py, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = ['cell subtype', 'drug'], \
                 Filters=Filters, cellID = 'cellID', AvgPerCell=True, modelFit=True)

renameAxes(ax,renameDict1)
fig.suptitle('aSFL-6FP expressing linker: E(fluo)')
yt = ax.get_yticks()
ax.set_yticklabels((yt/1000).astype(int))
ax.set_ylabel('E Chadwick (kPa)')

# archiveFig(fig, ax, name='aSFL_iMC_E(fluo)', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

Fitting condition: aSFL-6FP & doxycyclin with model y=ax+b
Y = 7.2594 X + 1.6826e+04
p-value on the 'a' coefficient: 4.0464e-01




C:\Users\JosephVermeil\anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: FixedFormatter should only be used together with FixedLocator
  del sys.path[0]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [72]:
Outliers = []
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), \
           (GlobalTable_meca_Py['drug'].apply(lambda x : x in ['none', 'doxycyclin'])), \
           (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'), 
           (GlobalTable_meca_Py['cell subtype'].apply(lambda x : x in ['aSFL'])), \
           (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers))]
fig, ax = D2Plot(GlobalTable_meca_Py, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = ['cell subtype', 'drug'], \
                 Filters=Filters, cellID = 'cellID', AvgPerCell=True, modelFit=True)

renameAxes(ax,renameDict1)
fig.suptitle('aSFL expressing linker: E(fluo)')
yt = ax.get_yticks()
ax.set_yticklabels((yt/1000).astype(int))
ax.set_ylabel('E Chadwick (kPa)')

# archiveFig(fig, ax, name='aSFL_iMC_E(fluo)', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

Fitting condition: aSFL & doxycyclin with model y=ax+b
Y = 32.283 X + 6320.5
p-value on the 'a' coefficient: 1.0863e-03


Fitting condition: aSFL & none with model y=ax+b
Y = 6.5912 X + 7465.0
p-value on the 'a' coefficient: 9.0693e-01




C:\Users\JosephVermeil\anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: FixedFormatter should only be used together with FixedLocator
  del sys.path[0]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### High expresser experiment, second version of the clone (aSFL A8-2) -- (Sergio's visit september 2021)

In [74]:
# Outliers = []
# Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True),
#            (GlobalTable_meca_Py['drug'].apply(lambda x : x in ['none', 'doxycyclin'])),
#            (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'), 
#            (GlobalTable_meca_Py['cell subtype'].apply(lambda x : x in ['aSFL', 'aSFL-A8', 'aSFL-A8-2'])),
#            (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers))]
# co_order = makeOrder(['aSFL', 'aSFL-A8', 'aSFL-A8-2'], ['none', 'doxycyclin'])
# box_pairs=[('aSFL & none', 'aSFL & doxycyclin'),
#  ('aSFL-A8 & none', 'aSFL-A8 & doxycyclin'),
#  ('aSFL-A8-2 & none', 'aSFL-A8-2 & doxycyclin'),
#  ('aSFL & none', 'aSFL-A8 & none'),
#  ('aSFL & none', 'aSFL-A8-2 & none'),
#  ('aSFL-A8 & none', 'aSFL-A8-2 & none')]
# fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['cell subtype', 'drug'],\
#                  Parameters=['surroundingThickness','EChadwick'], Filters=Filters,\
#                  AvgPerCell=True, cellID='cellID', co_order=co_order, box_pairs = box_pairs, 
#                  figSizeFactor = 1, orientation = 'v', useHue = True)
# ax[0].set_ylim([0, ax[0].get_ylim()[1]])
# renameAxes(ax,renameDict1)
# fig.suptitle('3T3aSFL A8-2')
# # archiveFig(fig, ax, name='3T3aSFL-A8-2_Thickness&EChadwick', figSubDir = figSubDir)
# fig.show()
# # 'ctFieldThickness', 'surroundingThickness', 'none', 'doxycyclin', 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [76]:
Outliers = []
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True),
           (GlobalTable_meca_Py['drug'].apply(lambda x : x in ['none', 'doxycyclin'])),
           (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'), 
           (GlobalTable_meca_Py['cell subtype'].apply(lambda x : x in ['aSFL', 'aSFL-A8-2'])),
           (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers))]
co_order = makeOrder(['aSFL', 'aSFL-A8-2'], ['none', 'doxycyclin'])
box_pairs=[('aSFL & none', 'aSFL & doxycyclin'),
 ('aSFL-A8-2 & none', 'aSFL-A8-2 & doxycyclin'),
 ('aSFL & none', 'aSFL-A8-2 & none')]
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['cell subtype', 'drug'],\
                 Parameters=['surroundingThickness','EChadwick'], Filters=Filters,\
                 AvgPerCell=True, cellID='cellID', co_order=co_order, box_pairs = box_pairs, 
                 figSizeFactor = 1, orientation = 'v', useHue = False)
ax[0].set_ylim([0, ax[0].get_ylim()[1]])
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL A8-2')
archiveFig(fig, ax, name='3T3aSFL-A8-2_Thickness&EChadwick', figSubDir = figSubDir)
fig.show()
# 'ctFieldThickness', 'surroundingThickness', 'none', 'doxycyclin', 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [78]:
Outliers = []
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), \
           (GlobalTable_meca_Py['drug'].apply(lambda x : x in ['none', 'doxycyclin'])), \
           (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'), 
           (GlobalTable_meca_Py['cell subtype'].apply(lambda x : x in ['aSFL-A8-2'])), \
           (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers))]
fig, ax = D2Plot(GlobalTable_meca_Py, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = ['cell subtype', 'drug'], \
                 Filters=Filters, cellID = 'cellID', AvgPerCell=True, modelFit=True)

renameAxes(ax,renameDict1)
fig.suptitle('aSFL-A8-2 expressing linker: E(fluo)')
yt = ax.get_yticks()
ax.set_yticklabels((yt/1000).astype(int))
ax.set_ylabel('E Chadwick (kPa)')

archiveFig(fig, ax, name='aSFL-A8-2_iMC_E(fluo)', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

Fitting condition: aSFL-A8-2 & doxycyclin with model y=ax+b
Y = -17.41 X + 3.0852e+04
p-value on the 'a' coefficient: 1.7150e-01


Fitting condition: aSFL-A8-2 & none with model y=ax+b
Y = -94.068 X + 1.6704e+04
p-value on the 'a' coefficient: 1.9090e-02




C:\Users\JosephVermeil\anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: FixedFormatter should only be used together with FixedLocator
  del sys.path[0]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [79]:
Outliers = []
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), \
           (GlobalTable_meca_Py['drug'].apply(lambda x : x in ['none', 'doxycyclin'])), \
           (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'), 
           (GlobalTable_meca_Py['cell subtype'].apply(lambda x : x in ['aSFL'])), \
           (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers))]
fig, ax = D2Plot(GlobalTable_meca_Py, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = ['cell subtype', 'drug'], \
                 Filters=Filters, cellID = 'cellID', AvgPerCell=True, modelFit=True)

renameAxes(ax,renameDict1)
fig.suptitle('aSFL expressing linker: E(fluo)')
yt = ax.get_yticks()
ax.set_yticklabels((yt/1000).astype(int))
ax.set_ylabel('E Chadwick (kPa)')

# archiveFig(fig, ax, name='aSFL_iMC_E(fluo)', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

Fitting condition: aSFL & doxycyclin with model y=ax+b
Y = 32.283 X + 6320.5
p-value on the 'a' coefficient: 1.0863e-03


Fitting condition: aSFL & none with model y=ax+b
Y = 6.5912 X + 7465.0
p-value on the 'a' coefficient: 9.0693e-01




C:\Users\JosephVermeil\anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: FixedFormatter should only be used together with FixedLocator
  del sys.path[0]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [80]:
Outliers = []
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), \
           (GlobalTable_meca_Py['drug'].apply(lambda x : x in ['none', 'doxycyclin'])), \
           (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'), 
           (GlobalTable_meca_Py['cell subtype'].apply(lambda x : x in ['aSFL', 'aSFL-A8-2'])), \
           (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers))]
fig, ax = D2Plot(GlobalTable_meca_Py, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = ['drug'], \
                 Filters=Filters, cellID = 'cellID', AvgPerCell=True, modelFit=True)

renameAxes(ax,renameDict1)
fig.suptitle('{aSFL & aSFL-A8-2 mixed} expressing linker: E(fluo)')
yt = ax.get_yticks()
ax.set_yticklabels((yt/1000).astype(int))
ax.set_ylabel('E Chadwick (kPa)')

# archiveFig(fig, ax, name='aSFL_A11+A8-2_iMC_E(fluo)', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

Fitting condition: doxycyclin with model y=ax+b
Y = 10.675 X + 1.5482e+04
p-value on the 'a' coefficient: 1.8469e-01


Fitting condition: none with model y=ax+b
Y = -17.334 X + 8472.0
p-value on the 'a' coefficient: 6.0038e-01




C:\Users\JosephVermeil\anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: FixedFormatter should only be used together with FixedLocator
  del sys.path[0]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [81]:
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True),\
           (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'), 
           (GlobalTable_meca_Py['drug'].apply(lambda x : x in ['doxycyclin'])), \
           (pd.isna(GlobalTable_meca_Py['meanFluoPeakAmplitude']) != True),
          (GlobalTable_meca_Py['cell subtype'].apply(lambda x : x in ['aSFL','aSFL-6FP','aSFL-A8','aSFL-6FP-2','aSFL-A8-2'])),]
co_order = makeOrder(['aSFL','aSFL-A8','aSFL-6FP','aSFL-A8-2','aSFL-6FP-2'])
box_pairs=[('aSFL', 'aSFL-A8'),
 ('aSFL', 'aSFL-6FP'),
 ('aSFL', 'aSFL-A8-2'),
 ('aSFL', 'aSFL-6FP-2')]
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['cell subtype'],Parameters=['meanFluoPeakAmplitude'],\
                 Filters=Filters,AvgPerCell=True,cellID='cellID',co_order=co_order,box_pairs=box_pairs,stats=True,figSizeFactor=1.75)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL - different cell lines expressing the linker\nLinker expression quantification by fluo')
archiveFig(fig, ax, name='3T3aSFL_likerType&_fluoExp_NewVersion', figSubDir = figSubDir)
fig.show()

C:\Users\JosephVermeil\anaconda3\lib\site-packages\ipykernel_launcher.py:43: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [82]:
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True),\
           (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'), 
           (GlobalTable_meca_Py['drug'].apply(lambda x : x in ['none', 'doxycyclin'])), \
           (pd.isna(GlobalTable_meca_Py['meanFluoPeakAmplitude']) != True),
          (GlobalTable_meca_Py['cell subtype'].apply(lambda x : x in ['aSFL','aSFL-6FP','aSFL-A8','aSFL-6FP-2','aSFL-A8-2'])),]
co_order = makeOrder(['aSFL','aSFL-A8','aSFL-6FP','aSFL-A8-2','aSFL-6FP-2'], ['none', 'doxycyclin'])
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['cell subtype', 'drug'],Parameters=['meanFluoPeakAmplitude'],\
                 Filters=Filters,AvgPerCell=True,cellID='cellID',co_order=co_order,stats=False,figSizeFactor=1.05)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL different cell lines\nLinker expression quantif by fluo')
# archiveFig(fig, ax, name='3T3aSFL_likerType&_fluoExp_NewVersion_2', figSubDir = figSubDir)
fig.show()

C:\Users\JosephVermeil\anaconda3\lib\site-packages\seaborn\categorical.py:1296: UserWarning: 66.7% of the points cannot be placed; you may want to decrease the size of the markers or use stripplot.
  warnings.warn(msg, UserWarning)
C:\Users\JosephVermeil\anaconda3\lib\site-packages\seaborn\categorical.py:1296: UserWarning: 15.8% of the points cannot be placed; you may want to decrease the size of the markers or use stripplot.
  warnings.warn(msg, UserWarning)
C:\Users\JosephVermeil\anaconda3\lib\site-packages\seaborn\categorical.py:1296: UserWarning: 14.3% of the points cannot be placed; you may want to decrease the size of the markers or use stripplot.
  warnings.warn(msg, UserWarning)
C:\Users\JosephVermeil\anaconda3\lib\site-packages\seaborn\categorical.py:1296: UserWarning: 19.0% of the points cannot be placed; you may want to decrease the size of the markers or use stripplot.
  warnings.warn(msg, UserWarning)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [86]:
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True),\
           (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'), 
           (GlobalTable_meca_Py['drug'].apply(lambda x : x in ['none', 'doxycyclin'])), \
           (pd.isna(GlobalTable_meca_Py['meanFluoPeakAmplitude']) != True),
          (GlobalTable_meca_Py['cell subtype'].apply(lambda x : x in ['aSFL','aSFL-6FP-2','aSFL-A8-2'])),]
co_order = makeOrder(['aSFL','aSFL-A8-2','aSFL-6FP-2'], ['none', 'doxycyclin'])
box_pairs=[('aSFL & doxycyclin', 'aSFL-A8-2 & doxycyclin'),
 ('aSFL & doxycyclin', 'aSFL-6FP-2 & doxycyclin')]
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['cell subtype', 'drug'],Parameters=['meanFluoPeakAmplitude'],\
                 Filters=Filters,AvgPerCell=True,cellID='cellID',co_order=co_order,box_pairs=box_pairs,stats=True,
                 figSizeFactor=0.95)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL - different cell lines\nLinker expression quantification by fluo')
archiveFig(fig, ax, name='3T3aSFL_likerType&_fluoExp_NewVersion_2', figSubDir = figSubDir)
fig.show()

C:\Users\JosephVermeil\anaconda3\lib\site-packages\seaborn\categorical.py:1296: UserWarning: 41.7% of the points cannot be placed; you may want to decrease the size of the markers or use stripplot.
  warnings.warn(msg, UserWarning)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Utility scripts

### Create a test table to test different fits

In [ ]:
# Script !




testDir = os.path.join(dataDir, 'TestDataSet')
testFileName = 'testFitCompression.txt'
testFilePath = os.path.join(testDir, testFileName)

list_mecaFiles = [f for f in os.listdir(timeSeriesDataDir) \
                  if (os.path.isfile(os.path.join(timeSeriesDataDir, f)) and f.endswith(".csv") \
                  and ('R40' in f))] # Change to allow different formats in the future
expDf = getExperimentalConditions()
tableDictTest = {}

for f in list_mecaFiles:
    print(f)
    tS_DataFilePath = os.path.join(timeSeriesDataDir, f)
    tsDF = pd.read_csv(tS_DataFilePath, ';')
    
    split_f = f.split('_')
    tsDF.dx, tsDF.dy, tsDF.dz, tsDF.D2, tsDF.D3 = tsDF.dx*1000, tsDF.dy*1000, tsDF.dz*1000, tsDF.D2*1000, tsDF.D3*1000
    thisManipID = split_f[0] + '_' + split_f[1]
    expDf['manipID'] = expDf['date'] + '_' + expDf['manip']
    thisExpDf = expDf.loc[expDf['manipID'] == thisManipID]
    DIAMETER = thisExpDf.at[thisExpDf.index.values[0], 'bead diameter']
    thisCellID = split_f[0] + '_' + split_f[1] + '_' + split_f[2] + '_' + split_f[3]
    Ncomp = max(tsDF['idxCompression'])
    
    for i in range(1, Ncomp+1): #Ncomp+1):
        thisCompHiD = thisCellID + '__' + str(i) + '__h'
        thisCompFiD = thisCellID + '__' + str(i) + '__f'
        print(thisCompHiD)
        
        thisCompDf = tsDF.loc[tsDF['idxCompression'] == i, :]
        iStart = (findFirst(tsDF['idxCompression'], i))
        iStop = iStart + thisCompDf.shape[0]
        
        # Delimit the start of the increase of B (typically the moment when the field decrease from 5 to 3)
        # and the end of its decrease (typically when it goes back from 3 to 5)
        
        listB = thisCompDf.B.values
        offsetStart, offsetStop = 0, 0
        minB, maxB = min(listB), max(listB)
        thresholdB = (maxB-minB)/50
        k = 0
        
        while (listB[k] > minB+thresholdB) or (listB[-1-k] > minB+thresholdB):
            offsetStart += int(listB[k] > minB+thresholdB)
            k += 1
        
        jStart = offsetStart
        jMax = np.argmax(thisCompDf.B)
        
        hCompr = (thisCompDf.D3.values[jStart:jMax+1] - DIAMETER)
        fCompr = (thisCompDf.F.values[jStart:jMax+1])
        
        tableDictTest[thisCompHiD] = hCompr
        tableDictTest[thisCompFiD] = fCompr
        
saveFile = open(testFilePath, 'w')
for k in tableDictTest.keys():
    saveFile.write(k)
    for i in range(len(tableDictTest[k])):
        saveFile.write(';')
        saveFile.write(str(tableDictTest[k][i]))
    saveFile.write('\n')
saveFile.close()

### Create a test table to try statistical tests

In [ ]:
# Script !

# Create a test table to try statistical tests

testDir = os.path.join(dataDir, 'TestDataSet')
GlobalTable_meca = getGlobalTable_meca()
table_ExpConditions = getExperimentalConditions()
table_fluo = getFluoData()
GlobalTable_meca = pd.merge(GlobalTable_meca, table_ExpConditions, how="inner", on='manipID',
#     left_on=None,right_on=None,left_index=False,right_index=False,sort=True,
#     suffixes=("_x", "_y"),copy=True,indicator=False,validate=None,
)
GlobalTable_meca = pd.merge(GlobalTable_meca, table_fluo, how="left", left_on='CellName', right_on='cellID'
#     left_on=None,right_on=None,left_index=False,right_index=False,sort=True,
#     suffixes=("_x", "_y"),copy=True,indicator=False,validate=None,
)
print('Merged table has ' + str(GlobalTable_meca.shape[0]) + ' lines and ' + str(GlobalTable_meca.shape[1]) + ' columns.')

In [ ]:
# Table 1

testFileName = 'testStats01_allComp.csv'
testFilePath = os.path.join(testDir, testFileName)

Filters = [(GlobalTable_meca['Validated'] == 1), (GlobalTable_meca['cell subtype'] == 'aSFL')]
co_order = makeOrder([['BSA coated glass','20um fibronectin discs'],['none','doxycyclin']])
data = GlobalTable_meca
CondCol=['substrate','drug']
Parameters=['SurroundingThickness','EChadwick']
AvgPerCell=False
cellID='CellName'

data_filtered = data
for fltr in Filters:
    data_filtered = data_filtered.loc[fltr]

NCond = len(CondCol)    
if NCond == 1:
    CondCol = CondCol[0]
elif NCond > 1:
    newColName = ''
    for i in range(NCond):
        newColName += CondCol[i]
        newColName += ' & '
    newColName = newColName[:-3]
    data_filtered[newColName] = ''
    for i in range(NCond):
        data_filtered[newColName] += data_filtered[CondCol[i]].astype(str)
        data_filtered[newColName] = data_filtered[newColName].apply(lambda x : x + ' & ')
    data_filtered[newColName] = data_filtered[newColName].apply(lambda x : x[:-3])
    CondCol = newColName
    
if AvgPerCell:
    group = data_filtered.groupby(cellID)
    dictAggMean = getDictAggMean(data_filtered)
    data_filtered = group.agg(dictAggMean)

data_filtered.sort_values(CondCol, axis=0, ascending=True, inplace=True)

df_output = data_filtered[[cellID, 'CompNum', newColName] + Parameters]

# df_output.to_csv(testFilePath)

df_output

In [ ]:
# Table 2

testFileName = 'testStats02_avgPerCell.csv'
testFilePath = os.path.join(testDir, testFileName)

Filters = [(GlobalTable_meca['Validated'] == 1), (GlobalTable_meca['cell subtype'] == 'aSFL')]
co_order = makeOrder([['BSA coated glass','20um fibronectin discs'],['none','doxycyclin']])
data = GlobalTable_meca
CondCol=['substrate','drug']
Parameters=['SurroundingThickness','EChadwick']
AvgPerCell=True
cellID='CellName'

data_filtered = data
for fltr in Filters:
    data_filtered = data_filtered.loc[fltr]

NCond = len(CondCol)    
if NCond == 1:
    CondCol = CondCol[0]
elif NCond > 1:
    newColName = ''
    for i in range(NCond):
        newColName += CondCol[i]
        newColName += ' & '
    newColName = newColName[:-3]
    data_filtered[newColName] = ''
    for i in range(NCond):
        data_filtered[newColName] += data_filtered[CondCol[i]].astype(str)
        data_filtered[newColName] = data_filtered[newColName].apply(lambda x : x + ' & ')
    data_filtered[newColName] = data_filtered[newColName].apply(lambda x : x[:-3])
    CondCol = newColName
    
if AvgPerCell:
    group = data_filtered.groupby(cellID)
    dictAggMean = getDictAggMean(data_filtered)
    data_filtered = group.agg(dictAggMean)

data_filtered.sort_values(CondCol, axis=0, ascending=True, inplace=True)

df_output = data_filtered[[cellID, 'CompNum', newColName] + Parameters]

# df_output.to_csv(testFilePath)

df_output

In [ ]:
# Table 3
# Fake data

testDir = os.path.join(dataDir, 'TestDataSet')

testFileName = 'testStats03_FakeData.csv'
testFilePath = os.path.join(testDir, testFileName)

Npop = 10
Npoints = 30
minAvg = 100
maxAvg = 600
step = (maxAvg - minAvg)/(Npop-1)
std = 250
dictFakeData = {}
np.random.seed(11)

for i in [1, 2, 3, 10]:
    dictFakeData['Distribution_' + str(i)] = np.random.normal(loc=minAvg + step*(i-1), scale=std, size=Npoints)

dfFakeData = pd.DataFrame(dictFakeData)

# dfFakeData.to_csv(testFilePath)

dfFakeData

In [ ]:
# Table 4
# Fake data 2

testDir = os.path.join(dataDir, 'TestDataSet')

testFileName = 'testStats04_FakeDataLarge.csv'
testFilePath = os.path.join(testDir, testFileName)

Npop = 10
Npoints = 300
minAvg = 100
maxAvg = 600
step = (maxAvg - minAvg)/(Npop-1)
std = 250
dictFakeData = {}
np.random.seed(11)

for i in [1, 2, 3, 10]:
    dictFakeData['Distribution_' + str(i)] = np.random.normal(loc=minAvg + step*(i-1), scale=std, size=Npoints)

dfFakeData = pd.DataFrame(dictFakeData)

# dfFakeData.to_csv(testFilePath)

dfFakeData

### Comparison of stat tests

#### With the fake data

In [ ]:
testDir = os.path.join(dataDir, 'TestDataSet')

# Table 3
# Fake data
testFileName = 'testStats03_FakeData.csv'
testFilePath = os.path.join(testDir, testFileName)

dfFakeData = pd.read_csv(testFilePath)
dfFakeData = dfFakeData.drop(columns=['Unnamed: 0'])

Ncol = len(dfFakeData.columns)

refCol = dfFakeData[dfFakeData.columns[0]]
boxPlotMatrix = []

for i in range(0,Ncol):
    boxPlotMatrix.append(dfFakeData[dfFakeData.columns[i]].values)
    if i > 0:
        print('Comparison between distribution 1 and ' + str(i+1))
        tTest = st.ttest_ind(refCol.values, dfFakeData[dfFakeData.columns[i]].values)
        print('tTest : ' + str(tTest.pvalue))
        wilcox = st.wilcoxon(refCol.values, dfFakeData[dfFakeData.columns[i]].values)
        print('wilcox : ' + str(wilcox.pvalue))
        mannwhitneyu = st.mannwhitneyu(refCol.values, dfFakeData[dfFakeData.columns[i]].values)
        print('mannwhitneyu : ' + str(mannwhitneyu.pvalue))
        print('')

fig, ax = plt.subplots(1,1)
ax.boxplot(boxPlotMatrix,labels=dfFakeData.columns.values)
ax.tick_params(axis='x', labelrotation = 15, labelsize = 7)
fig.show()

In [ ]:
testDir = os.path.join(dataDir, 'TestDataSet')

# Table 4
# Fake data Large
testFileName = 'testStats04_FakeDataLarge.csv'
testFilePath = os.path.join(testDir, testFileName)

dfFakeData = pd.read_csv(testFilePath)
dfFakeData = dfFakeData.drop(columns=['Unnamed: 0'])

Ncol = len(dfFakeData.columns)

refCol = dfFakeData[dfFakeData.columns[0]]
boxPlotMatrix = []

for i in range(0,Ncol):
    boxPlotMatrix.append(dfFakeData[dfFakeData.columns[i]].values)
    if i > 0:
        print('Comparison between distribution 1 and ' + str(i+1))
        tTest = st.ttest_ind(refCol.values, dfFakeData[dfFakeData.columns[i]].values)
        print('tTest : ' + str(tTest.pvalue))
        wilcox = st.wilcoxon(refCol.values, dfFakeData[dfFakeData.columns[i]].values)
        print('wilcox : ' + str(wilcox.pvalue))
        mannwhitneyu = st.mannwhitneyu(refCol.values, dfFakeData[dfFakeData.columns[i]].values)
        print('mannwhitneyu : ' + str(mannwhitneyu.pvalue))
        print('')

fig, ax = plt.subplots(1,1)
ax.boxplot(boxPlotMatrix,labels=dfFakeData.columns.values)
ax.tick_params(axis='x', labelrotation = 15, labelsize = 7)
fig.show()

In [ ]:
# Summary of results: numpy.random.seed = 11
    
dictResultsFakeData = {}
dictResultsFakeData['language'] = ['python'   , 'python', 'python'      , 'R'     , 'R'          , 'Matlab' , 'Matlab' ]
dictResultsFakeData['test']     = ['ttest_ind', 'wilcox', 'mannwhitneyu', 't.test', 'wilcox.test', 'ttest2' , 'ranksum']
dictResultsFakeData['1 vs 2']   = [0.2700     , 0.2711  , 0.1353        , 0.2701  , 0.2729       , 0.2700   , 0.2707   ]
dictResultsFakeData['1 vs 3']   = [0.0714     , 0.0822  , 0.0452        , 0.0715  , 0.0906       , 0.0714   , 0.0905   ]
dictResultsFakeData['1 vs 10']  = [1.33e-11   , 4.28e-06, 2.98e-09      , 1.44e-11, 5.799e-11    , 1.33e-11 , 5.96e-09 ]
dfResultFakeData = pd.DataFrame(dictResultsFakeData)
dfResultFakeData

In [ ]:
# Summary of results: numpy.random.seed = 11

dictResultsFakeDataLarge = {}
dictResultsFakeDataLarge['language'] = ['python'   , 'python', 'python'      , 'R'     , 'R'          , 'Matlab' , 'Matlab' ]
dictResultsFakeDataLarge['test']     = ['ttest_ind', 'wilcox', 'mannwhitneyu', 't.test', 'wilcox.test', 'ttest2' , 'ranksum']
dictResultsFakeDataLarge['1 vs 2']   = [0.0082     , 0.0049  , 0.0038        , 0.0082  , 0.0077       , 0.0082   , 0.0077   ]
dictResultsFakeDataLarge['1 vs 3']   = [1.26e-06   , 9.29e-06, 1.37e-06      , 1.27e-06, 2.75e-06     , 1.26e-06 , 2.74e-06 ]
dictResultsFakeDataLarge['1 vs 10']  = [1.74e-98   , 5.64e-48, 6.39e-74      , 2.2e-16 , 2.2e-16      , 1.74e-98 , 1.27e-73 ]
dictResultsFakeDataLarge = pd.DataFrame(dictResultsFakeDataLarge)
dictResultsFakeDataLarge

#### With real data

In [ ]:
# testDir = os.path.join(dataDir, 'TestDataSet')

# Table 1
# Avg Per Cell
testFileName = 'testStats01_AvgPerCell.csv'
testFilePath = os.path.join(testDir, testFileName)

dfAvgPerCell = pd.read_csv(testFilePath)
# dfAvgPerCell = dfAvgPerCell.drop(columns=['Unnamed: 0'])
dfAvgPerCell
categories = list(dfAvgPerCell['substrate & drug'].unique())
Ncat = len(categories)
for i in range(Ncat):
    for j in range(i,Ncat):
        if j != i:
            x = dfAvgPerCell.loc[dfAvgPerCell['substrate & drug'] == categories[i], 'EChadwick'].values
            y = dfAvgPerCell.loc[dfAvgPerCell['substrate & drug'] == categories[j], 'EChadwick'].values
            print('Comparison between ' + categories[i] + ' and ' + categories[j])
            tTest = st.ttest_ind(x, y)
            print('tTest : ' + str(tTest.pvalue))
            mannwhitneyu = st.mannwhitneyu(x, y)
            print('mannwhitneyu : ' + str(mannwhitneyu.pvalue))
            print('')

# refCol = dfFakeData[dfFakeData.columns[0]]
# boxPlotMatrix = []

# for i in range(0,Ncol):
#     boxPlotMatrix.append(dfFakeData[dfFakeData.columns[i]].values)
#     if i > 0:
#         print('Comparison between distribution 1 and ' + str(i+1))
#         tTest = st.ttest_ind(refCol.values, dfFakeData[dfFakeData.columns[i]].values)
#         print('tTest : ' + str(tTest.pvalue))
#         wilcox = st.wilcoxon(refCol.values, dfFakeData[dfFakeData.columns[i]].values)
#         print('wilcox : ' + str(wilcox.pvalue))
#         mannwhitneyu = st.mannwhitneyu(refCol.values, dfFakeData[dfFakeData.columns[i]].values)
#         print('mannwhitneyu : ' + str(mannwhitneyu.pvalue))
#         print('')

# fig, ax = plt.subplots(1,1)
# ax.boxplot(boxPlotMatrix,labels=dfFakeData.columns.values)
# ax.tick_params(axis='x', labelrotation = 15, labelsize = 7)
# fig.show()

In [ ]:
dictResultsAvgPerCell = {}
order = ['language','test','c & NA vs iMC & A','c & NA vs c & A','iMC & NA vs iMC & A','c & NA vs iMC & NA','iMC & NA vs c & A','c & A vs iMC & A']
dictResultsAvgPerCell['language']            = ['python_statAnot'   , 'python_statAnot' ,'python'   , 'python', 'R'      , 'R'          , 'Matlab' , 'Matlab' ]
dictResultsAvgPerCell['test']                = ['ttest_ind', 'mannwhitneyu', 'ttest_ind', 'mannwhitneyu', 't.test' , 'wilcox.test', 'ttest2' , 'ranksum']
dictResultsAvgPerCell['c & NA vs iMC & NA']  = [1.000e+00  , 5.669e-01     ,0.19226082553146542  , 0.047244130659518     , 0.1503   , 0.09502       , 0.1923       , 0.0945]
dictResultsAvgPerCell['iMC & NA vs c & A']   = [9.673e-02  , 8.625e-03     ,0.016121864893694285  ,0.0007187494204219925     , 0.04082  , 0.0009726    , 0.0161     , 0.0014]
dictResultsAvgPerCell['c & A vs iMC & A']    = [2.331e-02  , 1.376e-01     ,0.00388458593467288  , 0.01146586677766893     , 0.007326 , 0.02214      , 0.0039      , 0.0229]
dictResultsAvgPerCell['c & NA vs c & A']     = [1.000e+00  , 1.000e+00     ,0.6977550928576132  , 0.2884535746840493     , 0.6948   , 0.5838       , 0.6978     , 0.5769]
dictResultsAvgPerCell['iMC & NA vs iMC & A'] = [1.000e+00  , 1.000e+00     ,0.5573451346686198  , 0.41831870120029446,  0.5031  , 0.8387       , 0.5573      , 0.8366 ]
dictResultsAvgPerCell['c & NA vs iMC & A']   = [9.726e-01  , 1.000e+00     ,0.16209530366557973  , 0.14893352365754048     , 0.04353  , 0.3043       , 0.1621       , 0.2979 ]
dfResultsAvgPerCell = pd.DataFrame(dictResultsAvgPerCell)
dfResultsAvgPerCell[order]

# dfResultsAvgPerCell = dfResultsAvgPerCell.sort_values(by='test',ascending=False)
# dfResultsAvgPerCell

### Plot small stuff

In [ ]:
SMALLER_SIZE = 6
SMALL_SIZE = 6
MEDIUM_SIZE = 10
BIGGER_SIZE = 10

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALLER_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

F0, F1 = 500, 250
h0, h1 = 200, 50
om = 2*np.pi
delta = 1
T = np.arange(0,3,0.01)
F = F1 * np.sin(2*np.pi*T) + F0
h = h1 * np.sin(2*np.pi*T - delta) + h0

fig, ax = plt.subplots(figsize = (4,2))
color = 'firebrick'
ax.plot(T, F, color = color, lw = 1)
ax.plot([np.min(T), np.max(T)], [F0, F0], 'k--', lw = 1)
ax.set_xlabel('t (s)')
ax.set_ylabel('F (pN)', color=color)
ax.tick_params(axis='y', labelcolor=color)

axbis = ax.twinx()
color = 'blue'
axbis.set_ylabel('h (nm)', color=color)
axbis.plot(T, h, color=color, lw = 1)
axbis.tick_params(axis='y', labelcolor=color)
# axbis.set_yticks([0,500,1000,1500])
# minh = np.min(tsDF['D3'].values-DIAMETER)
# ratio = min(1/abs(minh/axM), 5)
# #             print(ratio)
#             (axmbis, axMbis) = ax1bis.get_ylim()
#             ax1bis.set_ylim([0, max(axMbis*ratio, 3*max(tsDF['F'].values))])

## Old Code

### Tweaked version of analyseTimeSeries_meca to plot specific stuff

In [ ]:
def analyseTimeSeries_meca_tweaked(f, tsDF, expDf, listColumnsMeca, PLOT, PLOT_SHOW):
    
    split_f = f.split('_')
    tsDF.dx, tsDF.dy, tsDF.dz, tsDF.D2, tsDF.D3 = tsDF.dx*1000, tsDF.dy*1000, tsDF.dz*1000, tsDF.D2*1000, tsDF.D3*1000
    thisManipID = split_f[0] + '_' + split_f[1]
    expDf['manipID'] = expDf['date'] + '_' + expDf['manip']
    thisExpDf = expDf.loc[expDf['manipID'] == thisManipID]
    DIAMETER = thisExpDf.at[thisExpDf.index.values[0], 'bead diameter']
    
    results = {}
    for c in listColumnsMeca:
        results[c] = []
        
    Ncomp = max(tsDF['idxCompression'])
    NimgComp = np.sum((tsDF['idxCompression'] != 0))/Ncomp
    NimgCompTh = round(0.49999999999 + np.sum((tsDF['idxCompression'] != 0))/Ncomp)
    NimgBtwComp = np.sum((tsDF['idxCompression'] == 0))/Ncomp
    NimgBtwCompTh = round(0.4999999999 + np.sum((tsDF['idxCompression'] == 0))/Ncomp)
#     print('Ncomp : ' + str(Ncomp) + ' ; ' + 'NimgComp : ' + str(NimgComp) + '/' + str(NimgCompTh) + ' ; ' + 'NimgBtwComp : ' + str(NimgBtwComp) + '/' + str(NimgBtwCompTh))
#     if not NimgBtwComp%2 == 0:
#         print('Bug with the compressions sequence delimitation')

    # These values are computed once for the whole cell D3 time series, but since the table has 1 line per compression, 
    # that same value will be put in the table for each line corresponding to that cell
    ctFieldH = (tsDF.loc[tsDF['idxCompression'] == 0, 'D3'].values - DIAMETER)
    ctFieldThickness   = np.median(ctFieldH)
    ctFieldFluctuAmpli = np.percentile(ctFieldH,90) - np.percentile(ctFieldH,10)
    
    # First part of the plot [mainly ax1 and ax1bis]
    if PLOT:
        nColsSubplot = 5
        nRowsSubplot = ((Ncomp-1) // nColsSubplot) + 1
        fig1, ax1 = plt.subplots(1,1,figsize=(tsDF.shape[0]*(1/100)*0.25,4))
#         fig2, ax2 = plt.subplots(1,Ncomp,figsize=(3*Ncomp,3))
        fig2, ax2 = plt.subplots(nRowsSubplot,nColsSubplot,figsize=(3*nColsSubplot,3*nRowsSubplot))
        
        color = 'blue'
#         ax1.plot(tsDF['T'].values, tsDF['D3'].values-DIAMETER, color = color, ls = '-', linewidth = 1)
        ax1.set_xlabel('t (s)')
#         ax1.set_ylabel('h (nm)', color=color)
#         ax1.tick_params(axis='y', labelcolor=color)
#         (axm, axM) = ax1.get_ylim()
#         ax1.set_ylim([min(0,axm), axM-50])
        ax1.set_xlim([0, 45])
#         if (max(tsDF['D3'].values-DIAMETER) > 200):
#             ax1.set_yticks(np.arange(0, max(tsDF['D3'].values-DIAMETER), 100))
        ax1.set_yticks([])
        
        twinAxis = True
        if twinAxis:
#             ax1.tick_params(axis='y', labelcolor='b')
            ax1bis = ax1.twinx()
            color = 'purple'
            ax1bis.set_ylabel('B (mT)', color=color)
            ax1bis.plot(tsDF['T'].values, tsDF['B'].values, color=color)
            ax1bis.tick_params(axis='y', labelcolor=color)
            ax1bis.set_yticks([0,3,6,10,20,30,40])
#             ax1bis.set_yticks([0,500,1000,1500])
#             minh = np.min(tsDF['D3'].values-DIAMETER)
#             ratio = min(1/abs(minh/axM), 5)
#             print(ratio)
#             (axmbis, axMbis) = ax1bis.get_ylim()
#             ax1bis.set_ylim([0, max(axMbis*ratio, 3*max(tsDF['F'].values))])
            yt = ax1bis.get_yticklabels()
            for item in yt:
                item.set_fontsize(11)
            
    
    for i in range(1, Ncomp+1):#Ncomp+1):

        # (1) Identifiers
        results['date'].append(split_f[0])
        results['cellName'].append(split_f[1] + '_' + split_f[2] + '_' + split_f[3])
        results['cellID'].append(split_f[0] + '_' + split_f[1] + '_' + split_f[2] + '_' + split_f[3])
        results['manipID'].append(split_f[0] + '_' + split_f[1])
        
        # (2) Segment the compression n°i
        thisCompDf = tsDF.loc[tsDF['idxCompression'] == i,:]
        iStart = (findFirst(tsDF['idxCompression'], i))
        iStop = iStart+thisCompDf.shape[0]
        
        # Easy-to-get parameters
        results['compNum'].append(i)
        results['compDuration'].append(thisExpDf.at[thisExpDf.index.values[0], 'compression duration'])
        results['compStartTime'].append(thisCompDf['T'].values[0])
        results['compAbsStartTime'].append(thisCompDf['Tabs'].values[0])
        results['compStartTimeThisDay'].append(thisCompDf['Tabs'].values[0]) # Will be modifyed later !
        
        # (3) Inside the compression n°i, delimit the compression and relaxation phases
        
        # Delimit the start of the increase of B (typically the moment when the field decrease from 5 to 3)
        # and the end of its decrease (typically when it goes back from 3 to 5)
        listB = thisCompDf.B.values
        offsetStart, offsetStop = 0, 0
        minB, maxB = min(listB), max(listB)
        thresholdB = (maxB-minB)/50
        
        k = 0
        while (listB[k] > minB+thresholdB) or (listB[-1-k] > minB+thresholdB):
            offsetStart += int(listB[k] > minB+thresholdB)
            offsetStop += int(listB[-1-k] > minB+thresholdB)
            k += 1
        jStart = offsetStart # Beginning of compression
        jMax = np.argmax(thisCompDf.B) # End of compression, beginning of relaxation
        jStop = thisCompDf.shape[0] - offsetStop # End of relaxation
        
        # Four arrays
        hCompr = (thisCompDf.D3.values[jStart:jMax+1] - DIAMETER)
        hRelax = (thisCompDf.D3.values[jMax+1:jStop] - DIAMETER)
        fCompr = (thisCompDf.F.values[jStart:jMax+1])
        fRelax = (thisCompDf.F.values[jMax+1:jStop])
        
        # Refinement of the compression delimitation.
        # Remove the 1-2 points at the begining where there is just the viscous relaxation of the cortex
        # because of the initial decrease of B and the cortex thickness increases.
        offsetStart2 = 0
        k = 0
        while (hCompr[k] < np.max(hCompr[k+1:min(k+10, len(hCompr))])) and k<len(hCompr)-10:
            offsetStart2 += 1
            k += 1
        # Better compressions arrays
        hCompr = hCompr[offsetStart2:]
        fCompr = fCompr[offsetStart2:]
        
        # Get the points of constant field preceding and surrounding the current compression
        # Ex : if the labview code was set so that there is 6 points of ct field before and after each compression,
        # previousPoints will contains D3[iStart-12:iStart]
        # surroundingPoints will contains D3[iStart-6:iStart] and D3[iStop:iStop+6]
        previousPoints = (tsDF.D3.values[max(0,iStart-(NimgBtwCompTh)):iStart]) - DIAMETER
        surroundingPoints = np.concatenate([tsDF.D3.values[max(0,iStart-(NimgBtwCompTh//2)):iStart],tsDF.D3.values[iStop:iStop+(NimgBtwCompTh//2)]]) - DIAMETER
        
        # Parameters relative to the thickness ( = D3-DIAMETER)
        results['initialThickness'].append(np.mean(hCompr[0:3]))
        results['minThickness'].append(np.min(hCompr))
        results['maxIndent'].append(results['initialThickness'][-1] - results['minThickness'][-1])
        results['previousThickness'].append(np.median(previousPoints))
        results['surroundingThickness'].append(np.median(surroundingPoints))
        results['ctFieldThickness'].append(ctFieldThickness)
        results['ctFieldFluctuAmpli'].append(ctFieldFluctuAmpli)
        
        validatedThickness = np.min([results['initialThickness'],results['minThickness'],results['previousThickness'],\
                                    results['surroundingThickness'],results['ctFieldThickness']]) > 0
        results['validatedThickness'].append(validatedThickness)

        # (4) Fit with Chadwick model of the force-thickness curve
        
        E, H0, hPredict, R2, confIntE, confIntH0, fitError = compressionFitChadwick(hCompr, fCompr, DIAMETER) # IMPORTANT SUBFUNCTION
        
        R2CRITERION = 0.9
        critFit = 'R2 > ' + str(R2CRITERION)
        results['critFit'].append(critFit)
        validatedFit = (R2 > R2CRITERION)
        
        if PLOT:

            # fig1
#             if not fitError:
#                 if validatedFit:
#                     ax1.plot(thisCompDf['T'].values, thisCompDf['D3'].values-DIAMETER, color = 'chartreuse', linestyle = '-', linewidth = 1.25)
#                 else:
#                     ax1.plot(thisCompDf['T'].values, thisCompDf['D3'].values-DIAMETER, color = 'gold', linestyle = '-', linewidth = 1.25)
#             else:
#                 ax1.plot(thisCompDf['T'].values, thisCompDf['D3'].values-DIAMETER, color = 'crimson', linestyle = '-', linewidth = 1.25)

            
            fig1.suptitle(results['cellID'][-1])
            
            # fig2
            colSp = (i-1) % nColsSubplot
            rowSp = (i-1) // nColsSubplot
            # ax2[i-1] with the 1 line plot
            if nRowsSubplot == 1:
                thisAx2 = ax2[colSp]
            elif nRowsSubplot >= 1:
                thisAx2 = ax2[rowSp,colSp]
            
            thisAx2.plot(hCompr,fCompr,'b-', linewidth = 0.8)
            thisAx2.plot(hRelax,fRelax,'r-', linewidth = 0.8)
            titleText = results['cellID'][-1] + '__c' + str(i)
            legendText = ''
            thisAx2.set_xlabel('h (nm)')
            thisAx2.set_ylabel('f (pN)')
            
            if not fitError:
                legendText += 'H0 = {:.1f}nm\nE = {:.2e}Pa\nR2 = {:.3f}'.format(H0, E, R2)
                thisAx2.plot(hPredict,fCompr,'k--', linewidth = 0.8, label = legendText)
                thisAx2.legend(loc = 'upper right', prop={'size': 6})
                if not validatedFit:
                    titleText += '\nNON VALIDATED'
            else:
                titleText += '\nFIT ERROR'
            
            thisAx2.title.set_text(titleText)

            for item in ([thisAx2.title, thisAx2.xaxis.label, \
                          thisAx2.yaxis.label] + thisAx2.get_xticklabels() + thisAx2.get_yticklabels()):
                item.set_fontsize(9)
                

        if not fitError:
            confIntEWidth = abs(confIntE[0] - confIntE[1])

            results['H0Chadwick'].append(H0)
            results['EChadwick'].append(E)
            results['R2Chadwick'].append(R2)
            results['EChadwick_CIWidth'].append(confIntEWidth)
            

            results['validatedFit'].append(validatedFit)
            if validatedFit:
                results['comments'].append('ok')
            else:
                results['comments'].append('R2 < ' + str(R2CRITERION))
                
        if fitError:
            validatedFit = False
            results['H0Chadwick'].append(np.nan)
            results['EChadwick'].append(np.nan)
            results['R2Chadwick'].append(np.nan)
            results['EChadwick_CIWidth'].append(np.nan)
            results['validatedFit'].append(validatedFit)
            results['comments'].append('fitFailure')
            
        
        # (5) hysteresis (its definition may change)
        results['hysteresis'].append(hCompr[0] - hRelax[-1])
    
    if PLOT:
        archiveFig(fig1, ax1, name=results['cellID'][-1] + '_h(t)', figSubDir = 'MecaAnalysis_allCells')
        archiveFig(fig2, ax2, name=results['cellID'][-1] + '_F(h)', figSubDir = 'MecaAnalysis_allCells')
        if PLOT_SHOW:
            fig1.show()
            fig2.tight_layout()
            fig2.show()
        else:
            plt.close('all')
    
    return(results)

### Data formatting and filtering

TBC

In [ ]:
mecaDataFile = 'Global_MecaData.csv'
mecaDataFilePath = os.path.join(dataDir, mecaDataFile)
mecaDF = pd.read_csv(mecaDataFilePath, sep=';')
print('Extracted a table with ' + str(mecaDF.shape[0]) + ' lines and ' + str(mecaDF.shape[1]) + ' columns.')

mecaDF = mecaDF.rename(columns={"CellID": "CellName", "CellName": "CellID"})

In [ ]:
# mecaDF

In [ ]:
experimentalDataFile = 'ExperimentalConditions.csv'
experimentalDataFilePath = os.path.join(experimentalDataDir, experimentalDataFile)
expConditionsDF = pd.read_csv(experimentalDataFilePath, sep=';',header=0)
print('Extracted a table with ' + str(expConditionsDF.shape[0]) + ' lines and ' + str(expConditionsDF.shape[1]) + ' columns.')

# Cleaning the table
try:
    expConditionsDF = expConditionsDF.convert_dtypes()

    listTextColumns = []
    for col in expConditionsDF.columns:
        if expConditionsDF[col].dtype == 'string':
            listTextColumns.append(col)

    expConditionsDF[listTextColumns] = expConditionsDF[listTextColumns].apply(lambda x: x.str.replace(',','.'))

    expConditionsDF['scale pixel per um'] = expConditionsDF['scale pixel per um'].astype(float)
    expConditionsDF['optical index correction'] = \
              expConditionsDF['optical index correction'].apply(lambda x: x.split('/')[0]).astype(float) \
            / expConditionsDF['optical index correction'].apply(lambda x: x.split('/')[1]).astype(float)
    expConditionsDF['magnetic field correction'] = expConditionsDF['magnetic field correction'].astype(float)
    expConditionsDF['with fluo images'] = expConditionsDF['with fluo images'].astype(bool)

    expConditionsDF['ramp field'] = \
    expConditionsDF['ramp field'].apply(lambda x: [x.split(';')[0], x.split(';')[1]] if not pd.isnull(x) else [])

except:
    print('Unexpected bug with the cleaning step')

In [ ]:
expConditionsDF

In [ ]:
# Unused for now
cellDescriptionDataFile = 'CellDescription.csv'
cellDescriptionDataFilePath = os.path.join(experimentalDataDir, cellDescriptionDataFile)
cellDescriptionDF = pd.read_csv(cellDescriptionDataFilePath, ',')
print('Extracted a table with ' + str(cellDescriptionDF.shape[0]) + ' lines and ' + str(cellDescriptionDF.shape[1]) + ' columns.')

In [ ]:
mecaDF['ManipID'] = mecaDF['ExpDay'] + '_' + mecaDF['CellName'].apply(lambda x: x.split('_')[0])
expConditionsDF['ManipID'] = expConditionsDF['date'] + '_' + expConditionsDF['manip']

mainMecaDF = pd.merge(
    expConditionsDF,
    mecaDF,
    how="inner",
    on='ManipID',
#     left_on=None,
#     right_on=None,
#     left_index=False,
#     right_index=False,
#     sort=True,
#     suffixes=("_x", "_y"),
#     copy=True,
#     indicator=False,
#     validate=None,
)

In [ ]:
# pd.set_option('display.max_columns', None)
# mainMecaDF.head()

In [ ]:
pd.reset_option('max_columns')

#### * Data filtering

In [ ]:
# pd.set_option('display.max_columns', None)
# mainMecaDF.head()

In [ ]:
# pd.reset_option('max_columns')
# mainMecaDF.columns

In [ ]:
mainMecaDF_f = mainMecaDF.loc[(mainMecaDF["Validated"] == 1)]
# mainMecaDF_f

In [ ]:
%matplotlib inline
listCells = mainMecaDF_f['CellID'].drop_duplicates().astype('string').values
timeSeriesDict = {}
for cell in listCells:
    currentCell_TimeSeriesData = getCellTimeSeriesData(cell)
    timeSeriesDict[cell] = currentCell_TimeSeriesData
start, stop = 80, 100
fig, axes = plt.subplots((stop-start),1, figsize = (7,4*(stop-start)))
fig.tight_layout()
for k in range(start, stop):
    if k < len(listCells):
        currentCell_TimeSeriesData = timeSeriesDict[listCells[k]]
        T = currentCell_TimeSeriesData['T'].values
        idxCompression = currentCell_TimeSeriesData['idxCompression'].values
        D3 = currentCell_TimeSeriesData['D3'].values
        maskConstant = (idxCompression == 0)
        maskCompression = (idxCompression > 0)
        axes[k - start].plot(T, D3*1000-4503, 'k-', linewidth = 0.5)
        axes[k - start].plot(T[maskCompression], D3[maskCompression]*1000-4503, 'ro', markersize=2)
        axes[k - start].plot(T[maskConstant], D3[maskConstant]*1000-4503, 'co', markersize=2)
        axes[k - start].set_title(listCells[k])
        axes[k - start].set_xlabel('T (s)')
        axes[k - start].set_ylabel('D3 (µm)')

In [ ]:
addExcludedCell('21-01-18_M1_P1_C2', 'passive')
addExcludedCell('21-01-18_M1_P1_C3', 'passive')
addExcludedCell('21-01-18_M1_P1_C4', 'passive')
addExcludedCell('21-01-21_M3_P1_C4', 'passive')
addExcludedCell('21-01-21_M3_P1_C5', 'passive')
addExcludedCell('20-08-07_M1_P1_C6', 'too thick')
addExcludedCell('20-08-07_M1_P1_C62', 'too thick')

excludedCellsDict = getExcludedCells()
# # excludedMask = (mainMecaDF_f["CellID"].values not in excludedCellsDict.keys())
# # mainMecaDF_f = mainMecaDF_f.loc[(mainMecaDF_f["CellID"].values not in excludedCellsDict.keys())]
# for i in range(len(excludedCellsDict)):
#     print('a')
# mainMecaDF_f["CellID"].drop_duplicates().astype('string').values
excludedCellsDict

In [ ]:
# currentCell_TimeSeriesData

In [ ]:
mainMecaDF_GroupedPerCell = mainMecaDF_f.groupby('CellID')
mainMecaDF_DataPerCell = mainMecaDF_GroupedPerCell.agg({"EChadwick": np.median, "SurroundingThickness": np.median, "H0Chadwick" : np.median})
# mainMecaDF_GroupedPerCell.agg({"EChadwick": np.median, "D": lambda x: np.std(x, ddof=1)})
cols = ['date', 'manip', 'experimentType', 'drug', 'substrate',
       'objective magnification', 'scale pixel per um', 'objective immersion',
       'optical index correction', 'magnetic field correction', 'cell type',
       'cell subtype', 'bead type', 'bead diameter', 'normal field',
       'ramp field', 'compression duration', 'with fluo images', 'comments',
       'ManipID', 'ExpType', 'CellName', 'CellID']
mainMecaDF_DataPerCell.dropna(inplace = True)
mainMecaDF_DataPerCell = pd.merge(mainMecaDF_DataPerCell,
                                  mainMecaDF_f[cols].drop_duplicates(subset=['CellID']),
                                  how="inner",
                                  on='CellID',
                                  #     left_on='CellID',
                                  #     right_on='CellID',
                                  #     left_index=False,
                                  #     right_index=False,
                                  #     sort=True,
                                  #     suffixes=("_x", "_y"),
                                  #     copy=True,
                                  #     indicator=False,
                                  #     validate=None,
                                  )
# mainMecaDF_DataPerCell

In [ ]:
GlobalTable_meca_Count = GlobalTable_meca.groupby(['cell type', 'cell subtype', 'bead type', 'drug', 'substrate']).count()
GlobalTable_meca_Count.loc[:, ['CellID']].rename(columns={'CellID' : 'Count'})

### Other old code

In [ ]:
# fig, axes = plt.subplots(1,2, figsize = (8, 5))
# axes[0].plot(np.ones(len(resDict['nodrug']['median'])), resDict['nodrug']['median'], 'co')
# axes[0].plot(2*np.ones(len(resDict['doxy']['median'])), resDict['doxy']['median'], 'ro')
# axes[0].set_xlim(0.5, 2.5)
# axes[0].set_ylabel('Median Thickness (nm)')
# axes[0].set_xticks([1,2])
# axes[0].set_xticklabels(['Control','Doxycylin'])
# axes[1].plot(np.ones(len(resDict['nodrug']['fluctu'])), resDict['nodrug']['fluctu'], 'co')
# axes[1].plot(2*np.ones(len(resDict['doxy']['fluctu'])), resDict['doxy']['fluctu'], 'ro')
# axes[1].set_xlim(0.5, 2.5)
# axes[1].set_ylabel('Thickness Fluctuations (nm)')
# axes[1].set_xticks([1,2])
# axes[1].set_xticklabels(['Control','Doxycylin'])
# fig.savefig("C://Users//JosephVermeil//Desktop//ActinCortexAnalysis//DataAnalysis//constantField.png")
# fig.show()

In [ ]:
# Old code
conditions = ['nodrug', 'doxy']
correspondance = {conditions[0] : 'M1', conditions[1] : 'M2'}
allTimeSeriesDataFiles = [f for f in os.listdir(timeSeriesDataDir) if (os.path.isfile(os.path.join(timeSeriesDataDir, f)) and f.endswith(".txt"))]
dates = ['21-02-10']
resDict = {conditions[0] : {}, conditions[1] : {}}
for C in conditions:
    resDict[C]['accepted'] = []
    resDict[C]['rejected'] = []
    resDict[C]['median'] = []
    resDict[C]['fluctu'] = []
    for D in dates:
        for f in allTimeSeriesDataFiles:
            if correspondance[C] in f and D in f:
                split_f = f.split('_')
                cellID = split_f[0] + '_' + split_f[1] + '_' + split_f[2] + '_' + split_f[3]
                currentCellTS = getCellTimeSeriesData(cellID)
                D3 = currentCellTS.D3.values
                decile_1 = np.percentile(D3, 10)
                median = np.median(D3)
                decile_9 = np.percentile(D3, 90)
                if decile_1 < 0:
                    resDict[C]['rejected'].append(cellID)
                else:
                    resDict[C]['accepted'].append(cellID)
                    resDict[C]['median'].append(median)
                    resDict[C]['fluctu'].append(decile_9-decile_1)
#resDict

In [ ]:
GlobalTable_ctField = getGlobalTable_ctField()
table_ExpConditions = getExperimentalConditions()
GlobalTable_ctField = pd.merge(table_ExpConditions, GlobalTable_ctField, how="inner", on='manipID',
#     left_on=None,right_on=None,left_index=False,right_index=False,sort=True,
#     suffixes=("_x", "_y"),copy=True,indicator=False,validate=None,
)

# data=pd.DataFrame(dict(
#             x=[1, 2, 3, 4, 5],
#             y=[2, 5, 8, 2, 7],
#             desc=['A', 'A', 'C', 'd', 'E'],
#         ))

data = GlobalTable_ctField[['medianThickness','fluctuAmpli','cellID']]

source = ColumnDataSource(
        data=data
    )

# hover = HoverTool(
#         tooltips=[
#             ("index", "$index"),
#             ("(x,y)", "($x, $y)"),
#             ("desc", "@desc"),
#         ]
#     )

hover = HoverTool(
        tooltips=[
            ("medianThickness", "@medianThickness"),
            ("fluctuAmpli", "@fluctuAmpli"),
        ]
    )

p = figure(plot_width=300, plot_height=300, tools=[hover], title="Mouse over the dots")

p.circle('medianThickness', 'fluctuAmpli', size=20, source=data)

show(p)

data

In [ ]:
data=pd.DataFrame(dict(
            x=[1, 2, 3, 4, 5],
            y=[2, 5, 8, 2, 7],
            desc=['A', 'A', 'C', 'd', 'E'],
        ))

Conditions = list(data['desc'].unique())
NCond = len(Conditions)
data['X'] = 0
for i in range(NCond):
    mask = data['desc'] == Conditions[i]
    data.loc[mask, ['X']] = i+1
data.index = data.x
data = data.drop(['x'], axis = 1)

data.reset_index(level=0, inplace=True)
data